In [1]:
import os
from datetime import datetime
import time

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

In [2]:
# Enable/Disable Eager Execution
# Reference: https://www.tensorflow.org/guide/eager
# TensorFlow's eager execution is an imperative programming environment that evaluates operations immediately, 
# without building graphs

#tf.compat.v1.disable_eager_execution()
#tf.compat.v1.enable_eager_execution()

print("tensorflow version", tf.__version__)
print("keras version", tf.keras.__version__)
print("Eager Execution Enabled:", tf.executing_eagerly())

# Get the number of replicas 
strategy = tf.distribute.MirroredStrategy()
print("Number of replicas:", strategy.num_replicas_in_sync)

devices = tf.config.experimental.get_visible_devices()
print("Devices:", devices)
print(tf.config.experimental.list_logical_devices('GPU'))

print("GPU Available: ", tf.config.list_physical_devices('GPU'))
print("All Physical Devices", tf.config.list_physical_devices())

# Better performance with the tf.data API
# Reference: https://www.tensorflow.org/guide/data_performance
AUTOTUNE = tf.data.experimental.AUTOTUNE

tensorflow version 2.7.0
keras version 2.7.0
Eager Execution Enabled: True
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of replicas: 1
Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[LogicalDevice(name='/device:GPU:0', device_type='GPU')]
GPU Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
All Physical Devices [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


COLUMNS:
- Open Time: Candle Open Time
- Open Price: Open Price in Quote (Secondary) Asset (USDT in the DF below)
- High Price: High Price in Quote (Secondary) Asset (USDT in the DF below)
- Low Price: Low Price in Quote (Secondary) Asset (USDT in the DF below)
- Close Price: Close Price in Quote (Secondary) Asset (USDT in the DF below)
- Volume Traded: Total Trade Volume in Primary (Base) Asset Units (BTC in the DF below)
- Close Time: Candle Close Time
- Quote Asset Volume: Total Trade Volume in Quote (Secondary) Asset Units (USDT in the DF below)
- Number of Trades: Total Number of Trades
- Taker Buy Base Asset Volume: Number of Trades in Primary (Base) Asset Resulting from a Taker Matching an Existing Order. In other words, it separates the 'Volume Traded Feature' by the amount traded as a result of orders baing placed in the order book and orders being matched in the order book (might need to revisit this, could provide additional interesting insights, if we have a better understanding of trading strategies). Volume Traded = Taker Buy Base Asset Volume + Maker Buy Base Asset Volume
- Taker Buy Quote Asset Volume: Number of Trades in Quote (Secondary) Asset Resulting from a Taker Mathcing an Existing Order. Explanation similar to the above, only difference the volume is calculated in Quote (Secondary) Asset units. Quote Asset Volume = Takes Buy Quote Asser Volume + Maker Buy Quote Asset Volume
- NA: Safe to Ignore

# START HERE

In [5]:
df = pd.read_csv('/content/gdrive/MyDrive/playground/btcusdt_data.csv', index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


# TEMPLATE CODE

In [ ]:
import os

data_folder = '../Data'
file_name = 'btcusdt_exchange_full.csv'

#location_data = os.path.join(data_folder, file_name)
location_data = '/content/gdrive/MyDrive/playground/btcusdt_data.csv'

exclude_x = ['NA', 'Open Time', 'Close Time', 'y', 'Date']

val_split, test_split = None, None

# log_transformation_feat = ['Open Price', 'High price', 'Low Price', 'Close Price', 'Volume Traded',
#  'Quote asset Volume', 'Number of Trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Mid Price', 'Open Price_prev', 'Close Price_prev', 'Mid Price_prev', 'high_low', 'high_close', 'high_open', 'open_low', 'spread',
#  'rolling_avg_high_low_1h', 'rolling_avg_high_low_1d', 'rolling_avg_high_low_1w', 'rolling_avg_high_low_1m', 'rolling_max_high_low_1h',
#  'rolling_max_high_low_1d', 'rolling_max_high_low_1w', 'rolling_max_high_low_1m', 'rolling_min_high_low_1h', 'rolling_min_high_low_1d',
#  'rolling_min_high_low_1w', 'rolling_min_high_low_1m', 'rolling_avg_high_close_1h', 'rolling_avg_high_close_1d', 'rolling_avg_high_close_1w', 'rolling_avg_high_close_1m', 'rolling_max_high_close_1h', 'rolling_max_high_close_1d', 'rolling_max_high_close_1w', 'rolling_max_high_close_1m', 'rolling_min_high_close_1h', 'rolling_min_high_close_1d', 'rolling_min_high_close_1w', 'rolling_min_high_close_1m', 'rolling_avg_high_open_1h', 'rolling_avg_high_open_1d', 'rolling_avg_high_open_1w', 'rolling_avg_high_open_1m', 'rolling_max_high_open_1h', 'rolling_max_high_open_1d',
#  'rolling_max_high_open_1w', 'rolling_max_high_open_1m', 'rolling_min_high_open_1h', 'rolling_min_high_open_1d',
#  'rolling_min_high_open_1w', 'rolling_min_high_open_1m', 'rolling_avg_open_low_1h', 'rolling_avg_open_low_1d',
#  'rolling_avg_open_low_1w', 'rolling_avg_open_low_1m', 'rolling_max_open_low_1h', 'rolling_max_open_low_1d',
#  'rolling_max_open_low_1w', 'rolling_max_open_low_1m', 'rolling_min_open_low_1h', 'rolling_min_open_low_1d',
#  'rolling_min_open_low_1w', 'rolling_min_open_low_1m', 'rolling_avg_spread_1h', 'rolling_avg_spread_1d',
#  'rolling_avg_spread_1w', 'rolling_avg_spread_1m', 'rolling_max_spread_1h', 'rolling_max_spread_1d', 'rolling_max_spread_1w',
#  'rolling_max_spread_1m', 'rolling_min_spread_1h', 'rolling_min_spread_1d', 'rolling_min_spread_1w', 'rolling_min_spread_1m',
#  'rolling_avg_Open Price_1h', 'rolling_avg_Open Price_1d', 'rolling_avg_Open Price_1w', 'rolling_avg_Open Price_1m',
#  'rolling_max_Open Price_1h', 'rolling_max_Open Price_1d', 'rolling_max_Open Price_1w', 'rolling_max_Open Price_1m',
#  'rolling_min_Open Price_1h', 'rolling_min_Open Price_1d', 'rolling_min_Open Price_1w', 'rolling_min_Open Price_1m',
#  'rolling_avg_High price_1h', 'rolling_avg_High price_1d', 'rolling_avg_High price_1w', 'rolling_avg_High price_1m',
#  'rolling_max_High price_1h', 'rolling_max_High price_1d', 'rolling_max_High price_1w', 'rolling_max_High price_1m',
#  'rolling_min_High price_1h', 'rolling_min_High price_1d', 'rolling_min_High price_1w', 'rolling_min_High price_1m',
#  'rolling_avg_Low Price_1h', 'rolling_avg_Low Price_1d', 'rolling_avg_Low Price_1w', 'rolling_avg_Low Price_1m',
#  'rolling_max_Low Price_1h', 'rolling_max_Low Price_1d', 'rolling_max_Low Price_1w', 'rolling_max_Low Price_1m',
#  'rolling_min_Low Price_1h', 'rolling_min_Low Price_1d', 'rolling_min_Low Price_1w', 'rolling_min_Low Price_1m',
#  'rolling_avg_Close Price_1h', 'rolling_avg_Close Price_1d', 'rolling_avg_Close Price_1w', 'rolling_avg_Close Price_1m',
#  'rolling_max_Close Price_1h', 'rolling_max_Close Price_1d', 'rolling_max_Close Price_1w', 'rolling_max_Close Price_1m',
#  'rolling_min_Close Price_1h', 'rolling_min_Close Price_1d', 'rolling_min_Close Price_1w', 'rolling_min_Close Price_1m']
# features where we apply log transformation


log_transformation_feat = ['Open Price', 'High price', 'Low Price', 'Close Price', 'Volume Traded',
                           'Quote asset Volume', 'Number of Trades', 'Taker buy base asset volume',
                           'Taker buy quote asset volume', 'Mid Price', 'Open Price_prev', 'Close Price_prev',
                           'Mid Price_prev', 'high_low', 'high_close', 'high_open', 'open_low', 'spread',
                           'rolling_avg_high_low_1h', 'rolling_avg_high_low_1d', 'rolling_avg_high_low_1w',
                           'rolling_avg_high_low_1m', 'rolling_max_high_low_1h',
                           'rolling_max_high_low_1d', 'rolling_max_high_low_1w', 'rolling_max_high_low_1m',
                           'rolling_avg_high_close_1h', 'rolling_avg_high_close_1d', 'rolling_avg_high_close_1w',
                           'rolling_avg_high_close_1m', 'rolling_max_high_close_1h', 'rolling_max_high_close_1d',
                           'rolling_max_high_close_1w', 'rolling_max_high_close_1m', 'rolling_avg_high_open_1h',
                           'rolling_avg_high_open_1d', 'rolling_avg_high_open_1w', 'rolling_avg_high_open_1m',
                           'rolling_max_high_open_1h', 'rolling_max_high_open_1d',
                           'rolling_max_high_open_1w', 'rolling_max_high_open_1m', 'rolling_avg_open_low_1h',
                           'rolling_avg_open_low_1d',
                           'rolling_avg_open_low_1w', 'rolling_avg_open_low_1m', 'rolling_max_open_low_1h',
                           'rolling_max_open_low_1d',
                           'rolling_max_open_low_1w', 'rolling_max_open_low_1m', 'rolling_avg_spread_1h',
                           'rolling_avg_spread_1d',
                           'rolling_avg_spread_1w', 'rolling_avg_spread_1m', 'rolling_max_spread_1h',
                           'rolling_max_spread_1d', 'rolling_max_spread_1w',
                           'rolling_max_spread_1m', 'rolling_avg_Open Price_1h', 'rolling_avg_Open Price_1d',
                           'rolling_avg_Open Price_1w', 'rolling_avg_Open Price_1m',
                           'rolling_max_Open Price_1h', 'rolling_max_Open Price_1d', 'rolling_max_Open Price_1w',
                           'rolling_max_Open Price_1m',
                           'rolling_avg_High price_1h', 'rolling_avg_High price_1d', 'rolling_avg_High price_1w',
                           'rolling_avg_High price_1m',
                           'rolling_max_High price_1h', 'rolling_max_High price_1d', 'rolling_max_High price_1w',
                           'rolling_max_High price_1m',
                           'rolling_avg_Low Price_1h', 'rolling_avg_Low Price_1d', 'rolling_avg_Low Price_1w',
                           'rolling_avg_Low Price_1m',
                           'rolling_max_Low Price_1h', 'rolling_max_Low Price_1d', 'rolling_max_Low Price_1w',
                           'rolling_max_Low Price_1m',
                           'rolling_avg_Close Price_1h', 'rolling_avg_Close Price_1d', 'rolling_avg_Close Price_1w',
                           'rolling_avg_Close Price_1m',
                           'rolling_max_Close Price_1h', 'rolling_max_Close Price_1d', 'rolling_max_Close Price_1w',
                           'rolling_max_Close Price_1m']
# features where we apply log transformation


continuous_features = ['Open Price', 'High price', 'Low Price', 'Close Price', 'Volume Traded',
                       'Quote asset Volume', 'Number of Trades', 'Taker buy base asset volume',
                       'Taker buy quote asset volume', 'Mid Price', 'Open Price_prev', 'Close Price_prev',
                       'Mid Price_prev', 'high_low', 'high_close', 'high_open', 'open_low', 'spread',
                       'rolling_avg_high_low_1h', 'rolling_avg_high_low_1d', 'rolling_avg_high_low_1w',
                       'rolling_avg_high_low_1m', 'rolling_max_high_low_1h',
                       'rolling_max_high_low_1d', 'rolling_max_high_low_1w', 'rolling_max_high_low_1m',
                       'rolling_min_high_low_1h', 'rolling_min_high_low_1d',
                       'rolling_min_high_low_1w', 'rolling_min_high_low_1m', 'rolling_avg_high_close_1h',
                       'rolling_avg_high_close_1d', 'rolling_avg_high_close_1w', 'rolling_avg_high_close_1m',
                       'rolling_max_high_close_1h', 'rolling_max_high_close_1d', 'rolling_max_high_close_1w',
                       'rolling_max_high_close_1m', 'rolling_min_high_close_1h', 'rolling_min_high_close_1d',
                       'rolling_min_high_close_1w', 'rolling_min_high_close_1m', 'rolling_avg_high_open_1h',
                       'rolling_avg_high_open_1d', 'rolling_avg_high_open_1w', 'rolling_avg_high_open_1m',
                       'rolling_max_high_open_1h', 'rolling_max_high_open_1d',
                       'rolling_max_high_open_1w', 'rolling_max_high_open_1m', 'rolling_min_high_open_1h',
                       'rolling_min_high_open_1d',
                       'rolling_min_high_open_1w', 'rolling_min_high_open_1m', 'rolling_avg_open_low_1h',
                       'rolling_avg_open_low_1d',
                       'rolling_avg_open_low_1w', 'rolling_avg_open_low_1m', 'rolling_max_open_low_1h',
                       'rolling_max_open_low_1d',
                       'rolling_max_open_low_1w', 'rolling_max_open_low_1m', 'rolling_min_open_low_1h',
                       'rolling_min_open_low_1d',
                       'rolling_min_open_low_1w', 'rolling_min_open_low_1m', 'rolling_avg_spread_1h',
                       'rolling_avg_spread_1d',
                       'rolling_avg_spread_1w', 'rolling_avg_spread_1m', 'rolling_max_spread_1h',
                       'rolling_max_spread_1d', 'rolling_max_spread_1w',
                       'rolling_max_spread_1m', 'rolling_min_spread_1h', 'rolling_min_spread_1d',
                       'rolling_min_spread_1w', 'rolling_min_spread_1m',
                       'rolling_avg_Open Price_1h', 'rolling_avg_Open Price_1d', 'rolling_avg_Open Price_1w',
                       'rolling_avg_Open Price_1m',
                       'rolling_max_Open Price_1h', 'rolling_max_Open Price_1d', 'rolling_max_Open Price_1w',
                       'rolling_max_Open Price_1m',
                       'rolling_min_Open Price_1h', 'rolling_min_Open Price_1d', 'rolling_min_Open Price_1w',
                       'rolling_min_Open Price_1m',
                       'rolling_avg_High price_1h', 'rolling_avg_High price_1d', 'rolling_avg_High price_1w',
                       'rolling_avg_High price_1m',
                       'rolling_max_High price_1h', 'rolling_max_High price_1d', 'rolling_max_High price_1w',
                       'rolling_max_High price_1m',
                       'rolling_min_High price_1h', 'rolling_min_High price_1d', 'rolling_min_High price_1w',
                       'rolling_min_High price_1m',
                       'rolling_avg_Low Price_1h', 'rolling_avg_Low Price_1d', 'rolling_avg_Low Price_1w',
                       'rolling_avg_Low Price_1m',
                       'rolling_max_Low Price_1h', 'rolling_max_Low Price_1d', 'rolling_max_Low Price_1w',
                       'rolling_max_Low Price_1m',
                       'rolling_min_Low Price_1h', 'rolling_min_Low Price_1d', 'rolling_min_Low Price_1w',
                       'rolling_min_Low Price_1m',
                       'rolling_avg_Close Price_1h', 'rolling_avg_Close Price_1d', 'rolling_avg_Close Price_1w',
                       'rolling_avg_Close Price_1m',
                       'rolling_max_Close Price_1h', 'rolling_max_Close Price_1d', 'rolling_max_Close Price_1w',
                       'rolling_max_Close Price_1m',
                       'rolling_min_Close Price_1h', 'rolling_min_Close Price_1d', 'rolling_min_Close Price_1w',
                       'rolling_min_Close Price_1m']

In [ ]:
import tensorflow as tf
#from preprocessing_pipeline import preprocess_df
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm
import matplotlib.pyplot as plt
#from parameters_modeling import exclude_x, log_transformation_feat, continuous_features, val_split, test_split, location_data
#import wandb
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
#from wandb.keras import WandbCallback
import tensorflow as tf


AUTOTUNE = tf.data.experimental.AUTOTUNE


def select_last_year(df):
    df = df.sort_values('Open Time')
    df = df.iloc[-1000000:, :]
    df = df.reset_index(drop=True)
    print('Only Selecting the last 2 years of data')
    return df 

def time_features_func(df):
    df['Date'] = pd.to_datetime(df['Open Time'], unit = 'ms')
    df['Year'] = pd.DatetimeIndex(df['Date']).year
    df['Month'] = pd.DatetimeIndex(df['Date']).month
    df['Day of Week'] = pd.DatetimeIndex(df['Date']).dayofweek
    df['Day of Month'] = pd.DatetimeIndex(df['Date']).day
    df['Day of Year'] = pd.DatetimeIndex(df['Date']).dayofyear
    df['Week of Month'] = pd.DatetimeIndex(df['Date']).day // 7
    df['Week of Year'] = df['Date'].dt.isocalendar().week.astype('int64')
    df['hour'] =df.Date.dt.hour
    df['minute'] =df.Date.dt.minute
    df['minute_of_day'] = df.hour*60 + df.minute
    print('Time preprocessing done')
    return df

def create_target_variables(df):
    # df = df.drop('Unnamed: 0', axis=1)
    df['Mid Price'] = (df['Open Price'] + df['Close Price'])/2
    df['y'] = (df['Open Price'].shift(-1) + df['Close Price'].shift(-1))/2  # the target is predicting the next close price
    df['benchmark'] = df['Mid Price']
    print('Target preprocessing done')
    return df


def create_last_reporting(df, columns=['Open Price', 'Close Price', 'Mid Price']):
    """Twice periods previously w.r.t the y variable"""
    for column in columns:
        df[column+'_prev'] = df[column].shift(1)
    df = df.dropna()
    print('Last preprocessing done')
    return df

def create_deviations(df):
    df['high_low'] = df['High price'] - df['Low Price']
    df['high_close'] = df['High price'] - df['Close Price']
    df['high_open'] = df['High price'] - df['Open Price']
    df['open_low'] = df['Close Price'] - df['Low Price']
    df['spread'] =df['Close Price'] - df['Open Price']
    df['spread_ind'] = 1*(df['spread'] < 0)
    df['spread'] =np.abs(df.spread)
    print('Deviations preprocessing done')
    return df

def create_stats(df, features = ['high_low', 'high_close', 'high_open', 'open_low', 'spread', 'Open Price', 'High price', 'Low Price', 'Close Price']):
    for feature in features:
        series_1h, series_1d, series_1w, series_1m =  df[feature].rolling(window = 60), df[feature].rolling(window = 1500), df[feature].rolling(window = 10000), df[feature].rolling(window = 50000)
        df['rolling_avg_{}_1h'.format(feature)] = series_1d.mean() # rolling avg over 1 hour
        df['rolling_avg_{}_1d'.format(feature)] = series_1d.mean() # rolling avg over 1 day
        df['rolling_avg_{}_1w'.format(feature)] = series_1w.mean() # rolling avg over 1 week
        df['rolling_avg_{}_1m'.format(feature)] = series_1m.mean() # rolling avg over 1 month
        df['rolling_max_{}_1h'.format(feature)] = series_1d.max() # rolling max over 1 hour
        df['rolling_max_{}_1d'.format(feature)] = series_1d.max() # rolling max over 1 day
        df['rolling_max_{}_1w'.format(feature)] = series_1w.max() # rolling max over 1 week
        df['rolling_max_{}_1m'.format(feature)] = series_1m.max()  # rolling max over 1 month
        #df['rolling_min_{}_1h'.format(feature)] = series_1d.min() # rolling min over 1 hour
        #df['rolling_min_{}_1d'.format(feature)] = series_1d.min() # rolling min over 1 day
        #df['rolling_min_{}_1w'.format(feature)] = series_1w.min() # rolling min over 1 week
        #df['rolling_min_{}_1m'.format(feature)] = series_1m.min() # rolling min over 1 month
    df = df.dropna()
    print('Rolling preprocessing done')
    return df

def log_transformation(df, log_transformation_features=log_transformation_feat):
    for feature in log_transformation_features:
        df['log_{}'.format(feature)] = np.log(1+df[feature].values)
        df = df.drop(feature, axis=1)
    print('Logs preprocessing done')
    return df

def _split(df, train_val_date=val_split, val_test_date=test_split, exclude_x=exclude_x):
    # Standardize the dataframe
    df = df.reset_index(drop=True)
    if train_val_date is None and val_test_date is None:
        index_val, index_test = int(len(df)*0.8), int(len(df)*0.9)
        print(index_val, index_test)
        print(df.head(), len(df.index))
        train_val_date, val_test_date = df.loc[index_val, 'Date'], df.loc[index_test, 'Date']
    print('We are going to build a model with train/val date {} and val/test date {}'.format(train_val_date, val_test_date))
    df_train, df_val, df_test = df[df.Date <= pd.to_datetime(train_val_date)], df[(df.Date > pd.to_datetime(train_val_date)) & (df.Date <= pd.to_datetime(val_test_date))], df[df.Date > pd.to_datetime(val_test_date)]
    features_list = [col for col in df_train.columns if col not in exclude_x]
    X_train, X_val, X_test = df_train[features_list], df_val[features_list],df_test[features_list]
    y_train, y_val, y_test = df_train[['y']], df_val[['y']], df_test[['y']]
    print('Splits done')
    return X_train, X_val, X_test, y_train, y_val, y_test, features_list

def standardize(X_train, X_val, X_test, y_train, y_val, y_test):
    x_mean = X_train.mean()
    x_std = X_train.std()
    y_mean = y_train.mean()
    y_std = y_train.std()
    X_train, X_val, X_test = (X_train - x_mean) / x_std, (X_val - x_mean) / x_std, (X_test - x_mean) / x_std
    y_train, y_val, y_test = (y_train - y_mean) / y_std, (y_val - y_mean) / y_std, (y_test - y_mean) / y_std
    print('Standardization done')
    return X_train, X_val, X_test, y_train, y_val, y_test, x_mean, x_std, y_mean, y_std

def generate_input_sequences(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, input_seq_len, len(feats)))
    dfvs = data[feats].values
    for i in (range(input_seq_len, len(data)-output_seq_len+1)):
        x = np.expand_dims(dfvs[i-input_seq_len:i, :], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_output_sequences(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, output_seq_len, 1))
    dfvs = data[feats].values
    for i in range(input_seq_len, len(data)-output_seq_len+1):
        x = np.expand_dims(dfvs[i:i+output_seq_len, :], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_baseline_predictions(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, output_seq_len, 1))
    dfvs = data[feats].values
    for i in range(input_seq_len, len(data)-output_seq_len+1):
        x = np.expand_dims(np.ones((output_seq_len, 1))*dfvs[i][0], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_tf_data(x_train, x_val, y_train, y_val, y_train_baseline, y_val_baseline, batch_size):
    x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
    x_val = tf.convert_to_tensor(x_val, dtype=tf.float32)
    y_train = tf.convert_to_tensor(y_train-y_train_baseline, dtype=tf.float32)
    y_val = tf.convert_to_tensor(y_val-y_val_baseline, dtype=tf.float32)

    train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_data = train_data.batch(batch_size)
    train_data = train_data.prefetch(AUTOTUNE)

    validation_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    validation_data = validation_data.batch(batch_size)
    validation_data = validation_data.prefetch(AUTOTUNE)
    return train_data, validation_data

def preprocess_df(df, batch_size, location_data=location_data):
    df = select_last_year(df)
    df = time_features_func(df)
    df = create_target_variables(df)
    df = create_last_reporting(df)
    df = create_deviations(df)
    df = create_stats(df)
    df = log_transformation(df)
    X_train, X_val, X_test, y_train, y_val, y_test, features_list = _split(df)
    X_train, X_val, X_test, y_train, y_val, y_test, x_mean, x_std, y_mean, y_std = standardize(X_train, X_val, X_test, y_train, y_val, y_test)
    X_train_seq = generate_input_sequences(X_train, X_train.columns.values.tolist())
    X_val_seq = generate_input_sequences(X_val, X_val.columns.values.tolist())
    X_test_seq = generate_input_sequences(X_test, X_test.columns.values.tolist())
    print('Inputs done')
    y_train_seq = generate_output_sequences(y_train, y_train.columns.values.tolist())
    y_val_seq = generate_output_sequences(y_val, y_val.columns.values.tolist())
    y_test_seq = generate_output_sequences(y_test, y_test.columns.values.tolist())
    print('Outputs done')
    y_train_baseline = generate_baseline_predictions(y_train, y_train.columns.values.tolist())
    y_val_baseline = generate_baseline_predictions(y_val, y_val.columns.values.tolist())
    y_test_baseline = generate_baseline_predictions(y_test, y_test.columns.values.tolist())
    print('Baseline done')
    train_data, validation_data = generate_tf_data(X_train_seq, X_val_seq, y_train_seq, y_val_seq, y_train_baseline, y_val_baseline, batch_size)
    return train_data, validation_data, features_list, x_mean, x_std, y_mean, y_std

print('here')
df = pd.read_csv('/content/gdrive/MyDrive/playground/btcusdt_data.csv', index_col=0)
batch_size = 128
lr = 1e-3
optimizer = 'Adam'
n_layers = 3
n_units = 32
dropout = 0.2
weight_decay = 1e-4
train_data, validation_data, features_list, x_mean, x_std, y_mean, y_std = preprocess_df(df, batch_size)

if optimizer == 'Adam':
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
elif optimizer == 'SGD':
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=500, decay_rate=0.9,staircase=True)  
    opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

lstm_model = tf.keras.models.Sequential()

if n_layers == 1:
    lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
else:
    for n in range(n_layers):
        if n == n_layers - 1:
            lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
            #lstm_model.add(batchNormalization())
            lstm_model.add(tf.keras.layers.Dropout(dropout))
        else:
            lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=True))
            lstm_model.add(BatchNormalization())
            lstm_model.add(tf.keras.layers.Dropout(dropout))
            
lstm_model.add(tf.keras.layers.Dense(units=n_units, kernel_regularizer=regularizers.l2(l2=weight_decay)))

MAX_EPOCHS = 10
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, mode='min')
lstm_model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=opt, metrics=[tf.metrics.MeanAbsoluteError(), tf.keras.losses.MeanSquaredError()])
# history = lstm_model.fit(train_data, epochs=MAX_EPOCHS, validation_data=validation_data, callbacks=[early_stopping, WandbCallback()])
history = lstm_model.fit(train_data, epochs=MAX_EPOCHS, validation_data=validation_data, callbacks=[early_stopping])

In [4]:
from google.colab import auth
auth.authenticate_user()

In [5]:
project_id = 'crypto-forecasting-app'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://artifacts.crypto-forecasting-app.appspot.com/
gs://crypto-forecasting-bucket/


# FINAL MODELING PER PAIR STARTS HERE

# BTCUSDT MODELING

In [6]:
import os

# data_folder = '../Data'
# file_name = 'btcusdt_exchange_full.csv'

#location_data = os.path.join(data_folder, file_name)
location_data = '/content/gdrive/MyDrive/playground/btcusdt_data.csv'

exclude_x = ['NA', 'Open Time', 'Close Time', 'y', 'Date']

val_split, test_split = None, None

log_transformation_feat = ['Open Price', 'High price', 'Low Price', 'Close Price', 'Volume Traded',
                           'Quote asset Volume', 'Number of Trades', 'Taker buy base asset volume',
                           'Taker buy quote asset volume', 'Mid Price', 'Open Price_prev', 'Close Price_prev',
                           'Mid Price_prev', 'high_low', 'high_close', 'high_open', 'open_low', 'spread',
                           'rolling_avg_high_low_1h', 'rolling_avg_high_low_1d', 'rolling_avg_high_low_1w',
                           'rolling_avg_high_low_1m', 'rolling_max_high_low_1h',
                           'rolling_max_high_low_1d', 'rolling_max_high_low_1w', 'rolling_max_high_low_1m',
                           'rolling_avg_high_close_1h', 'rolling_avg_high_close_1d', 'rolling_avg_high_close_1w',
                           'rolling_avg_high_close_1m', 'rolling_max_high_close_1h', 'rolling_max_high_close_1d',
                           'rolling_max_high_close_1w', 'rolling_max_high_close_1m', 'rolling_avg_high_open_1h',
                           'rolling_avg_high_open_1d', 'rolling_avg_high_open_1w', 'rolling_avg_high_open_1m',
                           'rolling_max_high_open_1h', 'rolling_max_high_open_1d',
                           'rolling_max_high_open_1w', 'rolling_max_high_open_1m', 'rolling_avg_open_low_1h',
                           'rolling_avg_open_low_1d',
                           'rolling_avg_open_low_1w', 'rolling_avg_open_low_1m', 'rolling_max_open_low_1h',
                           'rolling_max_open_low_1d',
                           'rolling_max_open_low_1w', 'rolling_max_open_low_1m', 'rolling_avg_spread_1h',
                           'rolling_avg_spread_1d',
                           'rolling_avg_spread_1w', 'rolling_avg_spread_1m', 'rolling_max_spread_1h',
                           'rolling_max_spread_1d', 'rolling_max_spread_1w',
                           'rolling_max_spread_1m', 'rolling_avg_Open Price_1h', 'rolling_avg_Open Price_1d',
                           'rolling_avg_Open Price_1w', 'rolling_avg_Open Price_1m',
                           'rolling_max_Open Price_1h', 'rolling_max_Open Price_1d', 'rolling_max_Open Price_1w',
                           'rolling_max_Open Price_1m',
                           'rolling_avg_High price_1h', 'rolling_avg_High price_1d', 'rolling_avg_High price_1w',
                           'rolling_avg_High price_1m',
                           'rolling_max_High price_1h', 'rolling_max_High price_1d', 'rolling_max_High price_1w',
                           'rolling_max_High price_1m',
                           'rolling_avg_Low Price_1h', 'rolling_avg_Low Price_1d', 'rolling_avg_Low Price_1w',
                           'rolling_avg_Low Price_1m',
                           'rolling_max_Low Price_1h', 'rolling_max_Low Price_1d', 'rolling_max_Low Price_1w',
                           'rolling_max_Low Price_1m',
                           'rolling_avg_Close Price_1h', 'rolling_avg_Close Price_1d', 'rolling_avg_Close Price_1w',
                           'rolling_avg_Close Price_1m',
                           'rolling_max_Close Price_1h', 'rolling_max_Close Price_1d', 'rolling_max_Close Price_1w',
                           'rolling_max_Close Price_1m']
# features where we apply log transformation


continuous_features = ['Open Price', 'High price', 'Low Price', 'Close Price', 'Volume Traded',
                       'Quote asset Volume', 'Number of Trades', 'Taker buy base asset volume',
                       'Taker buy quote asset volume', 'Mid Price', 'Open Price_prev', 'Close Price_prev',
                       'Mid Price_prev', 'high_low', 'high_close', 'high_open', 'open_low', 'spread',
                       'rolling_avg_high_low_1h', 'rolling_avg_high_low_1d', 'rolling_avg_high_low_1w',
                       'rolling_avg_high_low_1m', 'rolling_max_high_low_1h',
                       'rolling_max_high_low_1d', 'rolling_max_high_low_1w', 'rolling_max_high_low_1m',
                       'rolling_min_high_low_1h', 'rolling_min_high_low_1d',
                       'rolling_min_high_low_1w', 'rolling_min_high_low_1m', 'rolling_avg_high_close_1h',
                       'rolling_avg_high_close_1d', 'rolling_avg_high_close_1w', 'rolling_avg_high_close_1m',
                       'rolling_max_high_close_1h', 'rolling_max_high_close_1d', 'rolling_max_high_close_1w',
                       'rolling_max_high_close_1m', 'rolling_min_high_close_1h', 'rolling_min_high_close_1d',
                       'rolling_min_high_close_1w', 'rolling_min_high_close_1m', 'rolling_avg_high_open_1h',
                       'rolling_avg_high_open_1d', 'rolling_avg_high_open_1w', 'rolling_avg_high_open_1m',
                       'rolling_max_high_open_1h', 'rolling_max_high_open_1d',
                       'rolling_max_high_open_1w', 'rolling_max_high_open_1m', 'rolling_min_high_open_1h',
                       'rolling_min_high_open_1d',
                       'rolling_min_high_open_1w', 'rolling_min_high_open_1m', 'rolling_avg_open_low_1h',
                       'rolling_avg_open_low_1d',
                       'rolling_avg_open_low_1w', 'rolling_avg_open_low_1m', 'rolling_max_open_low_1h',
                       'rolling_max_open_low_1d',
                       'rolling_max_open_low_1w', 'rolling_max_open_low_1m', 'rolling_min_open_low_1h',
                       'rolling_min_open_low_1d',
                       'rolling_min_open_low_1w', 'rolling_min_open_low_1m', 'rolling_avg_spread_1h',
                       'rolling_avg_spread_1d',
                       'rolling_avg_spread_1w', 'rolling_avg_spread_1m', 'rolling_max_spread_1h',
                       'rolling_max_spread_1d', 'rolling_max_spread_1w',
                       'rolling_max_spread_1m', 'rolling_min_spread_1h', 'rolling_min_spread_1d',
                       'rolling_min_spread_1w', 'rolling_min_spread_1m',
                       'rolling_avg_Open Price_1h', 'rolling_avg_Open Price_1d', 'rolling_avg_Open Price_1w',
                       'rolling_avg_Open Price_1m',
                       'rolling_max_Open Price_1h', 'rolling_max_Open Price_1d', 'rolling_max_Open Price_1w',
                       'rolling_max_Open Price_1m',
                       'rolling_min_Open Price_1h', 'rolling_min_Open Price_1d', 'rolling_min_Open Price_1w',
                       'rolling_min_Open Price_1m',
                       'rolling_avg_High price_1h', 'rolling_avg_High price_1d', 'rolling_avg_High price_1w',
                       'rolling_avg_High price_1m',
                       'rolling_max_High price_1h', 'rolling_max_High price_1d', 'rolling_max_High price_1w',
                       'rolling_max_High price_1m',
                       'rolling_min_High price_1h', 'rolling_min_High price_1d', 'rolling_min_High price_1w',
                       'rolling_min_High price_1m',
                       'rolling_avg_Low Price_1h', 'rolling_avg_Low Price_1d', 'rolling_avg_Low Price_1w',
                       'rolling_avg_Low Price_1m',
                       'rolling_max_Low Price_1h', 'rolling_max_Low Price_1d', 'rolling_max_Low Price_1w',
                       'rolling_max_Low Price_1m',
                       'rolling_min_Low Price_1h', 'rolling_min_Low Price_1d', 'rolling_min_Low Price_1w',
                       'rolling_min_Low Price_1m',
                       'rolling_avg_Close Price_1h', 'rolling_avg_Close Price_1d', 'rolling_avg_Close Price_1w',
                       'rolling_avg_Close Price_1m',
                       'rolling_max_Close Price_1h', 'rolling_max_Close Price_1d', 'rolling_max_Close Price_1w',
                       'rolling_max_Close Price_1m',
                       'rolling_min_Close Price_1h', 'rolling_min_Close Price_1d', 'rolling_min_Close Price_1w',
                       'rolling_min_Close Price_1m']

In [7]:
import tensorflow as tf
#from preprocessing_pipeline import preprocess_df
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm
import matplotlib.pyplot as plt
#from parameters_modeling import exclude_x, log_transformation_feat, continuous_features, val_split, test_split, location_data
#import wandb
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
#from wandb.keras import WandbCallback
import tensorflow as tf


AUTOTUNE = tf.data.experimental.AUTOTUNE


def select_last_year(df):
    df = df.sort_values('Open Time')
    df = df.iloc[-1000000:, :]
    df = df.reset_index(drop=True)
    print('Only Selecting the last 2 years of data')
    return df 

def time_features_func(df):
    df['Date'] = pd.to_datetime(df['Open Time'], unit = 'ms')
    df['Year'] = pd.DatetimeIndex(df['Date']).year
    df['Month'] = pd.DatetimeIndex(df['Date']).month
    df['Day of Week'] = pd.DatetimeIndex(df['Date']).dayofweek
    df['Day of Month'] = pd.DatetimeIndex(df['Date']).day
    df['Day of Year'] = pd.DatetimeIndex(df['Date']).dayofyear
    df['Week of Month'] = pd.DatetimeIndex(df['Date']).day // 7
    df['Week of Year'] = df['Date'].dt.isocalendar().week.astype('int64')
    df['hour'] =df.Date.dt.hour
    df['minute'] =df.Date.dt.minute
    df['minute_of_day'] = df.hour*60 + df.minute
    print('Time preprocessing done')
    return df

def create_target_variables(df):
    # df = df.drop('Unnamed: 0', axis=1)
    df['Mid Price'] = (df['Open Price'] + df['Close Price'])/2
    df['y'] = (df['Open Price'].shift(-1) + df['Close Price'].shift(-1))/2  # the target is predicting the next close price
    df['benchmark'] = df['Mid Price']
    print('Target preprocessing done')
    return df


def create_last_reporting(df, columns=['Open Price', 'Close Price', 'Mid Price']):
    """Twice periods previously w.r.t the y variable"""
    for column in columns:
        df[column+'_prev'] = df[column].shift(1)
    df = df.dropna()
    print('Last preprocessing done')
    return df

def create_deviations(df):
    df['high_low'] = df['High price'] - df['Low Price']
    df['high_close'] = df['High price'] - df['Close Price']
    df['high_open'] = df['High price'] - df['Open Price']
    df['open_low'] = df['Close Price'] - df['Low Price']
    df['spread'] =df['Close Price'] - df['Open Price']
    df['spread_ind'] = 1*(df['spread'] < 0)
    df['spread'] =np.abs(df.spread)
    print('Deviations preprocessing done')
    return df

def create_stats(df, features = ['high_low', 'high_close', 'high_open', 'open_low', 'spread', 'Open Price', 'High price', 'Low Price', 'Close Price']):
    for feature in features:
        series_1h, series_1d, series_1w, series_1m =  df[feature].rolling(window = 60), df[feature].rolling(window = 1500), df[feature].rolling(window = 10000), df[feature].rolling(window = 50000)
        df['rolling_avg_{}_1h'.format(feature)] = series_1d.mean() # rolling avg over 1 hour
        df['rolling_avg_{}_1d'.format(feature)] = series_1d.mean() # rolling avg over 1 day
        df['rolling_avg_{}_1w'.format(feature)] = series_1w.mean() # rolling avg over 1 week
        df['rolling_avg_{}_1m'.format(feature)] = series_1m.mean() # rolling avg over 1 month
        df['rolling_max_{}_1h'.format(feature)] = series_1d.max() # rolling max over 1 hour
        df['rolling_max_{}_1d'.format(feature)] = series_1d.max() # rolling max over 1 day
        df['rolling_max_{}_1w'.format(feature)] = series_1w.max() # rolling max over 1 week
        df['rolling_max_{}_1m'.format(feature)] = series_1m.max()  # rolling max over 1 month
        #df['rolling_min_{}_1h'.format(feature)] = series_1d.min() # rolling min over 1 hour
        #df['rolling_min_{}_1d'.format(feature)] = series_1d.min() # rolling min over 1 day
        #df['rolling_min_{}_1w'.format(feature)] = series_1w.min() # rolling min over 1 week
        #df['rolling_min_{}_1m'.format(feature)] = series_1m.min() # rolling min over 1 month
    df = df.dropna()
    print('Rolling preprocessing done')
    return df

def log_transformation(df, log_transformation_features=log_transformation_feat):
    for feature in log_transformation_features:
        df['log_{}'.format(feature)] = np.log(1+df[feature].values)
        df = df.drop(feature, axis=1)
    print('Logs preprocessing done')
    return df

def _split(df, train_val_date=val_split, val_test_date=test_split, exclude_x=exclude_x):
    # Standardize the dataframe
    df = df.reset_index(drop=True)
    if train_val_date is None and val_test_date is None:
        index_val, index_test = int(len(df)*0.8), int(len(df)*0.9)
        print(index_val, index_test)
        print(df.head(), len(df.index))
        train_val_date, val_test_date = df.loc[index_val, 'Date'], df.loc[index_test, 'Date']
    print('We are going to build a model with train/val date {} and val/test date {}'.format(train_val_date, val_test_date))
    df_train, df_val, df_test = df[df.Date <= pd.to_datetime(train_val_date)], df[(df.Date > pd.to_datetime(train_val_date)) & (df.Date <= pd.to_datetime(val_test_date))], df[df.Date > pd.to_datetime(val_test_date)]
    features_list = [col for col in df_train.columns if col not in exclude_x]
    X_train, X_val, X_test = df_train[features_list], df_val[features_list],df_test[features_list]
    y_train, y_val, y_test = df_train[['y']], df_val[['y']], df_test[['y']]
    print('Splits done')
    return X_train, X_val, X_test, y_train, y_val, y_test, features_list

def standardize(X_train, X_val, X_test, y_train, y_val, y_test):
    x_mean = X_train.mean()
    x_std = X_train.std()
    y_mean = y_train.mean()
    y_std = y_train.std()
    X_train, X_val, X_test = (X_train - x_mean) / x_std, (X_val - x_mean) / x_std, (X_test - x_mean) / x_std
    y_train, y_val, y_test = (y_train - y_mean) / y_std, (y_val - y_mean) / y_std, (y_test - y_mean) / y_std
    print('Standardization done')
    return X_train, X_val, X_test, y_train, y_val, y_test, x_mean, x_std, y_mean, y_std

def generate_input_sequences(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, input_seq_len, len(feats)))
    dfvs = data[feats].values
    for i in (range(input_seq_len, len(data)-output_seq_len+1)):
        x = np.expand_dims(dfvs[i-input_seq_len:i, :], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_output_sequences(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, output_seq_len, 1))
    dfvs = data[feats].values
    for i in range(input_seq_len, len(data)-output_seq_len+1):
        x = np.expand_dims(dfvs[i:i+output_seq_len, :], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_baseline_predictions(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, output_seq_len, 1))
    dfvs = data[feats].values
    for i in range(input_seq_len, len(data)-output_seq_len+1):
        x = np.expand_dims(np.ones((output_seq_len, 1))*dfvs[i][0], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_tf_data(x_train, x_val, y_train, y_val, y_train_baseline, y_val_baseline, batch_size):
    x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
    x_val = tf.convert_to_tensor(x_val, dtype=tf.float32)
    y_train = tf.convert_to_tensor(y_train-y_train_baseline, dtype=tf.float32)
    y_val = tf.convert_to_tensor(y_val-y_val_baseline, dtype=tf.float32)

    train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_data = train_data.batch(batch_size)
    train_data = train_data.prefetch(AUTOTUNE)

    validation_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    validation_data = validation_data.batch(batch_size)
    validation_data = validation_data.prefetch(AUTOTUNE)
    return train_data, validation_data

def preprocess_df(df, batch_size, location_data=location_data):
    df = select_last_year(df)
    df = time_features_func(df)
    df = create_target_variables(df)
    df = create_last_reporting(df)
    df = create_deviations(df)
    df = create_stats(df)
    df = log_transformation(df)
    X_train, X_val, X_test, y_train, y_val, y_test, features_list = _split(df)
    X_train, X_val, X_test, y_train, y_val, y_test, x_mean, x_std, y_mean, y_std = standardize(X_train, X_val, X_test, y_train, y_val, y_test)
    X_train_seq = generate_input_sequences(X_train, X_train.columns.values.tolist())
    X_val_seq = generate_input_sequences(X_val, X_val.columns.values.tolist())
    X_test_seq = generate_input_sequences(X_test, X_test.columns.values.tolist())
    print('Inputs done')
    y_train_seq = generate_output_sequences(y_train, y_train.columns.values.tolist())
    y_val_seq = generate_output_sequences(y_val, y_val.columns.values.tolist())
    y_test_seq = generate_output_sequences(y_test, y_test.columns.values.tolist())
    print('Outputs done')
    y_train_baseline = generate_baseline_predictions(y_train, y_train.columns.values.tolist())
    y_val_baseline = generate_baseline_predictions(y_val, y_val.columns.values.tolist())
    y_test_baseline = generate_baseline_predictions(y_test, y_test.columns.values.tolist())
    print('Baseline done')
    train_data, validation_data = generate_tf_data(X_train_seq, X_val_seq, y_train_seq, y_val_seq, y_train_baseline, y_val_baseline, batch_size)
    return train_data, validation_data, features_list, x_mean, x_std, y_mean, y_std

print('here')
df = pd.read_csv('/content/gdrive/MyDrive/playground/btcusdt_data.csv', index_col=0)
batch_size = 128
lr = 1e-3
optimizer = 'Adam'
n_layers = 3
n_units = 32
dropout = 0.2
weight_decay = 1e-4
train_data, validation_data, features_list, x_mean, x_std, y_mean, y_std = preprocess_df(df, batch_size)

if optimizer == 'Adam':
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
elif optimizer == 'SGD':
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=500, decay_rate=0.9,staircase=True)  
    opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

lstm_model1 = tf.keras.models.Sequential()

if n_layers == 1:
    lstm_model1.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
else:
    for n in range(n_layers):
        if n == n_layers - 1:
            lstm_model1.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
            #lstm_model.add(batchNormalization())
            lstm_model1.add(tf.keras.layers.Dropout(dropout))
        else:
            lstm_model1.add(tf.keras.layers.LSTM(n_units, return_sequences=True))
            lstm_model1.add(BatchNormalization())
            lstm_model1.add(tf.keras.layers.Dropout(dropout))
            
lstm_model1.add(tf.keras.layers.Dense(units=n_units, kernel_regularizer=regularizers.l2(l2=weight_decay)))

MAX_EPOCHS = 10
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, mode='min')
lstm_model1.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=opt, metrics=[tf.metrics.MeanAbsoluteError(), tf.keras.losses.MeanSquaredError()])
# history = lstm_model.fit(train_data, epochs=MAX_EPOCHS, validation_data=validation_data, callbacks=[early_stopping, WandbCallback()])
history1 = lstm_model1.fit(train_data, epochs=MAX_EPOCHS, validation_data=validation_data, callbacks=[early_stopping])

here


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Only Selecting the last 2 years of data
Time preprocessing done
Target preprocessing done
Last preprocessing done
Deviations preprocessing done
Rolling preprocessing done
Logs preprocessing done
759999 854999
       Open Time  ...  log_rolling_max_Close Price_1m
0  1582259340000  ...                        9.259226
1  1582259400000  ...                        9.259226
2  1582259460000  ...                        9.259226
3  1582259520000  ...                        9.259226
4  1582259580000  ...                        9.259226

[5 rows x 107 columns] 949999
We are going to build a model with train/val date 2021-08-02 23:09:00 and val/test date 2021-10-08 04:59:00
Splits done
Standardization done
Inputs done
Outputs done
Baseline done
Epoch 1/10
5938/5938 [==============================] - 71s 11ms/step - loss: 5.3260e-04 - mean_absolute_error: 0.0058 - mean_squared_error: 2.7836e-04 - val_loss: 9.6257e-05 - val_mean_absolute_error: 0.0061 - val_mean_squared_error: 9.5889e-05
Epoch 2/10

In [8]:
import tensorflow as tf
#from preprocessing_pipeline import preprocess_df
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm
import matplotlib.pyplot as plt
#from parameters_modeling import exclude_x, log_transformation_feat, continuous_features, val_split, test_split, location_data
#import wandb
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
#from wandb.keras import WandbCallback
import tensorflow as tf


AUTOTUNE = tf.data.experimental.AUTOTUNE


def select_last_year(df):
    df = df.sort_values('Open Time')
    df = df.iloc[-1000000:, :]
    df = df.reset_index(drop=True)
    print('Only Selecting the last 2 years of data')
    return df 

def time_features_func(df):
    df['Date'] = pd.to_datetime(df['Open Time'], unit = 'ms')
    df['Year'] = pd.DatetimeIndex(df['Date']).year
    df['Month'] = pd.DatetimeIndex(df['Date']).month
    df['Day of Week'] = pd.DatetimeIndex(df['Date']).dayofweek
    df['Day of Month'] = pd.DatetimeIndex(df['Date']).day
    df['Day of Year'] = pd.DatetimeIndex(df['Date']).dayofyear
    df['Week of Month'] = pd.DatetimeIndex(df['Date']).day // 7
    df['Week of Year'] = df['Date'].dt.isocalendar().week.astype('int64')
    df['hour'] =df.Date.dt.hour
    df['minute'] =df.Date.dt.minute
    df['minute_of_day'] = df.hour*60 + df.minute
    print('Time preprocessing done')
    return df

def create_target_variables(df):
    # df = df.drop('Unnamed: 0', axis=1)
    df['Mid Price'] = (df['Open Price'] + df['Close Price'])/2
    df['y'] = (df['Open Price'].shift(-1) + df['Close Price'].shift(-1))/2  # the target is predicting the next close price
    df['benchmark'] = df['Mid Price']
    print('Target preprocessing done')
    return df


def create_last_reporting(df, columns=['Open Price', 'Close Price', 'Mid Price']):
    """Twice periods previously w.r.t the y variable"""
    for column in columns:
        df[column+'_prev'] = df[column].shift(1)
    df = df.dropna()
    print('Last preprocessing done')
    return df

def create_deviations(df):
    df['high_low'] = df['High price'] - df['Low Price']
    df['high_close'] = df['High price'] - df['Close Price']
    df['high_open'] = df['High price'] - df['Open Price']
    df['open_low'] = df['Close Price'] - df['Low Price']
    df['spread'] =df['Close Price'] - df['Open Price']
    df['spread_ind'] = 1*(df['spread'] < 0)
    df['spread'] =np.abs(df.spread)
    print('Deviations preprocessing done')
    return df

def create_stats(df, features = ['high_low', 'high_close', 'high_open', 'open_low', 'spread', 'Open Price', 'High price', 'Low Price', 'Close Price']):
    for feature in features:
        series_1h, series_1d, series_1w, series_1m =  df[feature].rolling(window = 60), df[feature].rolling(window = 1500), df[feature].rolling(window = 10000), df[feature].rolling(window = 50000)
        df['rolling_avg_{}_1h'.format(feature)] = series_1d.mean() # rolling avg over 1 hour
        df['rolling_avg_{}_1d'.format(feature)] = series_1d.mean() # rolling avg over 1 day
        df['rolling_avg_{}_1w'.format(feature)] = series_1w.mean() # rolling avg over 1 week
        df['rolling_avg_{}_1m'.format(feature)] = series_1m.mean() # rolling avg over 1 month
        df['rolling_max_{}_1h'.format(feature)] = series_1d.max() # rolling max over 1 hour
        df['rolling_max_{}_1d'.format(feature)] = series_1d.max() # rolling max over 1 day
        df['rolling_max_{}_1w'.format(feature)] = series_1w.max() # rolling max over 1 week
        df['rolling_max_{}_1m'.format(feature)] = series_1m.max()  # rolling max over 1 month
        #df['rolling_min_{}_1h'.format(feature)] = series_1d.min() # rolling min over 1 hour
        #df['rolling_min_{}_1d'.format(feature)] = series_1d.min() # rolling min over 1 day
        #df['rolling_min_{}_1w'.format(feature)] = series_1w.min() # rolling min over 1 week
        #df['rolling_min_{}_1m'.format(feature)] = series_1m.min() # rolling min over 1 month
    df = df.dropna()
    print('Rolling preprocessing done')
    return df

def log_transformation(df, log_transformation_features=log_transformation_feat):
    for feature in log_transformation_features:
        df['log_{}'.format(feature)] = np.log(1+df[feature].values)
        df = df.drop(feature, axis=1)
    print('Logs preprocessing done')
    return df

def _split(df, train_val_date=val_split, val_test_date=test_split, exclude_x=exclude_x):
    # Standardize the dataframe
    df = df.reset_index(drop=True)
    # if train_val_date is None and val_test_date is None:
    #     index_val, index_test = int(len(df)*0.8), int(len(df)*0.9)
    #     print(index_val, index_test)
    #     print(df.head(), len(df.index))
    #     train_val_date, val_test_date = df.loc[index_val, 'Date'], df.loc[index_test, 'Date']
    # print('We are going to build a model with train/val date {} and val/test date {}'.format(train_val_date, val_test_date))
    # df_train, df_val, df_test = df[df.Date <= pd.to_datetime(train_val_date)], df[(df.Date > pd.to_datetime(train_val_date)) & (df.Date <= pd.to_datetime(val_test_date))], df[df.Date > pd.to_datetime(val_test_date)]
    features_list = [col for col in df.columns if col not in exclude_x]
    X = df[features_list]
    y = df[['y']]
    print('Splits done')
    return X, y, features_list

def standardize(X, y):
    x_mean = X.mean()
    x_std = X.std()
    y_mean = y.mean()
    y_std = y.std()
    X = (X - x_mean) / x_std
    y = (y - y_mean) / y_std
    print('Standardization done')
    return X, y, x_mean, x_std, y_mean, y_std

def generate_input_sequences(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, input_seq_len, len(feats)))
    dfvs = data[feats].values
    for i in (range(input_seq_len, len(data)-output_seq_len+1)):
        x = np.expand_dims(dfvs[i-input_seq_len:i, :], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_output_sequences(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, output_seq_len, 1))
    dfvs = data[feats].values
    for i in range(input_seq_len, len(data)-output_seq_len+1):
        x = np.expand_dims(dfvs[i:i+output_seq_len, :], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_baseline_predictions(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, output_seq_len, 1))
    dfvs = data[feats].values
    for i in range(input_seq_len, len(data)-output_seq_len+1):
        x = np.expand_dims(np.ones((output_seq_len, 1))*dfvs[i][0], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_tf_data(x_train, y_train, y_train_baseline, batch_size):
    x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
    y_train = tf.convert_to_tensor(y_train-y_train_baseline, dtype=tf.float32)

    train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_data = train_data.batch(batch_size)
    train_data = train_data.prefetch(AUTOTUNE)
    return train_data

def preprocess_df(df, batch_size, location_data=location_data):
    df = select_last_year(df)
    df = time_features_func(df)
    df = create_target_variables(df)
    df = create_last_reporting(df)
    df = create_deviations(df)
    df = create_stats(df)
    df = log_transformation(df)
    X_train, y_train, features_list = _split(df)
    # display(X_train) WORKS
    X_train, y_train, x_mean, x_std, y_mean, y_std = standardize(X_train, y_train)
    # print(X_train.shape) WORKS
    X_train_seq = generate_input_sequences(X_train, X_train.columns.values.tolist())
    print(X_train_seq.shape)
    print('Inputs done')
    y_train_seq = generate_output_sequences(y_train, y_train.columns.values.tolist())
    print(y_train_seq.shape)
    print('Outputs done')
    y_train_baseline = generate_baseline_predictions(y_train, y_train.columns.values.tolist())
    print(y_train_baseline.shape)
    print('Baseline done')
    train_data = generate_tf_data(X_train_seq, y_train_seq, y_train_baseline, batch_size)
    return train_data, features_list, x_mean, x_std, y_mean, y_std

print('here')
df = pd.read_csv('/content/gdrive/MyDrive/playground/btcusdt_data.csv', index_col=0)
batch_size = 128
lr = 1e-3
optimizer = 'Adam'
n_layers = 3
n_units = 32
dropout = 0.2
weight_decay = 1e-4
train_data, features_list, x_mean, x_std, y_mean, y_std = preprocess_df(df, batch_size)

if optimizer == 'Adam':
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
elif optimizer == 'SGD':
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=500, decay_rate=0.9,staircase=True)  
    opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

lstm_model = tf.keras.models.Sequential()

if n_layers == 1:
    lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
else:
    for n in range(n_layers):
        if n == n_layers - 1:
            lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
            #lstm_model.add(batchNormalization())
            lstm_model.add(tf.keras.layers.Dropout(dropout))
        else:
            lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=True))
            lstm_model.add(BatchNormalization())
            lstm_model.add(tf.keras.layers.Dropout(dropout))
            
lstm_model.add(tf.keras.layers.Dense(units=n_units, kernel_regularizer=regularizers.l2(l2=weight_decay)))

MAX_EPOCHS = 10
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, mode='min')
lstm_model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=opt, metrics=[tf.metrics.MeanAbsoluteError(), tf.keras.losses.MeanSquaredError()])
# history = lstm_model.fit(train_data, epochs=MAX_EPOCHS, validation_data=validation_data, callbacks=[early_stopping, WandbCallback()])
history = lstm_model.fit(train_data, epochs=MAX_EPOCHS, callbacks=[early_stopping])

here


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Only Selecting the last 2 years of data
Time preprocessing done
Target preprocessing done
Last preprocessing done
Deviations preprocessing done
Rolling preprocessing done
Logs preprocessing done
Splits done
Standardization done
(949960, 8, 102)
Inputs done
(949960, 32, 1)
Outputs done
(949960, 32, 1)
Baseline done
Epoch 1/10
7422/7422 [==============================] - 79s 10ms/step - loss: 3.7299e-04 - mean_absolute_error: 0.0055 - mean_squared_error: 2.0299e-04
Epoch 2/10
7422/7422 [==============================] - 72s 10ms/step - loss: 6.6507e-05 - mean_absolute_error: 0.0041 - mean_squared_error: 6.5917e-05
Epoch 3/10
7422/7422 [==============================] - 72s 10ms/step - loss: 5.6633e-05 - mean_absolute_error: 0.0038 - mean_squared_error: 5.5213e-05
Epoch 4/10
7422/7422 [==============================] - 72s 10ms/step - loss: 5.2328e-05 - mean_absolute_error: 0.0037 - mean_squared_error: 5.0785e-05
Epoch 5/10
7422/7422 [==============================] - 72s 10ms/step - loss

In [ ]:
# !rm -r /content/gdrive/MyDrive/playground/LSTM_BTCUSDT_12-12-2021/

rm: cannot remove '/content/gdrive/MyDrive/playground/LSTM_BTCUSDT_12-12-2021/': No such file or directory


In [9]:
import json

data = {}

data['validation_score'] = history1.history['val_loss'][-1]
data['unique_id'] = time.time()
data['training_time_stamp'] = time.time()
data['hyperparameters'] = {'batch_size': 128, 'lr': 1e-3, 'optimizer': 'Adam', 'n_layers': 3, 'n_units': 8, 'dropout': 0.2, 'weight_decay': 1e-4}
data['symbol'] = 'BTCUSDT'
data['x_mean'] = x_mean.values.tolist()
data['x_std'] = x_std.values.tolist()
data['y_mean'] = y_mean.values[0]
data['y_std'] = y_std.values[0]

with open('model_metrics_BTCUSDT.json', 'w') as outfile:
    json.dump(data, outfile)

In [10]:
!mkdir /content/gdrive/MyDrive/playground/LSTM_BTCUSDT_$(date +"%d-%m-%Y")

In [11]:
!cp /content/model_metrics_BTCUSDT.json /content/gdrive/MyDrive/playground/LSTM_BTCUSDT_13-12-2021/

In [12]:
lstm_model.save_weights('/content/gdrive/MyDrive/playground/LSTM_BTCUSDT_13-12-2021/lstm_BTCUSDT')

In [13]:
!gsutil -m cp -r /content/gdrive/MyDrive/playground/LSTM_BTCUSDT_13-12-2021/ gs://crypto-forecasting-bucket/BTCUSDT/Model/

Copying file:///content/gdrive/MyDrive/playground/LSTM_BTCUSDT_13-12-2021/model_metrics_BTCUSDT.json [Content-Type=application/json]...
Copying file:///content/gdrive/MyDrive/playground/LSTM_BTCUSDT_13-12-2021/lstm_BTCUSDT.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file:///content/gdrive/MyDrive/playground/LSTM_BTCUSDT_13-12-2021/lstm_BTCUSDT.index [Content-Type=application/octet-stream]...
Copying file:///content/gdrive/MyDrive/playground/LSTM_BTCUSDT_13-12-2021/checkpoint [Content-Type=application/octet-stream]...
/ [4/4 files][427.6 KiB/427.6 KiB] 100% Done                                    
Operation completed over 4 objects/427.6 KiB.                                    


# BNBBTC MODELING

In [14]:
import os

location_data = '/content/gdrive/MyDrive/playground/bnbbtc_data.csv'

exclude_x = ['NA', 'Open Time', 'Close Time', 'y', 'Date']

val_split, test_split = None, None

log_transformation_feat = ['Open Price', 'High price', 'Low Price', 'Close Price', 'Volume Traded',
                           'Quote asset Volume', 'Number of Trades', 'Taker buy base asset volume',
                           'Taker buy quote asset volume', 'Mid Price', 'Open Price_prev', 'Close Price_prev',
                           'Mid Price_prev', 'high_low', 'high_close', 'high_open', 'open_low', 'spread',
                           'rolling_avg_high_low_1h', 'rolling_avg_high_low_1d', 'rolling_avg_high_low_1w',
                           'rolling_avg_high_low_1m', 'rolling_max_high_low_1h',
                           'rolling_max_high_low_1d', 'rolling_max_high_low_1w', 'rolling_max_high_low_1m',
                           'rolling_avg_high_close_1h', 'rolling_avg_high_close_1d', 'rolling_avg_high_close_1w',
                           'rolling_avg_high_close_1m', 'rolling_max_high_close_1h', 'rolling_max_high_close_1d',
                           'rolling_max_high_close_1w', 'rolling_max_high_close_1m', 'rolling_avg_high_open_1h',
                           'rolling_avg_high_open_1d', 'rolling_avg_high_open_1w', 'rolling_avg_high_open_1m',
                           'rolling_max_high_open_1h', 'rolling_max_high_open_1d',
                           'rolling_max_high_open_1w', 'rolling_max_high_open_1m', 'rolling_avg_open_low_1h',
                           'rolling_avg_open_low_1d',
                           'rolling_avg_open_low_1w', 'rolling_avg_open_low_1m', 'rolling_max_open_low_1h',
                           'rolling_max_open_low_1d',
                           'rolling_max_open_low_1w', 'rolling_max_open_low_1m', 'rolling_avg_spread_1h',
                           'rolling_avg_spread_1d',
                           'rolling_avg_spread_1w', 'rolling_avg_spread_1m', 'rolling_max_spread_1h',
                           'rolling_max_spread_1d', 'rolling_max_spread_1w',
                           'rolling_max_spread_1m', 'rolling_avg_Open Price_1h', 'rolling_avg_Open Price_1d',
                           'rolling_avg_Open Price_1w', 'rolling_avg_Open Price_1m',
                           'rolling_max_Open Price_1h', 'rolling_max_Open Price_1d', 'rolling_max_Open Price_1w',
                           'rolling_max_Open Price_1m',
                           'rolling_avg_High price_1h', 'rolling_avg_High price_1d', 'rolling_avg_High price_1w',
                           'rolling_avg_High price_1m',
                           'rolling_max_High price_1h', 'rolling_max_High price_1d', 'rolling_max_High price_1w',
                           'rolling_max_High price_1m',
                           'rolling_avg_Low Price_1h', 'rolling_avg_Low Price_1d', 'rolling_avg_Low Price_1w',
                           'rolling_avg_Low Price_1m',
                           'rolling_max_Low Price_1h', 'rolling_max_Low Price_1d', 'rolling_max_Low Price_1w',
                           'rolling_max_Low Price_1m',
                           'rolling_avg_Close Price_1h', 'rolling_avg_Close Price_1d', 'rolling_avg_Close Price_1w',
                           'rolling_avg_Close Price_1m',
                           'rolling_max_Close Price_1h', 'rolling_max_Close Price_1d', 'rolling_max_Close Price_1w',
                           'rolling_max_Close Price_1m']
# features where we apply log transformation


continuous_features = ['Open Price', 'High price', 'Low Price', 'Close Price', 'Volume Traded',
                       'Quote asset Volume', 'Number of Trades', 'Taker buy base asset volume',
                       'Taker buy quote asset volume', 'Mid Price', 'Open Price_prev', 'Close Price_prev',
                       'Mid Price_prev', 'high_low', 'high_close', 'high_open', 'open_low', 'spread',
                       'rolling_avg_high_low_1h', 'rolling_avg_high_low_1d', 'rolling_avg_high_low_1w',
                       'rolling_avg_high_low_1m', 'rolling_max_high_low_1h',
                       'rolling_max_high_low_1d', 'rolling_max_high_low_1w', 'rolling_max_high_low_1m',
                       'rolling_min_high_low_1h', 'rolling_min_high_low_1d',
                       'rolling_min_high_low_1w', 'rolling_min_high_low_1m', 'rolling_avg_high_close_1h',
                       'rolling_avg_high_close_1d', 'rolling_avg_high_close_1w', 'rolling_avg_high_close_1m',
                       'rolling_max_high_close_1h', 'rolling_max_high_close_1d', 'rolling_max_high_close_1w',
                       'rolling_max_high_close_1m', 'rolling_min_high_close_1h', 'rolling_min_high_close_1d',
                       'rolling_min_high_close_1w', 'rolling_min_high_close_1m', 'rolling_avg_high_open_1h',
                       'rolling_avg_high_open_1d', 'rolling_avg_high_open_1w', 'rolling_avg_high_open_1m',
                       'rolling_max_high_open_1h', 'rolling_max_high_open_1d',
                       'rolling_max_high_open_1w', 'rolling_max_high_open_1m', 'rolling_min_high_open_1h',
                       'rolling_min_high_open_1d',
                       'rolling_min_high_open_1w', 'rolling_min_high_open_1m', 'rolling_avg_open_low_1h',
                       'rolling_avg_open_low_1d',
                       'rolling_avg_open_low_1w', 'rolling_avg_open_low_1m', 'rolling_max_open_low_1h',
                       'rolling_max_open_low_1d',
                       'rolling_max_open_low_1w', 'rolling_max_open_low_1m', 'rolling_min_open_low_1h',
                       'rolling_min_open_low_1d',
                       'rolling_min_open_low_1w', 'rolling_min_open_low_1m', 'rolling_avg_spread_1h',
                       'rolling_avg_spread_1d',
                       'rolling_avg_spread_1w', 'rolling_avg_spread_1m', 'rolling_max_spread_1h',
                       'rolling_max_spread_1d', 'rolling_max_spread_1w',
                       'rolling_max_spread_1m', 'rolling_min_spread_1h', 'rolling_min_spread_1d',
                       'rolling_min_spread_1w', 'rolling_min_spread_1m',
                       'rolling_avg_Open Price_1h', 'rolling_avg_Open Price_1d', 'rolling_avg_Open Price_1w',
                       'rolling_avg_Open Price_1m',
                       'rolling_max_Open Price_1h', 'rolling_max_Open Price_1d', 'rolling_max_Open Price_1w',
                       'rolling_max_Open Price_1m',
                       'rolling_min_Open Price_1h', 'rolling_min_Open Price_1d', 'rolling_min_Open Price_1w',
                       'rolling_min_Open Price_1m',
                       'rolling_avg_High price_1h', 'rolling_avg_High price_1d', 'rolling_avg_High price_1w',
                       'rolling_avg_High price_1m',
                       'rolling_max_High price_1h', 'rolling_max_High price_1d', 'rolling_max_High price_1w',
                       'rolling_max_High price_1m',
                       'rolling_min_High price_1h', 'rolling_min_High price_1d', 'rolling_min_High price_1w',
                       'rolling_min_High price_1m',
                       'rolling_avg_Low Price_1h', 'rolling_avg_Low Price_1d', 'rolling_avg_Low Price_1w',
                       'rolling_avg_Low Price_1m',
                       'rolling_max_Low Price_1h', 'rolling_max_Low Price_1d', 'rolling_max_Low Price_1w',
                       'rolling_max_Low Price_1m',
                       'rolling_min_Low Price_1h', 'rolling_min_Low Price_1d', 'rolling_min_Low Price_1w',
                       'rolling_min_Low Price_1m',
                       'rolling_avg_Close Price_1h', 'rolling_avg_Close Price_1d', 'rolling_avg_Close Price_1w',
                       'rolling_avg_Close Price_1m',
                       'rolling_max_Close Price_1h', 'rolling_max_Close Price_1d', 'rolling_max_Close Price_1w',
                       'rolling_max_Close Price_1m',
                       'rolling_min_Close Price_1h', 'rolling_min_Close Price_1d', 'rolling_min_Close Price_1w',
                       'rolling_min_Close Price_1m']

In [15]:
import tensorflow as tf
#from preprocessing_pipeline import preprocess_df
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm
import matplotlib.pyplot as plt
#from parameters_modeling import exclude_x, log_transformation_feat, continuous_features, val_split, test_split, location_data
#import wandb
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
#from wandb.keras import WandbCallback
import tensorflow as tf


AUTOTUNE = tf.data.experimental.AUTOTUNE


def select_last_year(df):
    df = df.sort_values('Open Time')
    df = df.iloc[-1000000:, :]
    df = df.reset_index(drop=True)
    print('Only Selecting the last 2 years of data')
    return df 

def time_features_func(df):
    df['Date'] = pd.to_datetime(df['Open Time'], unit = 'ms')
    df['Year'] = pd.DatetimeIndex(df['Date']).year
    df['Month'] = pd.DatetimeIndex(df['Date']).month
    df['Day of Week'] = pd.DatetimeIndex(df['Date']).dayofweek
    df['Day of Month'] = pd.DatetimeIndex(df['Date']).day
    df['Day of Year'] = pd.DatetimeIndex(df['Date']).dayofyear
    df['Week of Month'] = pd.DatetimeIndex(df['Date']).day // 7
    df['Week of Year'] = df['Date'].dt.isocalendar().week.astype('int64')
    df['hour'] =df.Date.dt.hour
    df['minute'] =df.Date.dt.minute
    df['minute_of_day'] = df.hour*60 + df.minute
    print('Time preprocessing done')
    return df

def create_target_variables(df):
    # df = df.drop('Unnamed: 0', axis=1)
    df['Mid Price'] = (df['Open Price'] + df['Close Price'])/2
    df['y'] = (df['Open Price'].shift(-1) + df['Close Price'].shift(-1))/2  # the target is predicting the next close price
    df['benchmark'] = df['Mid Price']
    print('Target preprocessing done')
    return df


def create_last_reporting(df, columns=['Open Price', 'Close Price', 'Mid Price']):
    """Twice periods previously w.r.t the y variable"""
    for column in columns:
        df[column+'_prev'] = df[column].shift(1)
    df = df.dropna()
    print('Last preprocessing done')
    return df

def create_deviations(df):
    df['high_low'] = df['High price'] - df['Low Price']
    df['high_close'] = df['High price'] - df['Close Price']
    df['high_open'] = df['High price'] - df['Open Price']
    df['open_low'] = df['Close Price'] - df['Low Price']
    df['spread'] =df['Close Price'] - df['Open Price']
    df['spread_ind'] = 1*(df['spread'] < 0)
    df['spread'] =np.abs(df.spread)
    print('Deviations preprocessing done')
    return df

def create_stats(df, features = ['high_low', 'high_close', 'high_open', 'open_low', 'spread', 'Open Price', 'High price', 'Low Price', 'Close Price']):
    for feature in features:
        series_1h, series_1d, series_1w, series_1m =  df[feature].rolling(window = 60), df[feature].rolling(window = 1500), df[feature].rolling(window = 10000), df[feature].rolling(window = 50000)
        df['rolling_avg_{}_1h'.format(feature)] = series_1d.mean() # rolling avg over 1 hour
        df['rolling_avg_{}_1d'.format(feature)] = series_1d.mean() # rolling avg over 1 day
        df['rolling_avg_{}_1w'.format(feature)] = series_1w.mean() # rolling avg over 1 week
        df['rolling_avg_{}_1m'.format(feature)] = series_1m.mean() # rolling avg over 1 month
        df['rolling_max_{}_1h'.format(feature)] = series_1d.max() # rolling max over 1 hour
        df['rolling_max_{}_1d'.format(feature)] = series_1d.max() # rolling max over 1 day
        df['rolling_max_{}_1w'.format(feature)] = series_1w.max() # rolling max over 1 week
        df['rolling_max_{}_1m'.format(feature)] = series_1m.max()  # rolling max over 1 month
        #df['rolling_min_{}_1h'.format(feature)] = series_1d.min() # rolling min over 1 hour
        #df['rolling_min_{}_1d'.format(feature)] = series_1d.min() # rolling min over 1 day
        #df['rolling_min_{}_1w'.format(feature)] = series_1w.min() # rolling min over 1 week
        #df['rolling_min_{}_1m'.format(feature)] = series_1m.min() # rolling min over 1 month
    df = df.dropna()
    print('Rolling preprocessing done')
    return df

def log_transformation(df, log_transformation_features=log_transformation_feat):
    for feature in log_transformation_features:
        df['log_{}'.format(feature)] = np.log(1+df[feature].values)
        df = df.drop(feature, axis=1)
    print('Logs preprocessing done')
    return df

def _split(df, train_val_date=val_split, val_test_date=test_split, exclude_x=exclude_x):
    # Standardize the dataframe
    df = df.reset_index(drop=True)
    if train_val_date is None and val_test_date is None:
        index_val, index_test = int(len(df)*0.8), int(len(df)*0.9)
        print(index_val, index_test)
        print(df.head(), len(df.index))
        train_val_date, val_test_date = df.loc[index_val, 'Date'], df.loc[index_test, 'Date']
    print('We are going to build a model with train/val date {} and val/test date {}'.format(train_val_date, val_test_date))
    df_train, df_val, df_test = df[df.Date <= pd.to_datetime(train_val_date)], df[(df.Date > pd.to_datetime(train_val_date)) & (df.Date <= pd.to_datetime(val_test_date))], df[df.Date > pd.to_datetime(val_test_date)]
    features_list = [col for col in df_train.columns if col not in exclude_x]
    X_train, X_val, X_test = df_train[features_list], df_val[features_list],df_test[features_list]
    y_train, y_val, y_test = df_train[['y']], df_val[['y']], df_test[['y']]
    print('Splits done')
    return X_train, X_val, X_test, y_train, y_val, y_test, features_list

def standardize(X_train, X_val, X_test, y_train, y_val, y_test):
    x_mean = X_train.mean()
    x_std = X_train.std()
    y_mean = y_train.mean()
    y_std = y_train.std()
    X_train, X_val, X_test = (X_train - x_mean) / x_std, (X_val - x_mean) / x_std, (X_test - x_mean) / x_std
    y_train, y_val, y_test = (y_train - y_mean) / y_std, (y_val - y_mean) / y_std, (y_test - y_mean) / y_std
    print('Standardization done')
    return X_train, X_val, X_test, y_train, y_val, y_test, x_mean, x_std, y_mean, y_std

def generate_input_sequences(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, input_seq_len, len(feats)))
    dfvs = data[feats].values
    for i in (range(input_seq_len, len(data)-output_seq_len+1)):
        x = np.expand_dims(dfvs[i-input_seq_len:i, :], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_output_sequences(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, output_seq_len, 1))
    dfvs = data[feats].values
    for i in range(input_seq_len, len(data)-output_seq_len+1):
        x = np.expand_dims(dfvs[i:i+output_seq_len, :], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_baseline_predictions(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, output_seq_len, 1))
    dfvs = data[feats].values
    for i in range(input_seq_len, len(data)-output_seq_len+1):
        x = np.expand_dims(np.ones((output_seq_len, 1))*dfvs[i][0], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_tf_data(x_train, x_val, y_train, y_val, y_train_baseline, y_val_baseline, batch_size):
    x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
    x_val = tf.convert_to_tensor(x_val, dtype=tf.float32)
    y_train = tf.convert_to_tensor(y_train-y_train_baseline, dtype=tf.float32)
    y_val = tf.convert_to_tensor(y_val-y_val_baseline, dtype=tf.float32)

    train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_data = train_data.batch(batch_size)
    train_data = train_data.prefetch(AUTOTUNE)

    validation_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    validation_data = validation_data.batch(batch_size)
    validation_data = validation_data.prefetch(AUTOTUNE)
    return train_data, validation_data

def preprocess_df(df, batch_size, location_data=location_data):
    df = select_last_year(df)
    df = time_features_func(df)
    df = create_target_variables(df)
    df = create_last_reporting(df)
    df = create_deviations(df)
    df = create_stats(df)
    df = log_transformation(df)
    X_train, X_val, X_test, y_train, y_val, y_test, features_list = _split(df)
    X_train, X_val, X_test, y_train, y_val, y_test, x_mean, x_std, y_mean, y_std = standardize(X_train, X_val, X_test, y_train, y_val, y_test)
    X_train_seq = generate_input_sequences(X_train, X_train.columns.values.tolist())
    X_val_seq = generate_input_sequences(X_val, X_val.columns.values.tolist())
    X_test_seq = generate_input_sequences(X_test, X_test.columns.values.tolist())
    print('Inputs done')
    y_train_seq = generate_output_sequences(y_train, y_train.columns.values.tolist())
    y_val_seq = generate_output_sequences(y_val, y_val.columns.values.tolist())
    y_test_seq = generate_output_sequences(y_test, y_test.columns.values.tolist())
    print('Outputs done')
    y_train_baseline = generate_baseline_predictions(y_train, y_train.columns.values.tolist())
    y_val_baseline = generate_baseline_predictions(y_val, y_val.columns.values.tolist())
    y_test_baseline = generate_baseline_predictions(y_test, y_test.columns.values.tolist())
    print('Baseline done')
    train_data, validation_data = generate_tf_data(X_train_seq, X_val_seq, y_train_seq, y_val_seq, y_train_baseline, y_val_baseline, batch_size)
    return train_data, validation_data, features_list, x_mean, x_std, y_mean, y_std

print('here')
df = pd.read_csv('/content/gdrive/MyDrive/playground/bnbbtc_data.csv', index_col=0)
batch_size = 128
lr = 1e-3
optimizer = 'Adam'
n_layers = 3
n_units = 32
dropout = 0.2
weight_decay = 1e-4
train_data, validation_data, features_list, x_mean, x_std, y_mean, y_std = preprocess_df(df, batch_size)

if optimizer == 'Adam':
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
elif optimizer == 'SGD':
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=500, decay_rate=0.9,staircase=True)  
    opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

lstm_model1 = tf.keras.models.Sequential()

if n_layers == 1:
    lstm_model1.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
else:
    for n in range(n_layers):
        if n == n_layers - 1:
            lstm_model1.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
            #lstm_model.add(batchNormalization())
            lstm_model1.add(tf.keras.layers.Dropout(dropout))
        else:
            lstm_model1.add(tf.keras.layers.LSTM(n_units, return_sequences=True))
            lstm_model1.add(BatchNormalization())
            lstm_model1.add(tf.keras.layers.Dropout(dropout))
            
lstm_model1.add(tf.keras.layers.Dense(units=n_units, kernel_regularizer=regularizers.l2(l2=weight_decay)))

MAX_EPOCHS = 10
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, mode='min')
lstm_model1.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=opt, metrics=[tf.metrics.MeanAbsoluteError(), tf.keras.losses.MeanSquaredError()])
# history = lstm_model.fit(train_data, epochs=MAX_EPOCHS, validation_data=validation_data, callbacks=[early_stopping, WandbCallback()])
history1 = lstm_model1.fit(train_data, epochs=MAX_EPOCHS, validation_data=validation_data, callbacks=[early_stopping])

here


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Only Selecting the last 2 years of data
Time preprocessing done
Target preprocessing done
Last preprocessing done
Deviations preprocessing done
Rolling preprocessing done
Logs preprocessing done
759999 854999
       Open Time  ...  log_rolling_max_Close Price_1m
0  1582257540000  ...                        0.002614
1  1582257600000  ...                        0.002614
2  1582257660000  ...                        0.002614
3  1582257720000  ...                        0.002614
4  1582257780000  ...                        0.002614

[5 rows x 107 columns] 949999
We are going to build a model with train/val date 2021-08-02 22:39:00 and val/test date 2021-10-08 04:29:00
Splits done
Standardization done
Inputs done
Outputs done
Baseline done
Epoch 1/10
5938/5938 [==============================] - 69s 10ms/step - loss: 4.7369e-04 - mean_absolute_error: 0.0051 - mean_squared_error: 2.4963e-04 - val_loss: 8.0415e-05 - val_mean_absolute_error: 0.0059 - val_mean_squared_error: 8.0224e-05
Epoch 2/10

In [16]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf


AUTOTUNE = tf.data.experimental.AUTOTUNE


def select_last_year(df):
    df = df.sort_values('Open Time')
    df = df.iloc[-1000000:, :]
    df = df.reset_index(drop=True)
    print('Only Selecting the last 2 years of data')
    return df 

def time_features_func(df):
    df['Date'] = pd.to_datetime(df['Open Time'], unit = 'ms')
    df['Year'] = pd.DatetimeIndex(df['Date']).year
    df['Month'] = pd.DatetimeIndex(df['Date']).month
    df['Day of Week'] = pd.DatetimeIndex(df['Date']).dayofweek
    df['Day of Month'] = pd.DatetimeIndex(df['Date']).day
    df['Day of Year'] = pd.DatetimeIndex(df['Date']).dayofyear
    df['Week of Month'] = pd.DatetimeIndex(df['Date']).day // 7
    df['Week of Year'] = df['Date'].dt.isocalendar().week.astype('int64')
    df['hour'] =df.Date.dt.hour
    df['minute'] =df.Date.dt.minute
    df['minute_of_day'] = df.hour*60 + df.minute
    print('Time preprocessing done')
    return df

def create_target_variables(df):
    # df = df.drop('Unnamed: 0', axis=1)
    df['Mid Price'] = (df['Open Price'] + df['Close Price'])/2
    df['y'] = (df['Open Price'].shift(-1) + df['Close Price'].shift(-1))/2  # the target is predicting the next close price
    df['benchmark'] = df['Mid Price']
    print('Target preprocessing done')
    return df


def create_last_reporting(df, columns=['Open Price', 'Close Price', 'Mid Price']):
    """Twice periods previously w.r.t the y variable"""
    for column in columns:
        df[column+'_prev'] = df[column].shift(1)
    df = df.dropna()
    print('Last preprocessing done')
    return df

def create_deviations(df):
    df['high_low'] = df['High price'] - df['Low Price']
    df['high_close'] = df['High price'] - df['Close Price']
    df['high_open'] = df['High price'] - df['Open Price']
    df['open_low'] = df['Close Price'] - df['Low Price']
    df['spread'] =df['Close Price'] - df['Open Price']
    df['spread_ind'] = 1*(df['spread'] < 0)
    df['spread'] =np.abs(df.spread)
    print('Deviations preprocessing done')
    return df

def create_stats(df, features = ['high_low', 'high_close', 'high_open', 'open_low', 'spread', 'Open Price', 'High price', 'Low Price', 'Close Price']):
    for feature in features:
        series_1h, series_1d, series_1w, series_1m =  df[feature].rolling(window = 60), df[feature].rolling(window = 1500), df[feature].rolling(window = 10000), df[feature].rolling(window = 50000)
        df['rolling_avg_{}_1h'.format(feature)] = series_1d.mean() # rolling avg over 1 hour
        df['rolling_avg_{}_1d'.format(feature)] = series_1d.mean() # rolling avg over 1 day
        df['rolling_avg_{}_1w'.format(feature)] = series_1w.mean() # rolling avg over 1 week
        df['rolling_avg_{}_1m'.format(feature)] = series_1m.mean() # rolling avg over 1 month
        df['rolling_max_{}_1h'.format(feature)] = series_1d.max() # rolling max over 1 hour
        df['rolling_max_{}_1d'.format(feature)] = series_1d.max() # rolling max over 1 day
        df['rolling_max_{}_1w'.format(feature)] = series_1w.max() # rolling max over 1 week
        df['rolling_max_{}_1m'.format(feature)] = series_1m.max()  # rolling max over 1 month
        #df['rolling_min_{}_1h'.format(feature)] = series_1d.min() # rolling min over 1 hour
        #df['rolling_min_{}_1d'.format(feature)] = series_1d.min() # rolling min over 1 day
        #df['rolling_min_{}_1w'.format(feature)] = series_1w.min() # rolling min over 1 week
        #df['rolling_min_{}_1m'.format(feature)] = series_1m.min() # rolling min over 1 month
    df = df.dropna()
    print('Rolling preprocessing done')
    return df

def log_transformation(df, log_transformation_features=log_transformation_feat):
    for feature in log_transformation_features:
        df['log_{}'.format(feature)] = np.log(1+df[feature].values)
        df = df.drop(feature, axis=1)
    print('Logs preprocessing done')
    return df

def _split(df, train_val_date=val_split, val_test_date=test_split, exclude_x=exclude_x):
    # Standardize the dataframe
    df = df.reset_index(drop=True)
    features_list = [col for col in df.columns if col not in exclude_x]
    X = df[features_list]
    y = df[['y']]
    print('Splits done')
    return X, y, features_list

def standardize(X, y):
    x_mean = X.mean()
    x_std = X.std()
    y_mean = y.mean()
    y_std = y.std()
    X = (X - x_mean) / x_std
    y = (y - y_mean) / y_std
    print('Standardization done')
    return X, y, x_mean, x_std, y_mean, y_std

def generate_input_sequences(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, input_seq_len, len(feats)))
    dfvs = data[feats].values
    for i in (range(input_seq_len, len(data)-output_seq_len+1)):
        x = np.expand_dims(dfvs[i-input_seq_len:i, :], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_output_sequences(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, output_seq_len, 1))
    dfvs = data[feats].values
    for i in range(input_seq_len, len(data)-output_seq_len+1):
        x = np.expand_dims(dfvs[i:i+output_seq_len, :], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_baseline_predictions(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, output_seq_len, 1))
    dfvs = data[feats].values
    for i in range(input_seq_len, len(data)-output_seq_len+1):
        x = np.expand_dims(np.ones((output_seq_len, 1))*dfvs[i][0], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_tf_data(x_train, y_train, y_train_baseline, batch_size):
    x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
    y_train = tf.convert_to_tensor(y_train-y_train_baseline, dtype=tf.float32)

    train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_data = train_data.batch(batch_size)
    train_data = train_data.prefetch(AUTOTUNE)
    return train_data

def preprocess_df(df, batch_size, location_data=location_data):
    df = select_last_year(df)
    df = time_features_func(df)
    df = create_target_variables(df)
    df = create_last_reporting(df)
    df = create_deviations(df)
    df = create_stats(df)
    df = log_transformation(df)
    X_train, y_train, features_list = _split(df)
    # display(X_train) WORKS
    X_train, y_train, x_mean, x_std, y_mean, y_std = standardize(X_train, y_train)
    # print(X_train.shape) WORKS
    X_train_seq = generate_input_sequences(X_train, X_train.columns.values.tolist())
    print(X_train_seq.shape)
    print('Inputs done')
    y_train_seq = generate_output_sequences(y_train, y_train.columns.values.tolist())
    print(y_train_seq.shape)
    print('Outputs done')
    y_train_baseline = generate_baseline_predictions(y_train, y_train.columns.values.tolist())
    print(y_train_baseline.shape)
    print('Baseline done')
    train_data = generate_tf_data(X_train_seq, y_train_seq, y_train_baseline, batch_size)
    return train_data, features_list, x_mean, x_std, y_mean, y_std

print('here')
df = pd.read_csv('/content/gdrive/MyDrive/playground/bnbbtc_data.csv', index_col=0)
batch_size = 128
lr = 1e-3
optimizer = 'Adam'
n_layers = 3
n_units = 32
dropout = 0.2
weight_decay = 1e-4
train_data, features_list, x_mean, x_std, y_mean, y_std = preprocess_df(df, batch_size)

if optimizer == 'Adam':
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
elif optimizer == 'SGD':
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=500, decay_rate=0.9,staircase=True)  
    opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

lstm_model = tf.keras.models.Sequential()

if n_layers == 1:
    lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
else:
    for n in range(n_layers):
        if n == n_layers - 1:
            lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
            #lstm_model.add(batchNormalization())
            lstm_model.add(tf.keras.layers.Dropout(dropout))
        else:
            lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=True))
            lstm_model.add(BatchNormalization())
            lstm_model.add(tf.keras.layers.Dropout(dropout))
            
lstm_model.add(tf.keras.layers.Dense(units=n_units, kernel_regularizer=regularizers.l2(l2=weight_decay)))

MAX_EPOCHS = 10
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, mode='min')
lstm_model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=opt, metrics=[tf.metrics.MeanAbsoluteError(), tf.keras.losses.MeanSquaredError()])
# history = lstm_model.fit(train_data, epochs=MAX_EPOCHS, validation_data=validation_data, callbacks=[early_stopping, WandbCallback()])
history = lstm_model.fit(train_data, epochs=MAX_EPOCHS, callbacks=[early_stopping])

here


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Only Selecting the last 2 years of data
Time preprocessing done
Target preprocessing done
Last preprocessing done
Deviations preprocessing done
Rolling preprocessing done
Logs preprocessing done
Splits done
Standardization done
(949960, 8, 102)
Inputs done
(949960, 32, 1)
Outputs done
(949960, 32, 1)
Baseline done
Epoch 1/10
7422/7422 [==============================] - 79s 10ms/step - loss: 3.6203e-04 - mean_absolute_error: 0.0046 - mean_squared_error: 1.8313e-04
Epoch 2/10
7422/7422 [==============================] - 72s 10ms/step - loss: 5.2998e-05 - mean_absolute_error: 0.0034 - mean_squared_error: 5.2644e-05
Epoch 3/10
7422/7422 [==============================] - 73s 10ms/step - loss: 5.0854e-05 - mean_absolute_error: 0.0034 - mean_squared_error: 5.0267e-05
Epoch 4/10
7422/7422 [==============================] - 73s 10ms/step - loss: 4.8001e-05 - mean_absolute_error: 0.0033 - mean_squared_error: 4.7254e-05
Epoch 5/10
7422/7422 [==============================] - 72s 10ms/step - loss

In [17]:
# !rm -r /gdrive/MyDrive/playground/LSTM_BNBBTC_12-12-2021/

In [18]:
import json

data = {}

data['validation_score'] = history1.history['val_loss'][-1]
data['unique_id'] = time.time()
data['training_time_stamp'] = time.time()
data['hyperparameters'] = {'batch_size': 128, 'lr': 1e-3, 'optimizer': 'Adam', 'n_layers': 3, 'n_units': 8, 'dropout': 0.2, 'weight_decay': 1e-4}
data['symbol'] = 'BNBBTC'
data['x_mean'] = x_mean.values.tolist()
data['x_std'] = x_std.values.tolist()
data['y_mean'] = y_mean.values[0]
data['y_std'] = y_std.values[0]

with open('model_metrics_BNBBTC.json', 'w') as outfile:
    json.dump(data, outfile)

In [19]:
!mkdir /content/gdrive/MyDrive/playground/LSTM_BNBBTC_$(date +"%d-%m-%Y")

In [20]:
!cp /content/model_metrics_BNBBTC.json /content/gdrive/MyDrive/playground/LSTM_BNBBTC_13-12-2021/

In [21]:
lstm_model.save_weights('/content/gdrive/MyDrive/playground/LSTM_BNBBTC_13-12-2021/lstm_BNBBTC')

In [22]:
!gsutil -m cp -r /content/gdrive/MyDrive/playground/LSTM_BNBBTC_13-12-2021/ gs://crypto-forecasting-bucket/BNBBTC/Model/

Copying file:///content/gdrive/MyDrive/playground/LSTM_BNBBTC_13-12-2021/model_metrics_BNBBTC.json [Content-Type=application/json]...
Copying file:///content/gdrive/MyDrive/playground/LSTM_BNBBTC_13-12-2021/lstm_BNBBTC.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file:///content/gdrive/MyDrive/playground/LSTM_BNBBTC_13-12-2021/lstm_BNBBTC.index [Content-Type=application/octet-stream]...
Copying file:///content/gdrive/MyDrive/playground/LSTM_BNBBTC_13-12-2021/checkpoint [Content-Type=application/octet-stream]...
/ [4/4 files][428.2 KiB/428.2 KiB] 100% Done                                    
Operation completed over 4 objects/428.2 KiB.                                    


# BNBUSDT MODELING

In [4]:
import os

location_data = '/content/gdrive/MyDrive/playground/bnbusdt_data.csv'

exclude_x = ['NA', 'Open Time', 'Close Time', 'y', 'Date']

val_split, test_split = None, None

log_transformation_feat = ['Open Price', 'High price', 'Low Price', 'Close Price', 'Volume Traded',
                           'Quote asset Volume', 'Number of Trades', 'Taker buy base asset volume',
                           'Taker buy quote asset volume', 'Mid Price', 'Open Price_prev', 'Close Price_prev',
                           'Mid Price_prev', 'high_low', 'high_close', 'high_open', 'open_low', 'spread',
                           'rolling_avg_high_low_1h', 'rolling_avg_high_low_1d', 'rolling_avg_high_low_1w',
                           'rolling_avg_high_low_1m', 'rolling_max_high_low_1h',
                           'rolling_max_high_low_1d', 'rolling_max_high_low_1w', 'rolling_max_high_low_1m',
                           'rolling_avg_high_close_1h', 'rolling_avg_high_close_1d', 'rolling_avg_high_close_1w',
                           'rolling_avg_high_close_1m', 'rolling_max_high_close_1h', 'rolling_max_high_close_1d',
                           'rolling_max_high_close_1w', 'rolling_max_high_close_1m', 'rolling_avg_high_open_1h',
                           'rolling_avg_high_open_1d', 'rolling_avg_high_open_1w', 'rolling_avg_high_open_1m',
                           'rolling_max_high_open_1h', 'rolling_max_high_open_1d',
                           'rolling_max_high_open_1w', 'rolling_max_high_open_1m', 'rolling_avg_open_low_1h',
                           'rolling_avg_open_low_1d',
                           'rolling_avg_open_low_1w', 'rolling_avg_open_low_1m', 'rolling_max_open_low_1h',
                           'rolling_max_open_low_1d',
                           'rolling_max_open_low_1w', 'rolling_max_open_low_1m', 'rolling_avg_spread_1h',
                           'rolling_avg_spread_1d',
                           'rolling_avg_spread_1w', 'rolling_avg_spread_1m', 'rolling_max_spread_1h',
                           'rolling_max_spread_1d', 'rolling_max_spread_1w',
                           'rolling_max_spread_1m', 'rolling_avg_Open Price_1h', 'rolling_avg_Open Price_1d',
                           'rolling_avg_Open Price_1w', 'rolling_avg_Open Price_1m',
                           'rolling_max_Open Price_1h', 'rolling_max_Open Price_1d', 'rolling_max_Open Price_1w',
                           'rolling_max_Open Price_1m',
                           'rolling_avg_High price_1h', 'rolling_avg_High price_1d', 'rolling_avg_High price_1w',
                           'rolling_avg_High price_1m',
                           'rolling_max_High price_1h', 'rolling_max_High price_1d', 'rolling_max_High price_1w',
                           'rolling_max_High price_1m',
                           'rolling_avg_Low Price_1h', 'rolling_avg_Low Price_1d', 'rolling_avg_Low Price_1w',
                           'rolling_avg_Low Price_1m',
                           'rolling_max_Low Price_1h', 'rolling_max_Low Price_1d', 'rolling_max_Low Price_1w',
                           'rolling_max_Low Price_1m',
                           'rolling_avg_Close Price_1h', 'rolling_avg_Close Price_1d', 'rolling_avg_Close Price_1w',
                           'rolling_avg_Close Price_1m',
                           'rolling_max_Close Price_1h', 'rolling_max_Close Price_1d', 'rolling_max_Close Price_1w',
                           'rolling_max_Close Price_1m']
# features where we apply log transformation


continuous_features = ['Open Price', 'High price', 'Low Price', 'Close Price', 'Volume Traded',
                       'Quote asset Volume', 'Number of Trades', 'Taker buy base asset volume',
                       'Taker buy quote asset volume', 'Mid Price', 'Open Price_prev', 'Close Price_prev',
                       'Mid Price_prev', 'high_low', 'high_close', 'high_open', 'open_low', 'spread',
                       'rolling_avg_high_low_1h', 'rolling_avg_high_low_1d', 'rolling_avg_high_low_1w',
                       'rolling_avg_high_low_1m', 'rolling_max_high_low_1h',
                       'rolling_max_high_low_1d', 'rolling_max_high_low_1w', 'rolling_max_high_low_1m',
                       'rolling_min_high_low_1h', 'rolling_min_high_low_1d',
                       'rolling_min_high_low_1w', 'rolling_min_high_low_1m', 'rolling_avg_high_close_1h',
                       'rolling_avg_high_close_1d', 'rolling_avg_high_close_1w', 'rolling_avg_high_close_1m',
                       'rolling_max_high_close_1h', 'rolling_max_high_close_1d', 'rolling_max_high_close_1w',
                       'rolling_max_high_close_1m', 'rolling_min_high_close_1h', 'rolling_min_high_close_1d',
                       'rolling_min_high_close_1w', 'rolling_min_high_close_1m', 'rolling_avg_high_open_1h',
                       'rolling_avg_high_open_1d', 'rolling_avg_high_open_1w', 'rolling_avg_high_open_1m',
                       'rolling_max_high_open_1h', 'rolling_max_high_open_1d',
                       'rolling_max_high_open_1w', 'rolling_max_high_open_1m', 'rolling_min_high_open_1h',
                       'rolling_min_high_open_1d',
                       'rolling_min_high_open_1w', 'rolling_min_high_open_1m', 'rolling_avg_open_low_1h',
                       'rolling_avg_open_low_1d',
                       'rolling_avg_open_low_1w', 'rolling_avg_open_low_1m', 'rolling_max_open_low_1h',
                       'rolling_max_open_low_1d',
                       'rolling_max_open_low_1w', 'rolling_max_open_low_1m', 'rolling_min_open_low_1h',
                       'rolling_min_open_low_1d',
                       'rolling_min_open_low_1w', 'rolling_min_open_low_1m', 'rolling_avg_spread_1h',
                       'rolling_avg_spread_1d',
                       'rolling_avg_spread_1w', 'rolling_avg_spread_1m', 'rolling_max_spread_1h',
                       'rolling_max_spread_1d', 'rolling_max_spread_1w',
                       'rolling_max_spread_1m', 'rolling_min_spread_1h', 'rolling_min_spread_1d',
                       'rolling_min_spread_1w', 'rolling_min_spread_1m',
                       'rolling_avg_Open Price_1h', 'rolling_avg_Open Price_1d', 'rolling_avg_Open Price_1w',
                       'rolling_avg_Open Price_1m',
                       'rolling_max_Open Price_1h', 'rolling_max_Open Price_1d', 'rolling_max_Open Price_1w',
                       'rolling_max_Open Price_1m',
                       'rolling_min_Open Price_1h', 'rolling_min_Open Price_1d', 'rolling_min_Open Price_1w',
                       'rolling_min_Open Price_1m',
                       'rolling_avg_High price_1h', 'rolling_avg_High price_1d', 'rolling_avg_High price_1w',
                       'rolling_avg_High price_1m',
                       'rolling_max_High price_1h', 'rolling_max_High price_1d', 'rolling_max_High price_1w',
                       'rolling_max_High price_1m',
                       'rolling_min_High price_1h', 'rolling_min_High price_1d', 'rolling_min_High price_1w',
                       'rolling_min_High price_1m',
                       'rolling_avg_Low Price_1h', 'rolling_avg_Low Price_1d', 'rolling_avg_Low Price_1w',
                       'rolling_avg_Low Price_1m',
                       'rolling_max_Low Price_1h', 'rolling_max_Low Price_1d', 'rolling_max_Low Price_1w',
                       'rolling_max_Low Price_1m',
                       'rolling_min_Low Price_1h', 'rolling_min_Low Price_1d', 'rolling_min_Low Price_1w',
                       'rolling_min_Low Price_1m',
                       'rolling_avg_Close Price_1h', 'rolling_avg_Close Price_1d', 'rolling_avg_Close Price_1w',
                       'rolling_avg_Close Price_1m',
                       'rolling_max_Close Price_1h', 'rolling_max_Close Price_1d', 'rolling_max_Close Price_1w',
                       'rolling_max_Close Price_1m',
                       'rolling_min_Close Price_1h', 'rolling_min_Close Price_1d', 'rolling_min_Close Price_1w',
                       'rolling_min_Close Price_1m']

In [5]:
import tensorflow as tf
#from preprocessing_pipeline import preprocess_df
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm
import matplotlib.pyplot as plt
#from parameters_modeling import exclude_x, log_transformation_feat, continuous_features, val_split, test_split, location_data
#import wandb
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
#from wandb.keras import WandbCallback
import tensorflow as tf


AUTOTUNE = tf.data.experimental.AUTOTUNE


def select_last_year(df):
    df = df.sort_values('Open Time')
    df = df.iloc[-1000000:, :]
    df = df.reset_index(drop=True)
    print('Only Selecting the last 2 years of data')
    return df 

def time_features_func(df):
    df['Date'] = pd.to_datetime(df['Open Time'], unit = 'ms')
    df['Year'] = pd.DatetimeIndex(df['Date']).year
    df['Month'] = pd.DatetimeIndex(df['Date']).month
    df['Day of Week'] = pd.DatetimeIndex(df['Date']).dayofweek
    df['Day of Month'] = pd.DatetimeIndex(df['Date']).day
    df['Day of Year'] = pd.DatetimeIndex(df['Date']).dayofyear
    df['Week of Month'] = pd.DatetimeIndex(df['Date']).day // 7
    df['Week of Year'] = df['Date'].dt.isocalendar().week.astype('int64')
    df['hour'] =df.Date.dt.hour
    df['minute'] =df.Date.dt.minute
    df['minute_of_day'] = df.hour*60 + df.minute
    print('Time preprocessing done')
    return df

def create_target_variables(df):
    # df = df.drop('Unnamed: 0', axis=1)
    df['Mid Price'] = (df['Open Price'] + df['Close Price'])/2
    df['y'] = (df['Open Price'].shift(-1) + df['Close Price'].shift(-1))/2  # the target is predicting the next close price
    df['benchmark'] = df['Mid Price']
    print('Target preprocessing done')
    return df


def create_last_reporting(df, columns=['Open Price', 'Close Price', 'Mid Price']):
    """Twice periods previously w.r.t the y variable"""
    for column in columns:
        df[column+'_prev'] = df[column].shift(1)
    df = df.dropna()
    print('Last preprocessing done')
    return df

def create_deviations(df):
    df['high_low'] = df['High price'] - df['Low Price']
    df['high_close'] = df['High price'] - df['Close Price']
    df['high_open'] = df['High price'] - df['Open Price']
    df['open_low'] = df['Close Price'] - df['Low Price']
    df['spread'] =df['Close Price'] - df['Open Price']
    df['spread_ind'] = 1*(df['spread'] < 0)
    df['spread'] =np.abs(df.spread)
    print('Deviations preprocessing done')
    return df

def create_stats(df, features = ['high_low', 'high_close', 'high_open', 'open_low', 'spread', 'Open Price', 'High price', 'Low Price', 'Close Price']):
    for feature in features:
        series_1h, series_1d, series_1w, series_1m =  df[feature].rolling(window = 60), df[feature].rolling(window = 1500), df[feature].rolling(window = 10000), df[feature].rolling(window = 50000)
        df['rolling_avg_{}_1h'.format(feature)] = series_1d.mean() # rolling avg over 1 hour
        df['rolling_avg_{}_1d'.format(feature)] = series_1d.mean() # rolling avg over 1 day
        df['rolling_avg_{}_1w'.format(feature)] = series_1w.mean() # rolling avg over 1 week
        df['rolling_avg_{}_1m'.format(feature)] = series_1m.mean() # rolling avg over 1 month
        df['rolling_max_{}_1h'.format(feature)] = series_1d.max() # rolling max over 1 hour
        df['rolling_max_{}_1d'.format(feature)] = series_1d.max() # rolling max over 1 day
        df['rolling_max_{}_1w'.format(feature)] = series_1w.max() # rolling max over 1 week
        df['rolling_max_{}_1m'.format(feature)] = series_1m.max()  # rolling max over 1 month
        #df['rolling_min_{}_1h'.format(feature)] = series_1d.min() # rolling min over 1 hour
        #df['rolling_min_{}_1d'.format(feature)] = series_1d.min() # rolling min over 1 day
        #df['rolling_min_{}_1w'.format(feature)] = series_1w.min() # rolling min over 1 week
        #df['rolling_min_{}_1m'.format(feature)] = series_1m.min() # rolling min over 1 month
    df = df.dropna()
    print('Rolling preprocessing done')
    return df

def log_transformation(df, log_transformation_features=log_transformation_feat):
    for feature in log_transformation_features:
        df['log_{}'.format(feature)] = np.log(1+df[feature].values)
        df = df.drop(feature, axis=1)
    print('Logs preprocessing done')
    return df

def _split(df, train_val_date=val_split, val_test_date=test_split, exclude_x=exclude_x):
    # Standardize the dataframe
    df = df.reset_index(drop=True)
    if train_val_date is None and val_test_date is None:
        index_val, index_test = int(len(df)*0.8), int(len(df)*0.9)
        print(index_val, index_test)
        print(df.head(), len(df.index))
        train_val_date, val_test_date = df.loc[index_val, 'Date'], df.loc[index_test, 'Date']
    print('We are going to build a model with train/val date {} and val/test date {}'.format(train_val_date, val_test_date))
    df_train, df_val, df_test = df[df.Date <= pd.to_datetime(train_val_date)], df[(df.Date > pd.to_datetime(train_val_date)) & (df.Date <= pd.to_datetime(val_test_date))], df[df.Date > pd.to_datetime(val_test_date)]
    features_list = [col for col in df_train.columns if col not in exclude_x]
    X_train, X_val, X_test = df_train[features_list], df_val[features_list],df_test[features_list]
    y_train, y_val, y_test = df_train[['y']], df_val[['y']], df_test[['y']]
    print('Splits done')
    return X_train, X_val, X_test, y_train, y_val, y_test, features_list

def standardize(X_train, X_val, X_test, y_train, y_val, y_test):
    x_mean = X_train.mean()
    x_std = X_train.std()
    y_mean = y_train.mean()
    y_std = y_train.std()
    X_train, X_val, X_test = (X_train - x_mean) / x_std, (X_val - x_mean) / x_std, (X_test - x_mean) / x_std
    y_train, y_val, y_test = (y_train - y_mean) / y_std, (y_val - y_mean) / y_std, (y_test - y_mean) / y_std
    print('Standardization done')
    return X_train, X_val, X_test, y_train, y_val, y_test, x_mean, x_std, y_mean, y_std

def generate_input_sequences(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, input_seq_len, len(feats)))
    dfvs = data[feats].values
    for i in (range(input_seq_len, len(data)-output_seq_len+1)):
        x = np.expand_dims(dfvs[i-input_seq_len:i, :], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_output_sequences(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, output_seq_len, 1))
    dfvs = data[feats].values
    for i in range(input_seq_len, len(data)-output_seq_len+1):
        x = np.expand_dims(dfvs[i:i+output_seq_len, :], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_baseline_predictions(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, output_seq_len, 1))
    dfvs = data[feats].values
    for i in range(input_seq_len, len(data)-output_seq_len+1):
        x = np.expand_dims(np.ones((output_seq_len, 1))*dfvs[i][0], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_tf_data(x_train, x_val, y_train, y_val, y_train_baseline, y_val_baseline, batch_size):
    x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
    x_val = tf.convert_to_tensor(x_val, dtype=tf.float32)
    y_train = tf.convert_to_tensor(y_train-y_train_baseline, dtype=tf.float32)
    y_val = tf.convert_to_tensor(y_val-y_val_baseline, dtype=tf.float32)

    train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_data = train_data.batch(batch_size)
    train_data = train_data.prefetch(AUTOTUNE)

    validation_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    validation_data = validation_data.batch(batch_size)
    validation_data = validation_data.prefetch(AUTOTUNE)
    return train_data, validation_data

def preprocess_df(df, batch_size, location_data=location_data):
    df = select_last_year(df)
    df = time_features_func(df)
    df = create_target_variables(df)
    df = create_last_reporting(df)
    df = create_deviations(df)
    df = create_stats(df)
    df = log_transformation(df)
    X_train, X_val, X_test, y_train, y_val, y_test, features_list = _split(df)
    X_train, X_val, X_test, y_train, y_val, y_test, x_mean, x_std, y_mean, y_std = standardize(X_train, X_val, X_test, y_train, y_val, y_test)
    X_train_seq = generate_input_sequences(X_train, X_train.columns.values.tolist())
    X_val_seq = generate_input_sequences(X_val, X_val.columns.values.tolist())
    X_test_seq = generate_input_sequences(X_test, X_test.columns.values.tolist())
    print('Inputs done')
    y_train_seq = generate_output_sequences(y_train, y_train.columns.values.tolist())
    y_val_seq = generate_output_sequences(y_val, y_val.columns.values.tolist())
    y_test_seq = generate_output_sequences(y_test, y_test.columns.values.tolist())
    print('Outputs done')
    y_train_baseline = generate_baseline_predictions(y_train, y_train.columns.values.tolist())
    y_val_baseline = generate_baseline_predictions(y_val, y_val.columns.values.tolist())
    y_test_baseline = generate_baseline_predictions(y_test, y_test.columns.values.tolist())
    print('Baseline done')
    train_data, validation_data = generate_tf_data(X_train_seq, X_val_seq, y_train_seq, y_val_seq, y_train_baseline, y_val_baseline, batch_size)
    return train_data, validation_data, features_list, x_mean, x_std, y_mean, y_std

print('here')
df = pd.read_csv('/content/gdrive/MyDrive/playground/bnbusdt_data.csv', index_col=0)
batch_size = 128
lr = 1e-3
optimizer = 'Adam'
n_layers = 3
n_units = 32
dropout = 0.2
weight_decay = 1e-4
train_data, validation_data, features_list, x_mean, x_std, y_mean, y_std = preprocess_df(df, batch_size)

if optimizer == 'Adam':
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
elif optimizer == 'SGD':
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=500, decay_rate=0.9,staircase=True)  
    opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

lstm_model1 = tf.keras.models.Sequential()

if n_layers == 1:
    lstm_model1.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
else:
    for n in range(n_layers):
        if n == n_layers - 1:
            lstm_model1.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
            #lstm_model.add(batchNormalization())
            lstm_model1.add(tf.keras.layers.Dropout(dropout))
        else:
            lstm_model1.add(tf.keras.layers.LSTM(n_units, return_sequences=True))
            lstm_model1.add(BatchNormalization())
            lstm_model1.add(tf.keras.layers.Dropout(dropout))
            
lstm_model1.add(tf.keras.layers.Dense(units=n_units, kernel_regularizer=regularizers.l2(l2=weight_decay)))

MAX_EPOCHS = 10
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, mode='min')
lstm_model1.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=opt, metrics=[tf.metrics.MeanAbsoluteError(), tf.keras.losses.MeanSquaredError()])
# history = lstm_model.fit(train_data, epochs=MAX_EPOCHS, validation_data=validation_data, callbacks=[early_stopping, WandbCallback()])
history1 = lstm_model1.fit(train_data, epochs=MAX_EPOCHS, validation_data=validation_data, callbacks=[early_stopping])

here


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Only Selecting the last 2 years of data
Time preprocessing done
Target preprocessing done
Last preprocessing done
Deviations preprocessing done
Rolling preprocessing done
Logs preprocessing done
759999 854999
       Open Time  ...  log_rolling_max_Close Price_1m
0  1582261200000  ...                        3.338463
1  1582261260000  ...                        3.338463
2  1582261320000  ...                        3.338463
3  1582261380000  ...                        3.338463
4  1582261440000  ...                        3.338463

[5 rows x 107 columns] 949999
We are going to build a model with train/val date 2021-08-02 23:41:00 and val/test date 2021-10-08 05:31:00
Splits done
Standardization done
Inputs done
Outputs done
Baseline done
Epoch 1/10
5938/5938 [==============================] - 55s 8ms/step - loss: 5.0862e-04 - mean_absolute_error: 0.0055 - mean_squared_error: 2.8590e-04 - val_loss: 1.3513e-04 - val_mean_absolute_error: 0.0074 - val_mean_squared_error: 1.3494e-04
Epoch 2/10


In [7]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf


AUTOTUNE = tf.data.experimental.AUTOTUNE


def select_last_year(df):
    df = df.sort_values('Open Time')
    df = df.iloc[-1000000:, :]
    df = df.reset_index(drop=True)
    print('Only Selecting the last 2 years of data')
    return df 

def time_features_func(df):
    df['Date'] = pd.to_datetime(df['Open Time'], unit = 'ms')
    df['Year'] = pd.DatetimeIndex(df['Date']).year
    df['Month'] = pd.DatetimeIndex(df['Date']).month
    df['Day of Week'] = pd.DatetimeIndex(df['Date']).dayofweek
    df['Day of Month'] = pd.DatetimeIndex(df['Date']).day
    df['Day of Year'] = pd.DatetimeIndex(df['Date']).dayofyear
    df['Week of Month'] = pd.DatetimeIndex(df['Date']).day // 7
    df['Week of Year'] = df['Date'].dt.isocalendar().week.astype('int64')
    df['hour'] =df.Date.dt.hour
    df['minute'] =df.Date.dt.minute
    df['minute_of_day'] = df.hour*60 + df.minute
    print('Time preprocessing done')
    return df

def create_target_variables(df):
    # df = df.drop('Unnamed: 0', axis=1)
    df['Mid Price'] = (df['Open Price'] + df['Close Price'])/2
    df['y'] = (df['Open Price'].shift(-1) + df['Close Price'].shift(-1))/2  # the target is predicting the next close price
    df['benchmark'] = df['Mid Price']
    print('Target preprocessing done')
    return df


def create_last_reporting(df, columns=['Open Price', 'Close Price', 'Mid Price']):
    """Twice periods previously w.r.t the y variable"""
    for column in columns:
        df[column+'_prev'] = df[column].shift(1)
    df = df.dropna()
    print('Last preprocessing done')
    return df

def create_deviations(df):
    df['high_low'] = df['High price'] - df['Low Price']
    df['high_close'] = df['High price'] - df['Close Price']
    df['high_open'] = df['High price'] - df['Open Price']
    df['open_low'] = df['Close Price'] - df['Low Price']
    df['spread'] =df['Close Price'] - df['Open Price']
    df['spread_ind'] = 1*(df['spread'] < 0)
    df['spread'] =np.abs(df.spread)
    print('Deviations preprocessing done')
    return df

def create_stats(df, features = ['high_low', 'high_close', 'high_open', 'open_low', 'spread', 'Open Price', 'High price', 'Low Price', 'Close Price']):
    for feature in features:
        series_1h, series_1d, series_1w, series_1m =  df[feature].rolling(window = 60), df[feature].rolling(window = 1500), df[feature].rolling(window = 10000), df[feature].rolling(window = 50000)
        df['rolling_avg_{}_1h'.format(feature)] = series_1d.mean() # rolling avg over 1 hour
        df['rolling_avg_{}_1d'.format(feature)] = series_1d.mean() # rolling avg over 1 day
        df['rolling_avg_{}_1w'.format(feature)] = series_1w.mean() # rolling avg over 1 week
        df['rolling_avg_{}_1m'.format(feature)] = series_1m.mean() # rolling avg over 1 month
        df['rolling_max_{}_1h'.format(feature)] = series_1d.max() # rolling max over 1 hour
        df['rolling_max_{}_1d'.format(feature)] = series_1d.max() # rolling max over 1 day
        df['rolling_max_{}_1w'.format(feature)] = series_1w.max() # rolling max over 1 week
        df['rolling_max_{}_1m'.format(feature)] = series_1m.max()  # rolling max over 1 month
        #df['rolling_min_{}_1h'.format(feature)] = series_1d.min() # rolling min over 1 hour
        #df['rolling_min_{}_1d'.format(feature)] = series_1d.min() # rolling min over 1 day
        #df['rolling_min_{}_1w'.format(feature)] = series_1w.min() # rolling min over 1 week
        #df['rolling_min_{}_1m'.format(feature)] = series_1m.min() # rolling min over 1 month
    df = df.dropna()
    print('Rolling preprocessing done')
    return df

def log_transformation(df, log_transformation_features=log_transformation_feat):
    for feature in log_transformation_features:
        df['log_{}'.format(feature)] = np.log(1+df[feature].values)
        df = df.drop(feature, axis=1)
    print('Logs preprocessing done')
    return df

def _split(df, train_val_date=val_split, val_test_date=test_split, exclude_x=exclude_x):
    # Standardize the dataframe
    df = df.reset_index(drop=True)
    features_list = [col for col in df.columns if col not in exclude_x]
    X = df[features_list]
    y = df[['y']]
    print('Splits done')
    return X, y, features_list

def standardize(X, y):
    x_mean = X.mean()
    x_std = X.std()
    y_mean = y.mean()
    y_std = y.std()
    X = (X - x_mean) / x_std
    y = (y - y_mean) / y_std
    print('Standardization done')
    return X, y, x_mean, x_std, y_mean, y_std

def generate_input_sequences(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, input_seq_len, len(feats)))
    dfvs = data[feats].values
    for i in (range(input_seq_len, len(data)-output_seq_len+1)):
        x = np.expand_dims(dfvs[i-input_seq_len:i, :], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_output_sequences(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, output_seq_len, 1))
    dfvs = data[feats].values
    for i in range(input_seq_len, len(data)-output_seq_len+1):
        x = np.expand_dims(dfvs[i:i+output_seq_len, :], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_baseline_predictions(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, output_seq_len, 1))
    dfvs = data[feats].values
    for i in range(input_seq_len, len(data)-output_seq_len+1):
        x = np.expand_dims(np.ones((output_seq_len, 1))*dfvs[i][0], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_tf_data(x_train, y_train, y_train_baseline, batch_size):
    x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
    y_train = tf.convert_to_tensor(y_train-y_train_baseline, dtype=tf.float32)

    train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_data = train_data.batch(batch_size)
    train_data = train_data.prefetch(AUTOTUNE)
    return train_data

def preprocess_df(df, batch_size, location_data=location_data):
    df = select_last_year(df)
    df = time_features_func(df)
    df = create_target_variables(df)
    df = create_last_reporting(df)
    df = create_deviations(df)
    df = create_stats(df)
    df = log_transformation(df)
    X_train, y_train, features_list = _split(df)
    # display(X_train) WORKS
    X_train, y_train, x_mean, x_std, y_mean, y_std = standardize(X_train, y_train)
    # print(X_train.shape) WORKS
    X_train_seq = generate_input_sequences(X_train, X_train.columns.values.tolist())
    print(X_train_seq.shape)
    print('Inputs done')
    y_train_seq = generate_output_sequences(y_train, y_train.columns.values.tolist())
    print(y_train_seq.shape)
    print('Outputs done')
    y_train_baseline = generate_baseline_predictions(y_train, y_train.columns.values.tolist())
    print(y_train_baseline.shape)
    print('Baseline done')
    train_data = generate_tf_data(X_train_seq, y_train_seq, y_train_baseline, batch_size)
    return train_data, features_list, x_mean, x_std, y_mean, y_std

print('here')
df = pd.read_csv('/content/gdrive/MyDrive/playground/bnbusdt_data.csv', index_col=0)
batch_size = 128
lr = 1e-3
optimizer = 'Adam'
n_layers = 3
n_units = 32
dropout = 0.2
weight_decay = 1e-4
train_data, features_list, x_mean, x_std, y_mean, y_std = preprocess_df(df, batch_size)

if optimizer == 'Adam':
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
elif optimizer == 'SGD':
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=500, decay_rate=0.9,staircase=True)  
    opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

lstm_model = tf.keras.models.Sequential()

if n_layers == 1:
    lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
else:
    for n in range(n_layers):
        if n == n_layers - 1:
            lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
            #lstm_model.add(batchNormalization())
            lstm_model.add(tf.keras.layers.Dropout(dropout))
        else:
            lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=True))
            lstm_model.add(BatchNormalization())
            lstm_model.add(tf.keras.layers.Dropout(dropout))
            
lstm_model.add(tf.keras.layers.Dense(units=n_units, kernel_regularizer=regularizers.l2(l2=weight_decay)))

MAX_EPOCHS = 10
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, mode='min')
lstm_model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=opt, metrics=[tf.metrics.MeanAbsoluteError(), tf.keras.losses.MeanSquaredError()])
# history = lstm_model.fit(train_data, epochs=MAX_EPOCHS, validation_data=validation_data, callbacks=[early_stopping, WandbCallback()])
history = lstm_model.fit(train_data, epochs=MAX_EPOCHS, callbacks=[early_stopping])

here


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Only Selecting the last 2 years of data
Time preprocessing done
Target preprocessing done
Last preprocessing done
Deviations preprocessing done
Rolling preprocessing done
Logs preprocessing done
Splits done
Standardization done
(949960, 8, 102)
Inputs done
(949960, 32, 1)
Outputs done
(949960, 32, 1)
Baseline done
Epoch 1/10
7422/7422 [==============================] - 60s 7ms/step - loss: 3.8799e-04 - mean_absolute_error: 0.0051 - mean_squared_error: 2.1149e-04
Epoch 2/10
7422/7422 [==============================] - 56s 8ms/step - loss: 7.9611e-05 - mean_absolute_error: 0.0038 - mean_squared_error: 7.8992e-05
Epoch 3/10
7422/7422 [==============================] - 56s 8ms/step - loss: 6.9713e-05 - mean_absolute_error: 0.0035 - mean_squared_error: 6.8779e-05
Epoch 4/10
7422/7422 [==============================] - 57s 8ms/step - loss: 6.2776e-05 - mean_absolute_error: 0.0033 - mean_squared_error: 6.1579e-05
Epoch 5/10
7422/7422 [==============================] - 56s 8ms/step - loss: 6.0

In [26]:
# !rm -r /gdrive/MyDrive/playground/LSTM_BNBUSDT_12-12-2021/

In [27]:
import json

data = {}

data['validation_score'] = history1.history['val_loss'][-1]
data['unique_id'] = time.time()
data['training_time_stamp'] = time.time()
data['hyperparameters'] = {'batch_size': 128, 'lr': 1e-3, 'optimizer': 'Adam', 'n_layers': 3, 'n_units': 8, 'dropout': 0.2, 'weight_decay': 1e-4}
data['symbol'] = 'BNBUSDT'
data['x_mean'] = x_mean.values.tolist()
data['x_std'] = x_std.values.tolist()
data['y_mean'] = y_mean.values[0]
data['y_std'] = y_std.values[0]

with open('model_metrics_BNBUSDT.json', 'w') as outfile:
    json.dump(data, outfile)

In [28]:
!mkdir /content/gdrive/MyDrive/playground/LSTM_BNBUSDT_$(date +"%d-%m-%Y")

In [29]:
!cp /content/model_metrics_BNBUSDT.json /content/gdrive/MyDrive/playground/LSTM_BNBUSDT_13-12-2021/

In [30]:
lstm_model.save_weights('/content/gdrive/MyDrive/playground/LSTM_BNBUSDT_13-12-2021/lstm_BNBUSDT')

In [31]:
!gsutil -m cp -r /content/gdrive/MyDrive/playground/LSTM_BNBUSDT_13-12-2021/ gs://crypto-forecasting-bucket/BNBUSDT/Model/

Copying file:///content/gdrive/MyDrive/playground/LSTM_BNBUSDT_13-12-2021/model_metrics_BNBUSDT.json [Content-Type=application/json]...
Copying file:///content/gdrive/MyDrive/playground/LSTM_BNBUSDT_13-12-2021/lstm_BNBUSDT.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file:///content/gdrive/MyDrive/playground/LSTM_BNBUSDT_13-12-2021/lstm_BNBUSDT.index [Content-Type=application/octet-stream]...
Copying file:///content/gdrive/MyDrive/playground/LSTM_BNBUSDT_13-12-2021/checkpoint [Content-Type=application/octet-stream]...
/ [4/4 files][427.7 KiB/427.7 KiB] 100% Done                                    
Operation completed over 4 objects/427.7 KiB.                                    


# ETHBTC MODELING

In [32]:
import os

location_data = '/content/gdrive/MyDrive/playground/ethbtc_data.csv'

exclude_x = ['NA', 'Open Time', 'Close Time', 'y', 'Date']

val_split, test_split = None, None

log_transformation_feat = ['Open Price', 'High price', 'Low Price', 'Close Price', 'Volume Traded',
                           'Quote asset Volume', 'Number of Trades', 'Taker buy base asset volume',
                           'Taker buy quote asset volume', 'Mid Price', 'Open Price_prev', 'Close Price_prev',
                           'Mid Price_prev', 'high_low', 'high_close', 'high_open', 'open_low', 'spread',
                           'rolling_avg_high_low_1h', 'rolling_avg_high_low_1d', 'rolling_avg_high_low_1w',
                           'rolling_avg_high_low_1m', 'rolling_max_high_low_1h',
                           'rolling_max_high_low_1d', 'rolling_max_high_low_1w', 'rolling_max_high_low_1m',
                           'rolling_avg_high_close_1h', 'rolling_avg_high_close_1d', 'rolling_avg_high_close_1w',
                           'rolling_avg_high_close_1m', 'rolling_max_high_close_1h', 'rolling_max_high_close_1d',
                           'rolling_max_high_close_1w', 'rolling_max_high_close_1m', 'rolling_avg_high_open_1h',
                           'rolling_avg_high_open_1d', 'rolling_avg_high_open_1w', 'rolling_avg_high_open_1m',
                           'rolling_max_high_open_1h', 'rolling_max_high_open_1d',
                           'rolling_max_high_open_1w', 'rolling_max_high_open_1m', 'rolling_avg_open_low_1h',
                           'rolling_avg_open_low_1d',
                           'rolling_avg_open_low_1w', 'rolling_avg_open_low_1m', 'rolling_max_open_low_1h',
                           'rolling_max_open_low_1d',
                           'rolling_max_open_low_1w', 'rolling_max_open_low_1m', 'rolling_avg_spread_1h',
                           'rolling_avg_spread_1d',
                           'rolling_avg_spread_1w', 'rolling_avg_spread_1m', 'rolling_max_spread_1h',
                           'rolling_max_spread_1d', 'rolling_max_spread_1w',
                           'rolling_max_spread_1m', 'rolling_avg_Open Price_1h', 'rolling_avg_Open Price_1d',
                           'rolling_avg_Open Price_1w', 'rolling_avg_Open Price_1m',
                           'rolling_max_Open Price_1h', 'rolling_max_Open Price_1d', 'rolling_max_Open Price_1w',
                           'rolling_max_Open Price_1m',
                           'rolling_avg_High price_1h', 'rolling_avg_High price_1d', 'rolling_avg_High price_1w',
                           'rolling_avg_High price_1m',
                           'rolling_max_High price_1h', 'rolling_max_High price_1d', 'rolling_max_High price_1w',
                           'rolling_max_High price_1m',
                           'rolling_avg_Low Price_1h', 'rolling_avg_Low Price_1d', 'rolling_avg_Low Price_1w',
                           'rolling_avg_Low Price_1m',
                           'rolling_max_Low Price_1h', 'rolling_max_Low Price_1d', 'rolling_max_Low Price_1w',
                           'rolling_max_Low Price_1m',
                           'rolling_avg_Close Price_1h', 'rolling_avg_Close Price_1d', 'rolling_avg_Close Price_1w',
                           'rolling_avg_Close Price_1m',
                           'rolling_max_Close Price_1h', 'rolling_max_Close Price_1d', 'rolling_max_Close Price_1w',
                           'rolling_max_Close Price_1m']
# features where we apply log transformation


continuous_features = ['Open Price', 'High price', 'Low Price', 'Close Price', 'Volume Traded',
                       'Quote asset Volume', 'Number of Trades', 'Taker buy base asset volume',
                       'Taker buy quote asset volume', 'Mid Price', 'Open Price_prev', 'Close Price_prev',
                       'Mid Price_prev', 'high_low', 'high_close', 'high_open', 'open_low', 'spread',
                       'rolling_avg_high_low_1h', 'rolling_avg_high_low_1d', 'rolling_avg_high_low_1w',
                       'rolling_avg_high_low_1m', 'rolling_max_high_low_1h',
                       'rolling_max_high_low_1d', 'rolling_max_high_low_1w', 'rolling_max_high_low_1m',
                       'rolling_min_high_low_1h', 'rolling_min_high_low_1d',
                       'rolling_min_high_low_1w', 'rolling_min_high_low_1m', 'rolling_avg_high_close_1h',
                       'rolling_avg_high_close_1d', 'rolling_avg_high_close_1w', 'rolling_avg_high_close_1m',
                       'rolling_max_high_close_1h', 'rolling_max_high_close_1d', 'rolling_max_high_close_1w',
                       'rolling_max_high_close_1m', 'rolling_min_high_close_1h', 'rolling_min_high_close_1d',
                       'rolling_min_high_close_1w', 'rolling_min_high_close_1m', 'rolling_avg_high_open_1h',
                       'rolling_avg_high_open_1d', 'rolling_avg_high_open_1w', 'rolling_avg_high_open_1m',
                       'rolling_max_high_open_1h', 'rolling_max_high_open_1d',
                       'rolling_max_high_open_1w', 'rolling_max_high_open_1m', 'rolling_min_high_open_1h',
                       'rolling_min_high_open_1d',
                       'rolling_min_high_open_1w', 'rolling_min_high_open_1m', 'rolling_avg_open_low_1h',
                       'rolling_avg_open_low_1d',
                       'rolling_avg_open_low_1w', 'rolling_avg_open_low_1m', 'rolling_max_open_low_1h',
                       'rolling_max_open_low_1d',
                       'rolling_max_open_low_1w', 'rolling_max_open_low_1m', 'rolling_min_open_low_1h',
                       'rolling_min_open_low_1d',
                       'rolling_min_open_low_1w', 'rolling_min_open_low_1m', 'rolling_avg_spread_1h',
                       'rolling_avg_spread_1d',
                       'rolling_avg_spread_1w', 'rolling_avg_spread_1m', 'rolling_max_spread_1h',
                       'rolling_max_spread_1d', 'rolling_max_spread_1w',
                       'rolling_max_spread_1m', 'rolling_min_spread_1h', 'rolling_min_spread_1d',
                       'rolling_min_spread_1w', 'rolling_min_spread_1m',
                       'rolling_avg_Open Price_1h', 'rolling_avg_Open Price_1d', 'rolling_avg_Open Price_1w',
                       'rolling_avg_Open Price_1m',
                       'rolling_max_Open Price_1h', 'rolling_max_Open Price_1d', 'rolling_max_Open Price_1w',
                       'rolling_max_Open Price_1m',
                       'rolling_min_Open Price_1h', 'rolling_min_Open Price_1d', 'rolling_min_Open Price_1w',
                       'rolling_min_Open Price_1m',
                       'rolling_avg_High price_1h', 'rolling_avg_High price_1d', 'rolling_avg_High price_1w',
                       'rolling_avg_High price_1m',
                       'rolling_max_High price_1h', 'rolling_max_High price_1d', 'rolling_max_High price_1w',
                       'rolling_max_High price_1m',
                       'rolling_min_High price_1h', 'rolling_min_High price_1d', 'rolling_min_High price_1w',
                       'rolling_min_High price_1m',
                       'rolling_avg_Low Price_1h', 'rolling_avg_Low Price_1d', 'rolling_avg_Low Price_1w',
                       'rolling_avg_Low Price_1m',
                       'rolling_max_Low Price_1h', 'rolling_max_Low Price_1d', 'rolling_max_Low Price_1w',
                       'rolling_max_Low Price_1m',
                       'rolling_min_Low Price_1h', 'rolling_min_Low Price_1d', 'rolling_min_Low Price_1w',
                       'rolling_min_Low Price_1m',
                       'rolling_avg_Close Price_1h', 'rolling_avg_Close Price_1d', 'rolling_avg_Close Price_1w',
                       'rolling_avg_Close Price_1m',
                       'rolling_max_Close Price_1h', 'rolling_max_Close Price_1d', 'rolling_max_Close Price_1w',
                       'rolling_max_Close Price_1m',
                       'rolling_min_Close Price_1h', 'rolling_min_Close Price_1d', 'rolling_min_Close Price_1w',
                       'rolling_min_Close Price_1m']

In [33]:
import tensorflow as tf
#from preprocessing_pipeline import preprocess_df
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm
import matplotlib.pyplot as plt
#from parameters_modeling import exclude_x, log_transformation_feat, continuous_features, val_split, test_split, location_data
#import wandb
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
#from wandb.keras import WandbCallback
import tensorflow as tf


AUTOTUNE = tf.data.experimental.AUTOTUNE


def select_last_year(df):
    df = df.sort_values('Open Time')
    df = df.iloc[-1000000:, :]
    df = df.reset_index(drop=True)
    print('Only Selecting the last 2 years of data')
    return df 

def time_features_func(df):
    df['Date'] = pd.to_datetime(df['Open Time'], unit = 'ms')
    df['Year'] = pd.DatetimeIndex(df['Date']).year
    df['Month'] = pd.DatetimeIndex(df['Date']).month
    df['Day of Week'] = pd.DatetimeIndex(df['Date']).dayofweek
    df['Day of Month'] = pd.DatetimeIndex(df['Date']).day
    df['Day of Year'] = pd.DatetimeIndex(df['Date']).dayofyear
    df['Week of Month'] = pd.DatetimeIndex(df['Date']).day // 7
    df['Week of Year'] = df['Date'].dt.isocalendar().week.astype('int64')
    df['hour'] =df.Date.dt.hour
    df['minute'] =df.Date.dt.minute
    df['minute_of_day'] = df.hour*60 + df.minute
    print('Time preprocessing done')
    return df

def create_target_variables(df):
    # df = df.drop('Unnamed: 0', axis=1)
    df['Mid Price'] = (df['Open Price'] + df['Close Price'])/2
    df['y'] = (df['Open Price'].shift(-1) + df['Close Price'].shift(-1))/2  # the target is predicting the next close price
    df['benchmark'] = df['Mid Price']
    print('Target preprocessing done')
    return df


def create_last_reporting(df, columns=['Open Price', 'Close Price', 'Mid Price']):
    """Twice periods previously w.r.t the y variable"""
    for column in columns:
        df[column+'_prev'] = df[column].shift(1)
    df = df.dropna()
    print('Last preprocessing done')
    return df

def create_deviations(df):
    df['high_low'] = df['High price'] - df['Low Price']
    df['high_close'] = df['High price'] - df['Close Price']
    df['high_open'] = df['High price'] - df['Open Price']
    df['open_low'] = df['Close Price'] - df['Low Price']
    df['spread'] =df['Close Price'] - df['Open Price']
    df['spread_ind'] = 1*(df['spread'] < 0)
    df['spread'] =np.abs(df.spread)
    print('Deviations preprocessing done')
    return df

def create_stats(df, features = ['high_low', 'high_close', 'high_open', 'open_low', 'spread', 'Open Price', 'High price', 'Low Price', 'Close Price']):
    for feature in features:
        series_1h, series_1d, series_1w, series_1m =  df[feature].rolling(window = 60), df[feature].rolling(window = 1500), df[feature].rolling(window = 10000), df[feature].rolling(window = 50000)
        df['rolling_avg_{}_1h'.format(feature)] = series_1d.mean() # rolling avg over 1 hour
        df['rolling_avg_{}_1d'.format(feature)] = series_1d.mean() # rolling avg over 1 day
        df['rolling_avg_{}_1w'.format(feature)] = series_1w.mean() # rolling avg over 1 week
        df['rolling_avg_{}_1m'.format(feature)] = series_1m.mean() # rolling avg over 1 month
        df['rolling_max_{}_1h'.format(feature)] = series_1d.max() # rolling max over 1 hour
        df['rolling_max_{}_1d'.format(feature)] = series_1d.max() # rolling max over 1 day
        df['rolling_max_{}_1w'.format(feature)] = series_1w.max() # rolling max over 1 week
        df['rolling_max_{}_1m'.format(feature)] = series_1m.max()  # rolling max over 1 month
        #df['rolling_min_{}_1h'.format(feature)] = series_1d.min() # rolling min over 1 hour
        #df['rolling_min_{}_1d'.format(feature)] = series_1d.min() # rolling min over 1 day
        #df['rolling_min_{}_1w'.format(feature)] = series_1w.min() # rolling min over 1 week
        #df['rolling_min_{}_1m'.format(feature)] = series_1m.min() # rolling min over 1 month
    df = df.dropna()
    print('Rolling preprocessing done')
    return df

def log_transformation(df, log_transformation_features=log_transformation_feat):
    for feature in log_transformation_features:
        df['log_{}'.format(feature)] = np.log(1+df[feature].values)
        df = df.drop(feature, axis=1)
    print('Logs preprocessing done')
    return df

def _split(df, train_val_date=val_split, val_test_date=test_split, exclude_x=exclude_x):
    # Standardize the dataframe
    df = df.reset_index(drop=True)
    if train_val_date is None and val_test_date is None:
        index_val, index_test = int(len(df)*0.8), int(len(df)*0.9)
        print(index_val, index_test)
        print(df.head(), len(df.index))
        train_val_date, val_test_date = df.loc[index_val, 'Date'], df.loc[index_test, 'Date']
    print('We are going to build a model with train/val date {} and val/test date {}'.format(train_val_date, val_test_date))
    df_train, df_val, df_test = df[df.Date <= pd.to_datetime(train_val_date)], df[(df.Date > pd.to_datetime(train_val_date)) & (df.Date <= pd.to_datetime(val_test_date))], df[df.Date > pd.to_datetime(val_test_date)]
    features_list = [col for col in df_train.columns if col not in exclude_x]
    X_train, X_val, X_test = df_train[features_list], df_val[features_list],df_test[features_list]
    y_train, y_val, y_test = df_train[['y']], df_val[['y']], df_test[['y']]
    print('Splits done')
    return X_train, X_val, X_test, y_train, y_val, y_test, features_list

def standardize(X_train, X_val, X_test, y_train, y_val, y_test):
    x_mean = X_train.mean()
    x_std = X_train.std()
    y_mean = y_train.mean()
    y_std = y_train.std()
    X_train, X_val, X_test = (X_train - x_mean) / x_std, (X_val - x_mean) / x_std, (X_test - x_mean) / x_std
    y_train, y_val, y_test = (y_train - y_mean) / y_std, (y_val - y_mean) / y_std, (y_test - y_mean) / y_std
    print('Standardization done')
    return X_train, X_val, X_test, y_train, y_val, y_test, x_mean, x_std, y_mean, y_std

def generate_input_sequences(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, input_seq_len, len(feats)))
    dfvs = data[feats].values
    for i in (range(input_seq_len, len(data)-output_seq_len+1)):
        x = np.expand_dims(dfvs[i-input_seq_len:i, :], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_output_sequences(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, output_seq_len, 1))
    dfvs = data[feats].values
    for i in range(input_seq_len, len(data)-output_seq_len+1):
        x = np.expand_dims(dfvs[i:i+output_seq_len, :], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_baseline_predictions(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, output_seq_len, 1))
    dfvs = data[feats].values
    for i in range(input_seq_len, len(data)-output_seq_len+1):
        x = np.expand_dims(np.ones((output_seq_len, 1))*dfvs[i][0], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_tf_data(x_train, x_val, y_train, y_val, y_train_baseline, y_val_baseline, batch_size):
    x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
    x_val = tf.convert_to_tensor(x_val, dtype=tf.float32)
    y_train = tf.convert_to_tensor(y_train-y_train_baseline, dtype=tf.float32)
    y_val = tf.convert_to_tensor(y_val-y_val_baseline, dtype=tf.float32)

    train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_data = train_data.batch(batch_size)
    train_data = train_data.prefetch(AUTOTUNE)

    validation_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    validation_data = validation_data.batch(batch_size)
    validation_data = validation_data.prefetch(AUTOTUNE)
    return train_data, validation_data

def preprocess_df(df, batch_size, location_data=location_data):
    df = select_last_year(df)
    df = time_features_func(df)
    df = create_target_variables(df)
    df = create_last_reporting(df)
    df = create_deviations(df)
    df = create_stats(df)
    df = log_transformation(df)
    X_train, X_val, X_test, y_train, y_val, y_test, features_list = _split(df)
    X_train, X_val, X_test, y_train, y_val, y_test, x_mean, x_std, y_mean, y_std = standardize(X_train, X_val, X_test, y_train, y_val, y_test)
    X_train_seq = generate_input_sequences(X_train, X_train.columns.values.tolist())
    X_val_seq = generate_input_sequences(X_val, X_val.columns.values.tolist())
    X_test_seq = generate_input_sequences(X_test, X_test.columns.values.tolist())
    print('Inputs done')
    y_train_seq = generate_output_sequences(y_train, y_train.columns.values.tolist())
    y_val_seq = generate_output_sequences(y_val, y_val.columns.values.tolist())
    y_test_seq = generate_output_sequences(y_test, y_test.columns.values.tolist())
    print('Outputs done')
    y_train_baseline = generate_baseline_predictions(y_train, y_train.columns.values.tolist())
    y_val_baseline = generate_baseline_predictions(y_val, y_val.columns.values.tolist())
    y_test_baseline = generate_baseline_predictions(y_test, y_test.columns.values.tolist())
    print('Baseline done')
    train_data, validation_data = generate_tf_data(X_train_seq, X_val_seq, y_train_seq, y_val_seq, y_train_baseline, y_val_baseline, batch_size)
    return train_data, validation_data, features_list, x_mean, x_std, y_mean, y_std

print('here')
df = pd.read_csv('/content/gdrive/MyDrive/playground/ethbtc_data.csv', index_col=0)
batch_size = 128
lr = 1e-3
optimizer = 'Adam'
n_layers = 3
n_units = 32
dropout = 0.2
weight_decay = 1e-4
train_data, validation_data, features_list, x_mean, x_std, y_mean, y_std = preprocess_df(df, batch_size)

if optimizer == 'Adam':
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
elif optimizer == 'SGD':
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=500, decay_rate=0.9,staircase=True)  
    opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

lstm_model1 = tf.keras.models.Sequential()

if n_layers == 1:
    lstm_model1.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
else:
    for n in range(n_layers):
        if n == n_layers - 1:
            lstm_model1.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
            #lstm_model.add(batchNormalization())
            lstm_model1.add(tf.keras.layers.Dropout(dropout))
        else:
            lstm_model1.add(tf.keras.layers.LSTM(n_units, return_sequences=True))
            lstm_model1.add(BatchNormalization())
            lstm_model1.add(tf.keras.layers.Dropout(dropout))
            
lstm_model1.add(tf.keras.layers.Dense(units=n_units, kernel_regularizer=regularizers.l2(l2=weight_decay)))

MAX_EPOCHS = 10
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, mode='min')
lstm_model1.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=opt, metrics=[tf.metrics.MeanAbsoluteError(), tf.keras.losses.MeanSquaredError()])
# history = lstm_model.fit(train_data, epochs=MAX_EPOCHS, validation_data=validation_data, callbacks=[early_stopping, WandbCallback()])
history1 = lstm_model1.fit(train_data, epochs=MAX_EPOCHS, validation_data=validation_data, callbacks=[early_stopping])

here


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Only Selecting the last 2 years of data
Time preprocessing done
Target preprocessing done
Last preprocessing done
Deviations preprocessing done
Rolling preprocessing done
Logs preprocessing done
759999 854999
       Open Time  ...  log_rolling_max_Close Price_1m
0  1582264620000  ...                        0.028097
1  1582264680000  ...                        0.028097
2  1582264740000  ...                        0.028097
3  1582264800000  ...                        0.028097
4  1582264860000  ...                        0.028097

[5 rows x 107 columns] 949999
We are going to build a model with train/val date 2021-08-03 00:37:00 and val/test date 2021-10-08 06:27:00
Splits done
Standardization done
Inputs done
Outputs done
Baseline done
Epoch 1/10
5938/5938 [==============================] - 70s 11ms/step - loss: 5.2227e-04 - mean_absolute_error: 0.0066 - mean_squared_error: 2.9095e-04 - val_loss: 1.6098e-04 - val_mean_absolute_error: 0.0087 - val_mean_squared_error: 1.6081e-04
Epoch 2/10

In [34]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf


AUTOTUNE = tf.data.experimental.AUTOTUNE


def select_last_year(df):
    df = df.sort_values('Open Time')
    df = df.iloc[-1000000:, :]
    df = df.reset_index(drop=True)
    print('Only Selecting the last 2 years of data')
    return df 

def time_features_func(df):
    df['Date'] = pd.to_datetime(df['Open Time'], unit = 'ms')
    df['Year'] = pd.DatetimeIndex(df['Date']).year
    df['Month'] = pd.DatetimeIndex(df['Date']).month
    df['Day of Week'] = pd.DatetimeIndex(df['Date']).dayofweek
    df['Day of Month'] = pd.DatetimeIndex(df['Date']).day
    df['Day of Year'] = pd.DatetimeIndex(df['Date']).dayofyear
    df['Week of Month'] = pd.DatetimeIndex(df['Date']).day // 7
    df['Week of Year'] = df['Date'].dt.isocalendar().week.astype('int64')
    df['hour'] =df.Date.dt.hour
    df['minute'] =df.Date.dt.minute
    df['minute_of_day'] = df.hour*60 + df.minute
    print('Time preprocessing done')
    return df

def create_target_variables(df):
    # df = df.drop('Unnamed: 0', axis=1)
    df['Mid Price'] = (df['Open Price'] + df['Close Price'])/2
    df['y'] = (df['Open Price'].shift(-1) + df['Close Price'].shift(-1))/2  # the target is predicting the next close price
    df['benchmark'] = df['Mid Price']
    print('Target preprocessing done')
    return df


def create_last_reporting(df, columns=['Open Price', 'Close Price', 'Mid Price']):
    """Twice periods previously w.r.t the y variable"""
    for column in columns:
        df[column+'_prev'] = df[column].shift(1)
    df = df.dropna()
    print('Last preprocessing done')
    return df

def create_deviations(df):
    df['high_low'] = df['High price'] - df['Low Price']
    df['high_close'] = df['High price'] - df['Close Price']
    df['high_open'] = df['High price'] - df['Open Price']
    df['open_low'] = df['Close Price'] - df['Low Price']
    df['spread'] =df['Close Price'] - df['Open Price']
    df['spread_ind'] = 1*(df['spread'] < 0)
    df['spread'] =np.abs(df.spread)
    print('Deviations preprocessing done')
    return df

def create_stats(df, features = ['high_low', 'high_close', 'high_open', 'open_low', 'spread', 'Open Price', 'High price', 'Low Price', 'Close Price']):
    for feature in features:
        series_1h, series_1d, series_1w, series_1m =  df[feature].rolling(window = 60), df[feature].rolling(window = 1500), df[feature].rolling(window = 10000), df[feature].rolling(window = 50000)
        df['rolling_avg_{}_1h'.format(feature)] = series_1d.mean() # rolling avg over 1 hour
        df['rolling_avg_{}_1d'.format(feature)] = series_1d.mean() # rolling avg over 1 day
        df['rolling_avg_{}_1w'.format(feature)] = series_1w.mean() # rolling avg over 1 week
        df['rolling_avg_{}_1m'.format(feature)] = series_1m.mean() # rolling avg over 1 month
        df['rolling_max_{}_1h'.format(feature)] = series_1d.max() # rolling max over 1 hour
        df['rolling_max_{}_1d'.format(feature)] = series_1d.max() # rolling max over 1 day
        df['rolling_max_{}_1w'.format(feature)] = series_1w.max() # rolling max over 1 week
        df['rolling_max_{}_1m'.format(feature)] = series_1m.max()  # rolling max over 1 month
        #df['rolling_min_{}_1h'.format(feature)] = series_1d.min() # rolling min over 1 hour
        #df['rolling_min_{}_1d'.format(feature)] = series_1d.min() # rolling min over 1 day
        #df['rolling_min_{}_1w'.format(feature)] = series_1w.min() # rolling min over 1 week
        #df['rolling_min_{}_1m'.format(feature)] = series_1m.min() # rolling min over 1 month
    df = df.dropna()
    print('Rolling preprocessing done')
    return df

def log_transformation(df, log_transformation_features=log_transformation_feat):
    for feature in log_transformation_features:
        df['log_{}'.format(feature)] = np.log(1+df[feature].values)
        df = df.drop(feature, axis=1)
    print('Logs preprocessing done')
    return df

def _split(df, train_val_date=val_split, val_test_date=test_split, exclude_x=exclude_x):
    # Standardize the dataframe
    df = df.reset_index(drop=True)
    features_list = [col for col in df.columns if col not in exclude_x]
    X = df[features_list]
    y = df[['y']]
    print('Splits done')
    return X, y, features_list

def standardize(X, y):
    x_mean = X.mean()
    x_std = X.std()
    y_mean = y.mean()
    y_std = y.std()
    X = (X - x_mean) / x_std
    y = (y - y_mean) / y_std
    print('Standardization done')
    return X, y, x_mean, x_std, y_mean, y_std

def generate_input_sequences(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, input_seq_len, len(feats)))
    dfvs = data[feats].values
    for i in (range(input_seq_len, len(data)-output_seq_len+1)):
        x = np.expand_dims(dfvs[i-input_seq_len:i, :], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_output_sequences(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, output_seq_len, 1))
    dfvs = data[feats].values
    for i in range(input_seq_len, len(data)-output_seq_len+1):
        x = np.expand_dims(dfvs[i:i+output_seq_len, :], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_baseline_predictions(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, output_seq_len, 1))
    dfvs = data[feats].values
    for i in range(input_seq_len, len(data)-output_seq_len+1):
        x = np.expand_dims(np.ones((output_seq_len, 1))*dfvs[i][0], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_tf_data(x_train, y_train, y_train_baseline, batch_size):
    x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
    y_train = tf.convert_to_tensor(y_train-y_train_baseline, dtype=tf.float32)

    train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_data = train_data.batch(batch_size)
    train_data = train_data.prefetch(AUTOTUNE)
    return train_data

def preprocess_df(df, batch_size, location_data=location_data):
    df = select_last_year(df)
    df = time_features_func(df)
    df = create_target_variables(df)
    df = create_last_reporting(df)
    df = create_deviations(df)
    df = create_stats(df)
    df = log_transformation(df)
    X_train, y_train, features_list = _split(df)
    # display(X_train) WORKS
    X_train, y_train, x_mean, x_std, y_mean, y_std = standardize(X_train, y_train)
    # print(X_train.shape) WORKS
    X_train_seq = generate_input_sequences(X_train, X_train.columns.values.tolist())
    print(X_train_seq.shape)
    print('Inputs done')
    y_train_seq = generate_output_sequences(y_train, y_train.columns.values.tolist())
    print(y_train_seq.shape)
    print('Outputs done')
    y_train_baseline = generate_baseline_predictions(y_train, y_train.columns.values.tolist())
    print(y_train_baseline.shape)
    print('Baseline done')
    train_data = generate_tf_data(X_train_seq, y_train_seq, y_train_baseline, batch_size)
    return train_data, features_list, x_mean, x_std, y_mean, y_std

print('here')
df = pd.read_csv('/content/gdrive/MyDrive/playground/ethbtc_data.csv', index_col=0)
batch_size = 128
lr = 1e-3
optimizer = 'Adam'
n_layers = 3
n_units = 32
dropout = 0.2
weight_decay = 1e-4
train_data, features_list, x_mean, x_std, y_mean, y_std = preprocess_df(df, batch_size)

if optimizer == 'Adam':
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
elif optimizer == 'SGD':
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=500, decay_rate=0.9,staircase=True)  
    opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

lstm_model = tf.keras.models.Sequential()

if n_layers == 1:
    lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
else:
    for n in range(n_layers):
        if n == n_layers - 1:
            lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
            #lstm_model.add(batchNormalization())
            lstm_model.add(tf.keras.layers.Dropout(dropout))
        else:
            lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=True))
            lstm_model.add(BatchNormalization())
            lstm_model.add(tf.keras.layers.Dropout(dropout))
            
lstm_model.add(tf.keras.layers.Dense(units=n_units, kernel_regularizer=regularizers.l2(l2=weight_decay)))

MAX_EPOCHS = 10
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, mode='min')
lstm_model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=opt, metrics=[tf.metrics.MeanAbsoluteError(), tf.keras.losses.MeanSquaredError()])
# history = lstm_model.fit(train_data, epochs=MAX_EPOCHS, validation_data=validation_data, callbacks=[early_stopping, WandbCallback()])
history = lstm_model.fit(train_data, epochs=MAX_EPOCHS, callbacks=[early_stopping])

here


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Only Selecting the last 2 years of data
Time preprocessing done
Target preprocessing done
Last preprocessing done
Deviations preprocessing done
Rolling preprocessing done
Logs preprocessing done
Splits done
Standardization done
(949960, 8, 102)
Inputs done
(949960, 32, 1)
Outputs done
(949960, 32, 1)
Baseline done
Epoch 1/10
7422/7422 [==============================] - 77s 9ms/step - loss: 4.1451e-04 - mean_absolute_error: 0.0054 - mean_squared_error: 2.1314e-04
Epoch 2/10
7422/7422 [==============================] - 70s 9ms/step - loss: 5.8224e-05 - mean_absolute_error: 0.0042 - mean_squared_error: 5.8020e-05
Epoch 3/10
7422/7422 [==============================] - 71s 10ms/step - loss: 5.8319e-05 - mean_absolute_error: 0.0042 - mean_squared_error: 5.8027e-05
Epoch 4/10
7422/7422 [==============================] - 71s 10ms/step - loss: 5.8536e-05 - mean_absolute_error: 0.0042 - mean_squared_error: 5.8249e-05
Epoch 5/10
7422/7422 [==============================] - 70s 9ms/step - loss: 5

In [35]:
# !rm -r /gdrive/MyDrive/playground/LSTM_ETHBTC_12-12-2021/

In [36]:
import json

data = {}

data['validation_score'] = history1.history['val_loss'][-1]
data['unique_id'] = time.time()
data['training_time_stamp'] = time.time()
data['hyperparameters'] = {'batch_size': 128, 'lr': 1e-3, 'optimizer': 'Adam', 'n_layers': 3, 'n_units': 8, 'dropout': 0.2, 'weight_decay': 1e-4}
data['symbol'] = 'ETHBTC'
data['x_mean'] = x_mean.values.tolist()
data['x_std'] = x_std.values.tolist()
data['y_mean'] = y_mean.values[0]
data['y_std'] = y_std.values[0]

with open('model_metrics_ETHBTC.json', 'w') as outfile:
    json.dump(data, outfile)

In [37]:
!mkdir /content/gdrive/MyDrive/playground/LSTM_ETHBTC_$(date +"%d-%m-%Y")

In [38]:
!cp /content/model_metrics_ETHBTC.json /content/gdrive/MyDrive/playground/LSTM_ETHBTC_13-12-2021/

In [39]:
lstm_model.save_weights('/content/gdrive/MyDrive/playground/LSTM_ETHBTC_13-12-2021/lstm_ETHBTC')

In [40]:
!gsutil -m cp -r /content/gdrive/MyDrive/playground/LSTM_ETHBTC_13-12-2021/ gs://crypto-forecasting-bucket/ETHBTC/Model/

Copying file:///content/gdrive/MyDrive/playground/LSTM_ETHBTC_13-12-2021/lstm_ETHBTC.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file:///content/gdrive/MyDrive/playground/LSTM_ETHBTC_13-12-2021/model_metrics_ETHBTC.json [Content-Type=application/json]...
Copying file:///content/gdrive/MyDrive/playground/LSTM_ETHBTC_13-12-2021/lstm_ETHBTC.index [Content-Type=application/octet-stream]...
Copying file:///content/gdrive/MyDrive/playground/LSTM_ETHBTC_13-12-2021/checkpoint [Content-Type=application/octet-stream]...
/ [4/4 files][428.1 KiB/428.1 KiB] 100% Done                                    
Operation completed over 4 objects/428.1 KiB.                                    


# ETHUSDT MODELING

In [41]:
import os

location_data = '/content/gdrive/MyDrive/playground/ethusdt_data.csv'

exclude_x = ['NA', 'Open Time', 'Close Time', 'y', 'Date']

val_split, test_split = None, None

log_transformation_feat = ['Open Price', 'High price', 'Low Price', 'Close Price', 'Volume Traded',
                           'Quote asset Volume', 'Number of Trades', 'Taker buy base asset volume',
                           'Taker buy quote asset volume', 'Mid Price', 'Open Price_prev', 'Close Price_prev',
                           'Mid Price_prev', 'high_low', 'high_close', 'high_open', 'open_low', 'spread',
                           'rolling_avg_high_low_1h', 'rolling_avg_high_low_1d', 'rolling_avg_high_low_1w',
                           'rolling_avg_high_low_1m', 'rolling_max_high_low_1h',
                           'rolling_max_high_low_1d', 'rolling_max_high_low_1w', 'rolling_max_high_low_1m',
                           'rolling_avg_high_close_1h', 'rolling_avg_high_close_1d', 'rolling_avg_high_close_1w',
                           'rolling_avg_high_close_1m', 'rolling_max_high_close_1h', 'rolling_max_high_close_1d',
                           'rolling_max_high_close_1w', 'rolling_max_high_close_1m', 'rolling_avg_high_open_1h',
                           'rolling_avg_high_open_1d', 'rolling_avg_high_open_1w', 'rolling_avg_high_open_1m',
                           'rolling_max_high_open_1h', 'rolling_max_high_open_1d',
                           'rolling_max_high_open_1w', 'rolling_max_high_open_1m', 'rolling_avg_open_low_1h',
                           'rolling_avg_open_low_1d',
                           'rolling_avg_open_low_1w', 'rolling_avg_open_low_1m', 'rolling_max_open_low_1h',
                           'rolling_max_open_low_1d',
                           'rolling_max_open_low_1w', 'rolling_max_open_low_1m', 'rolling_avg_spread_1h',
                           'rolling_avg_spread_1d',
                           'rolling_avg_spread_1w', 'rolling_avg_spread_1m', 'rolling_max_spread_1h',
                           'rolling_max_spread_1d', 'rolling_max_spread_1w',
                           'rolling_max_spread_1m', 'rolling_avg_Open Price_1h', 'rolling_avg_Open Price_1d',
                           'rolling_avg_Open Price_1w', 'rolling_avg_Open Price_1m',
                           'rolling_max_Open Price_1h', 'rolling_max_Open Price_1d', 'rolling_max_Open Price_1w',
                           'rolling_max_Open Price_1m',
                           'rolling_avg_High price_1h', 'rolling_avg_High price_1d', 'rolling_avg_High price_1w',
                           'rolling_avg_High price_1m',
                           'rolling_max_High price_1h', 'rolling_max_High price_1d', 'rolling_max_High price_1w',
                           'rolling_max_High price_1m',
                           'rolling_avg_Low Price_1h', 'rolling_avg_Low Price_1d', 'rolling_avg_Low Price_1w',
                           'rolling_avg_Low Price_1m',
                           'rolling_max_Low Price_1h', 'rolling_max_Low Price_1d', 'rolling_max_Low Price_1w',
                           'rolling_max_Low Price_1m',
                           'rolling_avg_Close Price_1h', 'rolling_avg_Close Price_1d', 'rolling_avg_Close Price_1w',
                           'rolling_avg_Close Price_1m',
                           'rolling_max_Close Price_1h', 'rolling_max_Close Price_1d', 'rolling_max_Close Price_1w',
                           'rolling_max_Close Price_1m']
# features where we apply log transformation


continuous_features = ['Open Price', 'High price', 'Low Price', 'Close Price', 'Volume Traded',
                       'Quote asset Volume', 'Number of Trades', 'Taker buy base asset volume',
                       'Taker buy quote asset volume', 'Mid Price', 'Open Price_prev', 'Close Price_prev',
                       'Mid Price_prev', 'high_low', 'high_close', 'high_open', 'open_low', 'spread',
                       'rolling_avg_high_low_1h', 'rolling_avg_high_low_1d', 'rolling_avg_high_low_1w',
                       'rolling_avg_high_low_1m', 'rolling_max_high_low_1h',
                       'rolling_max_high_low_1d', 'rolling_max_high_low_1w', 'rolling_max_high_low_1m',
                       'rolling_min_high_low_1h', 'rolling_min_high_low_1d',
                       'rolling_min_high_low_1w', 'rolling_min_high_low_1m', 'rolling_avg_high_close_1h',
                       'rolling_avg_high_close_1d', 'rolling_avg_high_close_1w', 'rolling_avg_high_close_1m',
                       'rolling_max_high_close_1h', 'rolling_max_high_close_1d', 'rolling_max_high_close_1w',
                       'rolling_max_high_close_1m', 'rolling_min_high_close_1h', 'rolling_min_high_close_1d',
                       'rolling_min_high_close_1w', 'rolling_min_high_close_1m', 'rolling_avg_high_open_1h',
                       'rolling_avg_high_open_1d', 'rolling_avg_high_open_1w', 'rolling_avg_high_open_1m',
                       'rolling_max_high_open_1h', 'rolling_max_high_open_1d',
                       'rolling_max_high_open_1w', 'rolling_max_high_open_1m', 'rolling_min_high_open_1h',
                       'rolling_min_high_open_1d',
                       'rolling_min_high_open_1w', 'rolling_min_high_open_1m', 'rolling_avg_open_low_1h',
                       'rolling_avg_open_low_1d',
                       'rolling_avg_open_low_1w', 'rolling_avg_open_low_1m', 'rolling_max_open_low_1h',
                       'rolling_max_open_low_1d',
                       'rolling_max_open_low_1w', 'rolling_max_open_low_1m', 'rolling_min_open_low_1h',
                       'rolling_min_open_low_1d',
                       'rolling_min_open_low_1w', 'rolling_min_open_low_1m', 'rolling_avg_spread_1h',
                       'rolling_avg_spread_1d',
                       'rolling_avg_spread_1w', 'rolling_avg_spread_1m', 'rolling_max_spread_1h',
                       'rolling_max_spread_1d', 'rolling_max_spread_1w',
                       'rolling_max_spread_1m', 'rolling_min_spread_1h', 'rolling_min_spread_1d',
                       'rolling_min_spread_1w', 'rolling_min_spread_1m',
                       'rolling_avg_Open Price_1h', 'rolling_avg_Open Price_1d', 'rolling_avg_Open Price_1w',
                       'rolling_avg_Open Price_1m',
                       'rolling_max_Open Price_1h', 'rolling_max_Open Price_1d', 'rolling_max_Open Price_1w',
                       'rolling_max_Open Price_1m',
                       'rolling_min_Open Price_1h', 'rolling_min_Open Price_1d', 'rolling_min_Open Price_1w',
                       'rolling_min_Open Price_1m',
                       'rolling_avg_High price_1h', 'rolling_avg_High price_1d', 'rolling_avg_High price_1w',
                       'rolling_avg_High price_1m',
                       'rolling_max_High price_1h', 'rolling_max_High price_1d', 'rolling_max_High price_1w',
                       'rolling_max_High price_1m',
                       'rolling_min_High price_1h', 'rolling_min_High price_1d', 'rolling_min_High price_1w',
                       'rolling_min_High price_1m',
                       'rolling_avg_Low Price_1h', 'rolling_avg_Low Price_1d', 'rolling_avg_Low Price_1w',
                       'rolling_avg_Low Price_1m',
                       'rolling_max_Low Price_1h', 'rolling_max_Low Price_1d', 'rolling_max_Low Price_1w',
                       'rolling_max_Low Price_1m',
                       'rolling_min_Low Price_1h', 'rolling_min_Low Price_1d', 'rolling_min_Low Price_1w',
                       'rolling_min_Low Price_1m',
                       'rolling_avg_Close Price_1h', 'rolling_avg_Close Price_1d', 'rolling_avg_Close Price_1w',
                       'rolling_avg_Close Price_1m',
                       'rolling_max_Close Price_1h', 'rolling_max_Close Price_1d', 'rolling_max_Close Price_1w',
                       'rolling_max_Close Price_1m',
                       'rolling_min_Close Price_1h', 'rolling_min_Close Price_1d', 'rolling_min_Close Price_1w',
                       'rolling_min_Close Price_1m']

In [42]:
import tensorflow as tf
#from preprocessing_pipeline import preprocess_df
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm
import matplotlib.pyplot as plt
#from parameters_modeling import exclude_x, log_transformation_feat, continuous_features, val_split, test_split, location_data
#import wandb
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
#from wandb.keras import WandbCallback
import tensorflow as tf


AUTOTUNE = tf.data.experimental.AUTOTUNE


def select_last_year(df):
    df = df.sort_values('Open Time')
    df = df.iloc[-1000000:, :]
    df = df.reset_index(drop=True)
    print('Only Selecting the last 2 years of data')
    return df 

def time_features_func(df):
    df['Date'] = pd.to_datetime(df['Open Time'], unit = 'ms')
    df['Year'] = pd.DatetimeIndex(df['Date']).year
    df['Month'] = pd.DatetimeIndex(df['Date']).month
    df['Day of Week'] = pd.DatetimeIndex(df['Date']).dayofweek
    df['Day of Month'] = pd.DatetimeIndex(df['Date']).day
    df['Day of Year'] = pd.DatetimeIndex(df['Date']).dayofyear
    df['Week of Month'] = pd.DatetimeIndex(df['Date']).day // 7
    df['Week of Year'] = df['Date'].dt.isocalendar().week.astype('int64')
    df['hour'] =df.Date.dt.hour
    df['minute'] =df.Date.dt.minute
    df['minute_of_day'] = df.hour*60 + df.minute
    print('Time preprocessing done')
    return df

def create_target_variables(df):
    # df = df.drop('Unnamed: 0', axis=1)
    df['Mid Price'] = (df['Open Price'] + df['Close Price'])/2
    df['y'] = (df['Open Price'].shift(-1) + df['Close Price'].shift(-1))/2  # the target is predicting the next close price
    df['benchmark'] = df['Mid Price']
    print('Target preprocessing done')
    return df


def create_last_reporting(df, columns=['Open Price', 'Close Price', 'Mid Price']):
    """Twice periods previously w.r.t the y variable"""
    for column in columns:
        df[column+'_prev'] = df[column].shift(1)
    df = df.dropna()
    print('Last preprocessing done')
    return df

def create_deviations(df):
    df['high_low'] = df['High price'] - df['Low Price']
    df['high_close'] = df['High price'] - df['Close Price']
    df['high_open'] = df['High price'] - df['Open Price']
    df['open_low'] = df['Close Price'] - df['Low Price']
    df['spread'] =df['Close Price'] - df['Open Price']
    df['spread_ind'] = 1*(df['spread'] < 0)
    df['spread'] =np.abs(df.spread)
    print('Deviations preprocessing done')
    return df

def create_stats(df, features = ['high_low', 'high_close', 'high_open', 'open_low', 'spread', 'Open Price', 'High price', 'Low Price', 'Close Price']):
    for feature in features:
        series_1h, series_1d, series_1w, series_1m =  df[feature].rolling(window = 60), df[feature].rolling(window = 1500), df[feature].rolling(window = 10000), df[feature].rolling(window = 50000)
        df['rolling_avg_{}_1h'.format(feature)] = series_1d.mean() # rolling avg over 1 hour
        df['rolling_avg_{}_1d'.format(feature)] = series_1d.mean() # rolling avg over 1 day
        df['rolling_avg_{}_1w'.format(feature)] = series_1w.mean() # rolling avg over 1 week
        df['rolling_avg_{}_1m'.format(feature)] = series_1m.mean() # rolling avg over 1 month
        df['rolling_max_{}_1h'.format(feature)] = series_1d.max() # rolling max over 1 hour
        df['rolling_max_{}_1d'.format(feature)] = series_1d.max() # rolling max over 1 day
        df['rolling_max_{}_1w'.format(feature)] = series_1w.max() # rolling max over 1 week
        df['rolling_max_{}_1m'.format(feature)] = series_1m.max()  # rolling max over 1 month
        #df['rolling_min_{}_1h'.format(feature)] = series_1d.min() # rolling min over 1 hour
        #df['rolling_min_{}_1d'.format(feature)] = series_1d.min() # rolling min over 1 day
        #df['rolling_min_{}_1w'.format(feature)] = series_1w.min() # rolling min over 1 week
        #df['rolling_min_{}_1m'.format(feature)] = series_1m.min() # rolling min over 1 month
    df = df.dropna()
    print('Rolling preprocessing done')
    return df

def log_transformation(df, log_transformation_features=log_transformation_feat):
    for feature in log_transformation_features:
        df['log_{}'.format(feature)] = np.log(1+df[feature].values)
        df = df.drop(feature, axis=1)
    print('Logs preprocessing done')
    return df

def _split(df, train_val_date=val_split, val_test_date=test_split, exclude_x=exclude_x):
    # Standardize the dataframe
    df = df.reset_index(drop=True)
    if train_val_date is None and val_test_date is None:
        index_val, index_test = int(len(df)*0.8), int(len(df)*0.9)
        print(index_val, index_test)
        print(df.head(), len(df.index))
        train_val_date, val_test_date = df.loc[index_val, 'Date'], df.loc[index_test, 'Date']
    print('We are going to build a model with train/val date {} and val/test date {}'.format(train_val_date, val_test_date))
    df_train, df_val, df_test = df[df.Date <= pd.to_datetime(train_val_date)], df[(df.Date > pd.to_datetime(train_val_date)) & (df.Date <= pd.to_datetime(val_test_date))], df[df.Date > pd.to_datetime(val_test_date)]
    features_list = [col for col in df_train.columns if col not in exclude_x]
    X_train, X_val, X_test = df_train[features_list], df_val[features_list],df_test[features_list]
    y_train, y_val, y_test = df_train[['y']], df_val[['y']], df_test[['y']]
    print('Splits done')
    return X_train, X_val, X_test, y_train, y_val, y_test, features_list

def standardize(X_train, X_val, X_test, y_train, y_val, y_test):
    x_mean = X_train.mean()
    x_std = X_train.std()
    y_mean = y_train.mean()
    y_std = y_train.std()
    X_train, X_val, X_test = (X_train - x_mean) / x_std, (X_val - x_mean) / x_std, (X_test - x_mean) / x_std
    y_train, y_val, y_test = (y_train - y_mean) / y_std, (y_val - y_mean) / y_std, (y_test - y_mean) / y_std
    print('Standardization done')
    return X_train, X_val, X_test, y_train, y_val, y_test, x_mean, x_std, y_mean, y_std

def generate_input_sequences(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, input_seq_len, len(feats)))
    dfvs = data[feats].values
    for i in (range(input_seq_len, len(data)-output_seq_len+1)):
        x = np.expand_dims(dfvs[i-input_seq_len:i, :], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_output_sequences(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, output_seq_len, 1))
    dfvs = data[feats].values
    for i in range(input_seq_len, len(data)-output_seq_len+1):
        x = np.expand_dims(dfvs[i:i+output_seq_len, :], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_baseline_predictions(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, output_seq_len, 1))
    dfvs = data[feats].values
    for i in range(input_seq_len, len(data)-output_seq_len+1):
        x = np.expand_dims(np.ones((output_seq_len, 1))*dfvs[i][0], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_tf_data(x_train, x_val, y_train, y_val, y_train_baseline, y_val_baseline, batch_size):
    x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
    x_val = tf.convert_to_tensor(x_val, dtype=tf.float32)
    y_train = tf.convert_to_tensor(y_train-y_train_baseline, dtype=tf.float32)
    y_val = tf.convert_to_tensor(y_val-y_val_baseline, dtype=tf.float32)

    train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_data = train_data.batch(batch_size)
    train_data = train_data.prefetch(AUTOTUNE)

    validation_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    validation_data = validation_data.batch(batch_size)
    validation_data = validation_data.prefetch(AUTOTUNE)
    return train_data, validation_data

def preprocess_df(df, batch_size, location_data=location_data):
    df = select_last_year(df)
    df = time_features_func(df)
    df = create_target_variables(df)
    df = create_last_reporting(df)
    df = create_deviations(df)
    df = create_stats(df)
    df = log_transformation(df)
    X_train, X_val, X_test, y_train, y_val, y_test, features_list = _split(df)
    X_train, X_val, X_test, y_train, y_val, y_test, x_mean, x_std, y_mean, y_std = standardize(X_train, X_val, X_test, y_train, y_val, y_test)
    X_train_seq = generate_input_sequences(X_train, X_train.columns.values.tolist())
    X_val_seq = generate_input_sequences(X_val, X_val.columns.values.tolist())
    X_test_seq = generate_input_sequences(X_test, X_test.columns.values.tolist())
    print('Inputs done')
    y_train_seq = generate_output_sequences(y_train, y_train.columns.values.tolist())
    y_val_seq = generate_output_sequences(y_val, y_val.columns.values.tolist())
    y_test_seq = generate_output_sequences(y_test, y_test.columns.values.tolist())
    print('Outputs done')
    y_train_baseline = generate_baseline_predictions(y_train, y_train.columns.values.tolist())
    y_val_baseline = generate_baseline_predictions(y_val, y_val.columns.values.tolist())
    y_test_baseline = generate_baseline_predictions(y_test, y_test.columns.values.tolist())
    print('Baseline done')
    train_data, validation_data = generate_tf_data(X_train_seq, X_val_seq, y_train_seq, y_val_seq, y_train_baseline, y_val_baseline, batch_size)
    return train_data, validation_data, features_list, x_mean, x_std, y_mean, y_std

print('here')
df = pd.read_csv('/content/gdrive/MyDrive/playground/ethusdt_data.csv', index_col=0)
batch_size = 128
lr = 1e-3
optimizer = 'Adam'
n_layers = 3
n_units = 32
dropout = 0.2
weight_decay = 1e-4
train_data, validation_data, features_list, x_mean, x_std, y_mean, y_std = preprocess_df(df, batch_size)

if optimizer == 'Adam':
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
elif optimizer == 'SGD':
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=500, decay_rate=0.9,staircase=True)  
    opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

lstm_model1 = tf.keras.models.Sequential()

if n_layers == 1:
    lstm_model1.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
else:
    for n in range(n_layers):
        if n == n_layers - 1:
            lstm_model1.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
            #lstm_model.add(batchNormalization())
            lstm_model1.add(tf.keras.layers.Dropout(dropout))
        else:
            lstm_model1.add(tf.keras.layers.LSTM(n_units, return_sequences=True))
            lstm_model1.add(BatchNormalization())
            lstm_model1.add(tf.keras.layers.Dropout(dropout))
            
lstm_model1.add(tf.keras.layers.Dense(units=n_units, kernel_regularizer=regularizers.l2(l2=weight_decay)))

MAX_EPOCHS = 10
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, mode='min')
lstm_model1.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=opt, metrics=[tf.metrics.MeanAbsoluteError(), tf.keras.losses.MeanSquaredError()])
# history = lstm_model.fit(train_data, epochs=MAX_EPOCHS, validation_data=validation_data, callbacks=[early_stopping, WandbCallback()])
history1 = lstm_model1.fit(train_data, epochs=MAX_EPOCHS, validation_data=validation_data, callbacks=[early_stopping])

here


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Only Selecting the last 2 years of data
Time preprocessing done
Target preprocessing done
Last preprocessing done
Deviations preprocessing done
Rolling preprocessing done
Logs preprocessing done
759999 854999
       Open Time  ...  log_rolling_max_Close Price_1m
0  1582298460000  ...                        5.666946
1  1582298520000  ...                        5.666946
2  1582298580000  ...                        5.666946
3  1582298640000  ...                        5.666946
4  1582298700000  ...                        5.666946

[5 rows x 107 columns] 949999
We are going to build a model with train/val date 2021-08-03 10:02:00 and val/test date 2021-10-08 15:52:00
Splits done
Standardization done
Inputs done
Outputs done
Baseline done
Epoch 1/10
5938/5938 [==============================] - 71s 11ms/step - loss: 4.9728e-04 - mean_absolute_error: 0.0059 - mean_squared_error: 2.7411e-04 - val_loss: 2.6904e-04 - val_mean_absolute_error: 0.0105 - val_mean_squared_error: 2.6888e-04
Epoch 2/10

In [43]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf


AUTOTUNE = tf.data.experimental.AUTOTUNE


def select_last_year(df):
    df = df.sort_values('Open Time')
    df = df.iloc[-1000000:, :]
    df = df.reset_index(drop=True)
    print('Only Selecting the last 2 years of data')
    return df 

def time_features_func(df):
    df['Date'] = pd.to_datetime(df['Open Time'], unit = 'ms')
    df['Year'] = pd.DatetimeIndex(df['Date']).year
    df['Month'] = pd.DatetimeIndex(df['Date']).month
    df['Day of Week'] = pd.DatetimeIndex(df['Date']).dayofweek
    df['Day of Month'] = pd.DatetimeIndex(df['Date']).day
    df['Day of Year'] = pd.DatetimeIndex(df['Date']).dayofyear
    df['Week of Month'] = pd.DatetimeIndex(df['Date']).day // 7
    df['Week of Year'] = df['Date'].dt.isocalendar().week.astype('int64')
    df['hour'] =df.Date.dt.hour
    df['minute'] =df.Date.dt.minute
    df['minute_of_day'] = df.hour*60 + df.minute
    print('Time preprocessing done')
    return df

def create_target_variables(df):
    # df = df.drop('Unnamed: 0', axis=1)
    df['Mid Price'] = (df['Open Price'] + df['Close Price'])/2
    df['y'] = (df['Open Price'].shift(-1) + df['Close Price'].shift(-1))/2  # the target is predicting the next close price
    df['benchmark'] = df['Mid Price']
    print('Target preprocessing done')
    return df


def create_last_reporting(df, columns=['Open Price', 'Close Price', 'Mid Price']):
    """Twice periods previously w.r.t the y variable"""
    for column in columns:
        df[column+'_prev'] = df[column].shift(1)
    df = df.dropna()
    print('Last preprocessing done')
    return df

def create_deviations(df):
    df['high_low'] = df['High price'] - df['Low Price']
    df['high_close'] = df['High price'] - df['Close Price']
    df['high_open'] = df['High price'] - df['Open Price']
    df['open_low'] = df['Close Price'] - df['Low Price']
    df['spread'] =df['Close Price'] - df['Open Price']
    df['spread_ind'] = 1*(df['spread'] < 0)
    df['spread'] =np.abs(df.spread)
    print('Deviations preprocessing done')
    return df

def create_stats(df, features = ['high_low', 'high_close', 'high_open', 'open_low', 'spread', 'Open Price', 'High price', 'Low Price', 'Close Price']):
    for feature in features:
        series_1h, series_1d, series_1w, series_1m =  df[feature].rolling(window = 60), df[feature].rolling(window = 1500), df[feature].rolling(window = 10000), df[feature].rolling(window = 50000)
        df['rolling_avg_{}_1h'.format(feature)] = series_1d.mean() # rolling avg over 1 hour
        df['rolling_avg_{}_1d'.format(feature)] = series_1d.mean() # rolling avg over 1 day
        df['rolling_avg_{}_1w'.format(feature)] = series_1w.mean() # rolling avg over 1 week
        df['rolling_avg_{}_1m'.format(feature)] = series_1m.mean() # rolling avg over 1 month
        df['rolling_max_{}_1h'.format(feature)] = series_1d.max() # rolling max over 1 hour
        df['rolling_max_{}_1d'.format(feature)] = series_1d.max() # rolling max over 1 day
        df['rolling_max_{}_1w'.format(feature)] = series_1w.max() # rolling max over 1 week
        df['rolling_max_{}_1m'.format(feature)] = series_1m.max()  # rolling max over 1 month
        #df['rolling_min_{}_1h'.format(feature)] = series_1d.min() # rolling min over 1 hour
        #df['rolling_min_{}_1d'.format(feature)] = series_1d.min() # rolling min over 1 day
        #df['rolling_min_{}_1w'.format(feature)] = series_1w.min() # rolling min over 1 week
        #df['rolling_min_{}_1m'.format(feature)] = series_1m.min() # rolling min over 1 month
    df = df.dropna()
    print('Rolling preprocessing done')
    return df

def log_transformation(df, log_transformation_features=log_transformation_feat):
    for feature in log_transformation_features:
        df['log_{}'.format(feature)] = np.log(1+df[feature].values)
        df = df.drop(feature, axis=1)
    print('Logs preprocessing done')
    return df

def _split(df, train_val_date=val_split, val_test_date=test_split, exclude_x=exclude_x):
    # Standardize the dataframe
    df = df.reset_index(drop=True)
    features_list = [col for col in df.columns if col not in exclude_x]
    X = df[features_list]
    y = df[['y']]
    print('Splits done')
    return X, y, features_list

def standardize(X, y):
    x_mean = X.mean()
    x_std = X.std()
    y_mean = y.mean()
    y_std = y.std()
    X = (X - x_mean) / x_std
    y = (y - y_mean) / y_std
    print('Standardization done')
    return X, y, x_mean, x_std, y_mean, y_std

def generate_input_sequences(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, input_seq_len, len(feats)))
    dfvs = data[feats].values
    for i in (range(input_seq_len, len(data)-output_seq_len+1)):
        x = np.expand_dims(dfvs[i-input_seq_len:i, :], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_output_sequences(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, output_seq_len, 1))
    dfvs = data[feats].values
    for i in range(input_seq_len, len(data)-output_seq_len+1):
        x = np.expand_dims(dfvs[i:i+output_seq_len, :], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_baseline_predictions(data, feats, input_seq_len = 8, output_seq_len = 32):
    empty_np_array = np.zeros((len(data)-input_seq_len-output_seq_len+1, output_seq_len, 1))
    dfvs = data[feats].values
    for i in range(input_seq_len, len(data)-output_seq_len+1):
        x = np.expand_dims(np.ones((output_seq_len, 1))*dfvs[i][0], axis = 0)
        empty_np_array[i-input_seq_len] = x
    return empty_np_array

def generate_tf_data(x_train, y_train, y_train_baseline, batch_size):
    x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
    y_train = tf.convert_to_tensor(y_train-y_train_baseline, dtype=tf.float32)

    train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_data = train_data.batch(batch_size)
    train_data = train_data.prefetch(AUTOTUNE)
    return train_data

def preprocess_df(df, batch_size, location_data=location_data):
    df = select_last_year(df)
    df = time_features_func(df)
    df = create_target_variables(df)
    df = create_last_reporting(df)
    df = create_deviations(df)
    df = create_stats(df)
    df = log_transformation(df)
    X_train, y_train, features_list = _split(df)
    # display(X_train) WORKS
    X_train, y_train, x_mean, x_std, y_mean, y_std = standardize(X_train, y_train)
    # print(X_train.shape) WORKS
    X_train_seq = generate_input_sequences(X_train, X_train.columns.values.tolist())
    print(X_train_seq.shape)
    print('Inputs done')
    y_train_seq = generate_output_sequences(y_train, y_train.columns.values.tolist())
    print(y_train_seq.shape)
    print('Outputs done')
    y_train_baseline = generate_baseline_predictions(y_train, y_train.columns.values.tolist())
    print(y_train_baseline.shape)
    print('Baseline done')
    train_data = generate_tf_data(X_train_seq, y_train_seq, y_train_baseline, batch_size)
    return train_data, features_list, x_mean, x_std, y_mean, y_std

print('here')
df = pd.read_csv('/content/gdrive/MyDrive/playground/ethusdt_data.csv', index_col=0)
batch_size = 128
lr = 1e-3
optimizer = 'Adam'
n_layers = 3
n_units = 32
dropout = 0.2
weight_decay = 1e-4
train_data, features_list, x_mean, x_std, y_mean, y_std = preprocess_df(df, batch_size)

if optimizer == 'Adam':
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
elif optimizer == 'SGD':
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=500, decay_rate=0.9,staircase=True)  
    opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

lstm_model = tf.keras.models.Sequential()

if n_layers == 1:
    lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
else:
    for n in range(n_layers):
        if n == n_layers - 1:
            lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
            #lstm_model.add(batchNormalization())
            lstm_model.add(tf.keras.layers.Dropout(dropout))
        else:
            lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=True))
            lstm_model.add(BatchNormalization())
            lstm_model.add(tf.keras.layers.Dropout(dropout))
            
lstm_model.add(tf.keras.layers.Dense(units=n_units, kernel_regularizer=regularizers.l2(l2=weight_decay)))

MAX_EPOCHS = 10
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, mode='min')
lstm_model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=opt, metrics=[tf.metrics.MeanAbsoluteError(), tf.keras.losses.MeanSquaredError()])
# history = lstm_model.fit(train_data, epochs=MAX_EPOCHS, validation_data=validation_data, callbacks=[early_stopping, WandbCallback()])
history = lstm_model.fit(train_data, epochs=MAX_EPOCHS, callbacks=[early_stopping])

here


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Only Selecting the last 2 years of data
Time preprocessing done
Target preprocessing done
Last preprocessing done
Deviations preprocessing done
Rolling preprocessing done
Logs preprocessing done
Splits done
Standardization done
(949960, 8, 102)
Inputs done
(949960, 32, 1)
Outputs done
(949960, 32, 1)
Baseline done
Epoch 1/10
7422/7422 [==============================] - 79s 10ms/step - loss: 3.8505e-04 - mean_absolute_error: 0.0052 - mean_squared_error: 2.0227e-04
Epoch 2/10
7422/7422 [==============================] - 73s 10ms/step - loss: 6.7752e-05 - mean_absolute_error: 0.0039 - mean_squared_error: 6.7287e-05
Epoch 3/10
7422/7422 [==============================] - 74s 10ms/step - loss: 5.8439e-05 - mean_absolute_error: 0.0036 - mean_squared_error: 5.7585e-05
Epoch 4/10
7422/7422 [==============================] - 74s 10ms/step - loss: 5.2295e-05 - mean_absolute_error: 0.0034 - mean_squared_error: 5.1063e-05
Epoch 5/10
7422/7422 [==============================] - 74s 10ms/step - loss

In [44]:
# !rm -r /gdrive/MyDrive/playground/LSTM_ETHUSDT_12-12-2021/

In [45]:
import json

data = {}

data['validation_score'] = history1.history['val_loss'][-1]
data['unique_id'] = time.time()
data['training_time_stamp'] = time.time()
data['hyperparameters'] = {'batch_size': 128, 'lr': 1e-3, 'optimizer': 'Adam', 'n_layers': 3, 'n_units': 8, 'dropout': 0.2, 'weight_decay': 1e-4}
data['symbol'] = 'ETHUSDT'
data['x_mean'] = x_mean.values.tolist()
data['x_std'] = x_std.values.tolist()
data['y_mean'] = y_mean.values[0]
data['y_std'] = y_std.values[0]

with open('model_metrics_ETHUSDT.json', 'w') as outfile:
    json.dump(data, outfile)

In [46]:
!mkdir /content/gdrive/MyDrive/playground/LSTM_ETHUSDT_$(date +"%d-%m-%Y")

In [47]:
!cp /content/model_metrics_ETHUSDT.json /content/gdrive/MyDrive/playground/LSTM_ETHUSDT_13-12-2021/

In [48]:
lstm_model.save_weights('/content/gdrive/MyDrive/playground/LSTM_ETHUSDT_13-12-2021/lstm_ETHUSDT')

In [49]:
!gsutil -m cp -r /content/gdrive/MyDrive/playground/LSTM_ETHUSDT_13-12-2021/ gs://crypto-forecasting-bucket/ETHUSDT/Model/

Copying file:///content/gdrive/MyDrive/playground/LSTM_ETHUSDT_13-12-2021/model_metrics_ETHUSDT.json [Content-Type=application/json]...
Copying file:///content/gdrive/MyDrive/playground/LSTM_ETHUSDT_13-12-2021/lstm_ETHUSDT.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file:///content/gdrive/MyDrive/playground/LSTM_ETHUSDT_13-12-2021/lstm_ETHUSDT.index [Content-Type=application/octet-stream]...
Copying file:///content/gdrive/MyDrive/playground/LSTM_ETHUSDT_13-12-2021/checkpoint [Content-Type=application/octet-stream]...
/ [4/4 files][427.6 KiB/427.6 KiB] 100% Done                                    
Operation completed over 4 objects/427.6 KiB.                                    


# CODE FOR REAL-TIME PREDICTIONS

In [ ]:
import os

data_folder = '../Data'
file_name = 'btcusdt_exchange_full.csv'

location_data = os.path.join(data_folder, file_name)

exclude_x = ['NA', 'Open Time', 'Close Time', 'y', 'Date']

val_split, test_split = None, None

log_transformation_feat = ['Open Price', 'High price', 'Low Price', 'Close Price', 'Volume Traded',
                           'Quote asset Volume', 'Number of Trades', 'Taker buy base asset volume',
                           'Taker buy quote asset volume', 'Mid Price', 'Open Price_prev', 'Close Price_prev',
                           'Mid Price_prev', 'high_low', 'high_close', 'high_open', 'open_low', 'spread',
                           'rolling_avg_high_low_1h', 'rolling_avg_high_low_1d', 'rolling_avg_high_low_1w',
                           'rolling_avg_high_low_1m', 'rolling_max_high_low_1h',
                           'rolling_max_high_low_1d', 'rolling_max_high_low_1w', 'rolling_max_high_low_1m',
                           'rolling_avg_high_close_1h', 'rolling_avg_high_close_1d', 'rolling_avg_high_close_1w',
                           'rolling_avg_high_close_1m', 'rolling_max_high_close_1h', 'rolling_max_high_close_1d',
                           'rolling_max_high_close_1w', 'rolling_max_high_close_1m', 'rolling_avg_high_open_1h',
                           'rolling_avg_high_open_1d', 'rolling_avg_high_open_1w', 'rolling_avg_high_open_1m',
                           'rolling_max_high_open_1h', 'rolling_max_high_open_1d',
                           'rolling_max_high_open_1w', 'rolling_max_high_open_1m', 'rolling_avg_open_low_1h',
                           'rolling_avg_open_low_1d',
                           'rolling_avg_open_low_1w', 'rolling_avg_open_low_1m', 'rolling_max_open_low_1h',
                           'rolling_max_open_low_1d',
                           'rolling_max_open_low_1w', 'rolling_max_open_low_1m', 'rolling_avg_spread_1h',
                           'rolling_avg_spread_1d',
                           'rolling_avg_spread_1w', 'rolling_avg_spread_1m', 'rolling_max_spread_1h',
                           'rolling_max_spread_1d', 'rolling_max_spread_1w',
                           'rolling_max_spread_1m', 'rolling_avg_Open Price_1h', 'rolling_avg_Open Price_1d',
                           'rolling_avg_Open Price_1w', 'rolling_avg_Open Price_1m',
                           'rolling_max_Open Price_1h', 'rolling_max_Open Price_1d', 'rolling_max_Open Price_1w',
                           'rolling_max_Open Price_1m',
                           'rolling_avg_High price_1h', 'rolling_avg_High price_1d', 'rolling_avg_High price_1w',
                           'rolling_avg_High price_1m',
                           'rolling_max_High price_1h', 'rolling_max_High price_1d', 'rolling_max_High price_1w',
                           'rolling_max_High price_1m',
                           'rolling_avg_Low Price_1h', 'rolling_avg_Low Price_1d', 'rolling_avg_Low Price_1w',
                           'rolling_avg_Low Price_1m',
                           'rolling_max_Low Price_1h', 'rolling_max_Low Price_1d', 'rolling_max_Low Price_1w',
                           'rolling_max_Low Price_1m',
                           'rolling_avg_Close Price_1h', 'rolling_avg_Close Price_1d', 'rolling_avg_Close Price_1w',
                           'rolling_avg_Close Price_1m',
                           'rolling_max_Close Price_1h', 'rolling_max_Close Price_1d', 'rolling_max_Close Price_1w',
                           'rolling_max_Close Price_1m']
# features where we apply log transformation

continuous_features = ['Open Price', 'High price', 'Low Price', 'Close Price', 'Volume Traded',
                       'Quote asset Volume', 'Number of Trades', 'Taker buy base asset volume',
                       'Taker buy quote asset volume', 'Mid Price', 'Open Price_prev', 'Close Price_prev',
                       'Mid Price_prev', 'high_low', 'high_close', 'high_open', 'open_low', 'spread',
                       'rolling_avg_high_low_1h', 'rolling_avg_high_low_1d', 'rolling_avg_high_low_1w',
                       'rolling_avg_high_low_1m', 'rolling_max_high_low_1h',
                       'rolling_max_high_low_1d', 'rolling_max_high_low_1w', 'rolling_max_high_low_1m',
                       'rolling_min_high_low_1h', 'rolling_min_high_low_1d',
                       'rolling_min_high_low_1w', 'rolling_min_high_low_1m', 'rolling_avg_high_close_1h',
                       'rolling_avg_high_close_1d', 'rolling_avg_high_close_1w', 'rolling_avg_high_close_1m',
                       'rolling_max_high_close_1h', 'rolling_max_high_close_1d', 'rolling_max_high_close_1w',
                       'rolling_max_high_close_1m', 'rolling_min_high_close_1h', 'rolling_min_high_close_1d',
                       'rolling_min_high_close_1w', 'rolling_min_high_close_1m', 'rolling_avg_high_open_1h',
                       'rolling_avg_high_open_1d', 'rolling_avg_high_open_1w', 'rolling_avg_high_open_1m',
                       'rolling_max_high_open_1h', 'rolling_max_high_open_1d',
                       'rolling_max_high_open_1w', 'rolling_max_high_open_1m', 'rolling_min_high_open_1h',
                       'rolling_min_high_open_1d',
                       'rolling_min_high_open_1w', 'rolling_min_high_open_1m', 'rolling_avg_open_low_1h',
                       'rolling_avg_open_low_1d',
                       'rolling_avg_open_low_1w', 'rolling_avg_open_low_1m', 'rolling_max_open_low_1h',
                       'rolling_max_open_low_1d',
                       'rolling_max_open_low_1w', 'rolling_max_open_low_1m', 'rolling_min_open_low_1h',
                       'rolling_min_open_low_1d',
                       'rolling_min_open_low_1w', 'rolling_min_open_low_1m', 'rolling_avg_spread_1h',
                       'rolling_avg_spread_1d',
                       'rolling_avg_spread_1w', 'rolling_avg_spread_1m', 'rolling_max_spread_1h',
                       'rolling_max_spread_1d', 'rolling_max_spread_1w',
                       'rolling_max_spread_1m', 'rolling_min_spread_1h', 'rolling_min_spread_1d',
                       'rolling_min_spread_1w', 'rolling_min_spread_1m',
                       'rolling_avg_Open Price_1h', 'rolling_avg_Open Price_1d', 'rolling_avg_Open Price_1w',
                       'rolling_avg_Open Price_1m',
                       'rolling_max_Open Price_1h', 'rolling_max_Open Price_1d', 'rolling_max_Open Price_1w',
                       'rolling_max_Open Price_1m',
                       'rolling_min_Open Price_1h', 'rolling_min_Open Price_1d', 'rolling_min_Open Price_1w',
                       'rolling_min_Open Price_1m',
                       'rolling_avg_High price_1h', 'rolling_avg_High price_1d', 'rolling_avg_High price_1w',
                       'rolling_avg_High price_1m',
                       'rolling_max_High price_1h', 'rolling_max_High price_1d', 'rolling_max_High price_1w',
                       'rolling_max_High price_1m',
                       'rolling_min_High price_1h', 'rolling_min_High price_1d', 'rolling_min_High price_1w',
                       'rolling_min_High price_1m',
                       'rolling_avg_Low Price_1h', 'rolling_avg_Low Price_1d', 'rolling_avg_Low Price_1w',
                       'rolling_avg_Low Price_1m',
                       'rolling_max_Low Price_1h', 'rolling_max_Low Price_1d', 'rolling_max_Low Price_1w',
                       'rolling_max_Low Price_1m',
                       'rolling_min_Low Price_1h', 'rolling_min_Low Price_1d', 'rolling_min_Low Price_1w',
                       'rolling_min_Low Price_1m',
                       'rolling_avg_Close Price_1h', 'rolling_avg_Close Price_1d', 'rolling_avg_Close Price_1w',
                       'rolling_avg_Close Price_1m',
                       'rolling_max_Close Price_1h', 'rolling_max_Close Price_1d', 'rolling_max_Close Price_1w',
                       'rolling_max_Close Price_1m',
                       'rolling_min_Close Price_1h', 'rolling_min_Close Price_1d', 'rolling_min_Close Price_1w',
                       'rolling_min_Close Price_1m']

In [ ]:
import tensorflow as tf
#from preprocessing_pipeline import preprocess_df
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm
import matplotlib.pyplot as plt
from parameters_modeling import exclude_x, log_transformation_feat, continuous_features, val_split, test_split, location_data
#import wandb
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
#from wandb.keras import WandbCallback
import tensorflow as tf


AUTOTUNE = tf.data.experimental.AUTOTUNE


def time_features_func(df):
    df['Date'] = pd.to_datetime(df['Open Time'], unit = 'ms')
    df['Year'] = pd.DatetimeIndex(df['Date']).year
    df['Month'] = pd.DatetimeIndex(df['Date']).month
    df['Day of Week'] = pd.DatetimeIndex(df['Date']).dayofweek
    df['Day of Month'] = pd.DatetimeIndex(df['Date']).day
    df['Day of Year'] = pd.DatetimeIndex(df['Date']).dayofyear
    df['Week of Month'] = pd.DatetimeIndex(df['Date']).day // 7
    df['Week of Year'] = df['Date'].dt.isocalendar().week.astype('int64')
    df['hour'] =df.Date.dt.hour
    df['minute'] =df.Date.dt.minute
    df['minute_of_day'] = df.hour*60 + df.minute
    print('Time preprocessing done')
    return df

def create_target_variables(df):
    # df = df.drop('Unnamed: 0', axis=1)
    df['Mid Price'] = (df['Open Price'] + df['Close Price'])/2
    df['y'] = (df['Open Price'].shift(-1) + df['Close Price'].shift(-1))/2  # the target is predicting the next close price
    df['benchmark'] = df['Mid Price']
    print('Target preprocessing done')
    return df


def create_last_reporting(df, columns=['Open Price', 'Close Price', 'Mid Price']):
    """Twice periods previously w.r.t the y variable"""
    for column in columns:
        df[column+'_prev'] = df[column].shift(1)
    df = df.dropna()
    print('Last preprocessing done')
    return df

def create_deviations(df):
    df['high_low'] = df['High price'] - df['Low Price']
    df['high_close'] = df['High price'] - df['Close Price']
    df['high_open'] = df['High price'] - df['Open Price']
    df['open_low'] = df['Close Price'] - df['Low Price']
    df['spread'] =df['Close Price'] - df['Open Price']
    df['spread_ind'] = 1*(df['spread'] < 0)
    df['spread'] =np.abs(df.spread)
    print('Deviations preprocessing done')
    return df

def create_stats(df, features = ['high_low', 'high_close', 'high_open', 'open_low', 'spread', 'Open Price', 'High price', 'Low Price', 'Close Price']):
    for feature in features:
        series_1h, series_1d, series_1w, series_1m =  df[feature].rolling(window = 60), df[feature].rolling(window = 1500), df[feature].rolling(window = 10000), df[feature].rolling(window = 50000)
        df['rolling_avg_{}_1h'.format(feature)] = series_1d.mean() # rolling avg over 1 hour
        df['rolling_avg_{}_1d'.format(feature)] = series_1d.mean() # rolling avg over 1 day
        df['rolling_avg_{}_1w'.format(feature)] = series_1w.mean() # rolling avg over 1 week
        df['rolling_avg_{}_1m'.format(feature)] = series_1m.mean() # rolling avg over 1 month
        df['rolling_max_{}_1h'.format(feature)] = series_1d.max() # rolling max over 1 hour
        df['rolling_max_{}_1d'.format(feature)] = series_1d.max() # rolling max over 1 day
        df['rolling_max_{}_1w'.format(feature)] = series_1w.max() # rolling max over 1 week
        df['rolling_max_{}_1m'.format(feature)] = series_1m.max()  # rolling max over 1 month
        #df['rolling_min_{}_1h'.format(feature)] = series_1d.min() # rolling min over 1 hour
        #df['rolling_min_{}_1d'.format(feature)] = series_1d.min() # rolling min over 1 day
        #df['rolling_min_{}_1w'.format(feature)] = series_1w.min() # rolling min over 1 week
        #df['rolling_min_{}_1m'.format(feature)] = series_1m.min() # rolling min over 1 month
    df = df.dropna()
    print('Rolling preprocessing done')
    return df

def log_transformation(df, log_transformation_features=log_transformation_feat):
    for feature in log_transformation_features:
        df['log_{}'.format(feature)] = np.log(1+df[feature].values)
        df = df.drop(feature, axis=1)
    print('Logs preprocessing done')
    return df

def _split(df, train_val_date=val_split, val_test_date=test_split, exclude_x=exclude_x):
    # Standardize the dataframe
    df = df.reset_index(drop=True)
    features_list = [col for col in df.columns if col not in exclude_x]
    X = df[features_list]
    # y_train, y_val, y_test = df_train[['y']], df_val[['y']], df_test[['y']]
    print('Splits done')
    return X, features_list

def standardize(X, x_mean, x_std):
    # x_mean = X.mean()
    # x_std = X.std()
    X = (X - x_mean) / x_std
    print('Standardization done')
    return X

def generate_input_sequences(data, feats, seq_len = 32):
    empty_np_array = np.zeros((len(data)-seq_len+1, seq_len, len(feats)))
    dfvs = data[feats].values
    for i in range(len(data)-seq_len+1):
      x = np.expand_dims(dfvs[i:i+seq_len, :], axis = 0)
      empty_np_array[i] = x
    return empty_np_array

def generate_tf_data(X, batch_size):
    X = X[-128:,:,:]
    print(X.shape)
    X = tf.convert_to_tensor(X, dtype=tf.float32)
    input_data = tf.data.Dataset.from_tensor_slices(X)
    input_data = input_data.batch(batch_size)
    input_data = input_data.prefetch(AUTOTUNE)

    return input_data

def preprocess_df(df, batch_size, location_data=location_data):
    df = select_last_year(df)
    df = time_features_func(df)
    df = create_target_variables(df)
    df = create_last_reporting(df)
    df = create_deviations(df)
    df = create_stats(df)
    # display(df.head())
    df = log_transformation(df)
    # display(df.head())
    X, features_list = _split(df)
    X = standardize(X, x_mean, x_std)
    X_seq = generate_input_sequences(X, X.columns.values.tolist(), 32)
    print(X_seq.shape)
    print('Inputs done')
    input_data = generate_tf_data(X_seq, batch_size)
    return input_data, features_list

def create_model():
    batch_size = 128
    lr = 1e-3
    optimizer = 'Adam'
    n_layers = 3
    n_units = 32
    dropout = 0.2
    weight_decay = 1e-4
    if optimizer == 'Adam':
        opt = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'SGD':
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=500, decay_rate=0.9,staircase=True)  
        opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

    lstm_model = tf.keras.models.Sequential()

    if n_layers == 1:
        lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
    else:
        for n in range(n_layers):
            if n == n_layers - 1:
                lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
                #lstm_model.add(batchNormalization())
                lstm_model.add(tf.keras.layers.Dropout(dropout))
            else:
                lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=True))
                lstm_model.add(BatchNormalization())
                lstm_model.add(tf.keras.layers.Dropout(dropout))
                
    lstm_model.add(tf.keras.layers.Dense(units=n_units, kernel_regularizer=regularizers.l2(l2=weight_decay)))

    MAX_EPOCHS = 10
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, mode='min')
    lstm_model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=opt, metrics=[tf.metrics.MeanAbsoluteError(), tf.keras.losses.MeanSquaredError()])
    return lstm_model


if __name__ == '__main__':
    batch_size = 128

    trial_lst_dict = [
    {
        "id": 1228,
        "name": "BTCUSDT",
        "open_time": 1502974920000,
        "open_price": 4411,
        "high_price": 4411,
        "low_price": 4411,
        "close_price": 4411,
        "volume_traded": 0,
        "close_time": 1502974979999,
        "quote_asset_volume": 0,
        "number_of_trades": 0,
        "taker_buy_base_asset_volume": 0,
        "taker_buy_quote_asset_volume": 0
    },
    {
        "id": 163,
        "name": "BTCUSDT",
        "open_time": 1502942940000,
        "open_price": 4261.48,
        "high_price": 4261.48,
        "low_price": 4261.48,
        "close_price": 4261.48,
        "volume_traded": 0,
        "close_time": 1502942999999,
        "quote_asset_volume": 0,
        "number_of_trades": 0,
        "taker_buy_base_asset_volume": 0,
        "taker_buy_quote_asset_volume": 0
    }
    ]

    trial_lst_dict = trial_lst_dict*25500

    trial_df = pd.DataFrame(trial_lst_dict)
    trial_df = trial_df.drop(['id', 'name'], axis = 1)
    trial_df = trial_df.rename(columns = {'open_time': 'Open Time', 'open_price': 'Open Price', 'high_price': 'High price', 'low_price': 'Low Price', 'close_price': 'Close Price', 'volume_traded': 'Volume Traded', 'close_time': 'Close Time', 'quote_asset_volume': 'Quote asset Volume', 'number_of_trades': 'Number of Trades', 'taker_buy_base_asset_volume': 'Taker buy base asset volume', 'taker_buy_quote_asset_volume': 'Taker buy quote asset volume'})
    input_data, features_list = preprocess_df(trial_df, batch_size)

    model = create_model()
    model.load_weights('/content/gdrive/MyDrive/playground/LSTM_BTCUSDT_12-12-2021/lstm_BTCUSDT')

    predictions = model.predict(input_data)
    to_return = predictions[-1]
    

In [ ]:
def create_model():
    batch_size = 128
    lr = 1e-3
    optimizer = 'Adam'
    n_layers = 3
    n_units = 32
    dropout = 0.2
    weight_decay = 1e-4
    if optimizer == 'Adam':
        opt = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'SGD':
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=500, decay_rate=0.9,staircase=True)  
        opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

    lstm_model = tf.keras.models.Sequential()

    if n_layers == 1:
        lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
    else:
        for n in range(n_layers):
            if n == n_layers - 1:
                lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=False))
                #lstm_model.add(batchNormalization())
                lstm_model.add(tf.keras.layers.Dropout(dropout))
            else:
                lstm_model.add(tf.keras.layers.LSTM(n_units, return_sequences=True))
                lstm_model.add(BatchNormalization())
                lstm_model.add(tf.keras.layers.Dropout(dropout))
                
    lstm_model.add(tf.keras.layers.Dense(units=n_units, kernel_regularizer=regularizers.l2(l2=weight_decay)))

    MAX_EPOCHS = 10
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, mode='min')
    lstm_model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=opt, metrics=[tf.metrics.MeanAbsoluteError(), tf.keras.losses.MeanSquaredError()])
    return lstm_model

In [ ]:
model = create_model()
model.load_weights('/content/gdrive/MyDrive/playground/LSTM_BTCUSDT_12-12-2021/lstm_BTCUSDT')

In [ ]:
trial_lst_dict = [
    {
        "id": 1228,
        "name": "BTCUSDT",
        "open_time": 1502974920000,
        "open_price": 4411,
        "high_price": 4411,
        "low_price": 4411,
        "close_price": 4411,
        "volume_traded": 0,
        "close_time": 1502974979999,
        "quote_asset_volume": 0,
        "number_of_trades": 0,
        "taker_buy_base_asset_volume": 0,
        "taker_buy_quote_asset_volume": 0
    },
    {
        "id": 163,
        "name": "BTCUSDT",
        "open_time": 1502942940000,
        "open_price": 4261.48,
        "high_price": 4261.48,
        "low_price": 4261.48,
        "close_price": 4261.48,
        "volume_traded": 0,
        "close_time": 1502942999999,
        "quote_asset_volume": 0,
        "number_of_trades": 0,
        "taker_buy_base_asset_volume": 0,
        "taker_buy_quote_asset_volume": 0
    }
    ]

trial_lst_dict = trial_lst_dict*25500

In [ ]:
trial_df = pd.DataFrame(trial_lst_dict)

,id,name,open_time,open_price,high_price,low_price,close_price,volume_traded,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume
50995,163,BTCUSDT,1502942940000,4261.48,4261.48,4261.48,4261.48,0,1502942999999,0,0,0,0
50996,1228,BTCUSDT,1502974920000,4411.00,4411.00,4411.00,4411.00,0,1502974979999,0,0,0,0
50997,163,BTCUSDT,1502942940000,4261.48,4261.48,4261.48,4261.48,0,1502942999999,0,0,0,0
50998,1228,BTCUSDT,1502974920000,4411.00,4411.00,4411.00,4411.00,0,1502974979999,0,0,0,0
50999,163,BTCUSDT,1502942940000,4261.48,4261.48,4261.48,4261.48,0,1502942999999,0,0,0,0


In [ ]:
trial_df = trial_df.drop(['id', 'name'], axis = 1)

In [ ]:
trial_df = trial_df.rename(columns = {'open_time': 'Open Time', 'open_price': 'Open Price', 'high_price': 'High price', 'low_price': 'Low Price', 'close_price': 'Close Price', 'volume_traded': 'Volume Traded', 'close_time': 'Close Time', 'quote_asset_volume': 'Quote asset Volume', 'number_of_trades': 'Number of Trades', 'taker_buy_base_asset_volume': 'Taker buy base asset volume', 'taker_buy_quote_asset_volume': 'Taker buy quote asset volume'})

In [ ]:
import tensorflow as tf
#from preprocessing_pipeline import preprocess_df
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm
import matplotlib.pyplot as plt
#from parameters_modeling import exclude_x, log_transformation_feat, continuous_features, val_split, test_split, location_data
#import wandb
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
#from wandb.keras import WandbCallback
import tensorflow as tf
batch_size = 128


AUTOTUNE = tf.data.experimental.AUTOTUNE

# def select_last_year(df):
#     df = df.sort_values('Open Time')
#     df = df.iloc[:100000, :]
#     df = df.reset_index(drop=True)
#     print('Only Selecting the first 100000')
#     return df

def time_features_func(df):
    df['Date'] = pd.to_datetime(df['Open Time'], unit = 'ms')
    df['Year'] = pd.DatetimeIndex(df['Date']).year
    df['Month'] = pd.DatetimeIndex(df['Date']).month
    df['Day of Week'] = pd.DatetimeIndex(df['Date']).dayofweek
    df['Day of Month'] = pd.DatetimeIndex(df['Date']).day
    df['Day of Year'] = pd.DatetimeIndex(df['Date']).dayofyear
    df['Week of Month'] = pd.DatetimeIndex(df['Date']).day // 7
    df['Week of Year'] = df['Date'].dt.isocalendar().week.astype('int64')
    df['hour'] =df.Date.dt.hour
    df['minute'] =df.Date.dt.minute
    df['minute_of_day'] = df.hour*60 + df.minute
    print('Time preprocessing done')
    return df

def create_target_variables(df):
    # df = df.drop('Unnamed: 0', axis=1)
    df['Mid Price'] = (df['Open Price'] + df['Close Price'])/2
    df['y'] = (df['Open Price'].shift(-1) + df['Close Price'].shift(-1))/2  # the target is predicting the next close price
    df['benchmark'] = df['Mid Price']
    print('Target preprocessing done')
    return df


def create_last_reporting(df, columns=['Open Price', 'Close Price', 'Mid Price']):
    """Twice periods previously w.r.t the y variable"""
    for column in columns:
        df[column+'_prev'] = df[column].shift(1)
    df = df.dropna()
    print('Last preprocessing done')
    return df

def create_deviations(df):
    df['high_low'] = df['High price'] - df['Low Price']
    df['high_close'] = df['High price'] - df['Close Price']
    df['high_open'] = df['High price'] - df['Open Price']
    df['open_low'] = df['Close Price'] - df['Low Price']
    df['spread'] =df['Close Price'] - df['Open Price']
    df['spread_ind'] = 1*(df['spread'] < 0)
    df['spread'] =np.abs(df.spread)
    print('Deviations preprocessing done')
    return df

def create_stats(df, features = ['high_low', 'high_close', 'high_open', 'open_low', 'spread', 'Open Price', 'High price', 'Low Price', 'Close Price']):
    for feature in features:
        series_1h, series_1d, series_1w, series_1m =  df[feature].rolling(window = 60), df[feature].rolling(window = 1500), df[feature].rolling(window = 10000), df[feature].rolling(window = 50000)
        df['rolling_avg_{}_1h'.format(feature)] = series_1d.mean() # rolling avg over 1 hour
        df['rolling_avg_{}_1d'.format(feature)] = series_1d.mean() # rolling avg over 1 day
        df['rolling_avg_{}_1w'.format(feature)] = series_1w.mean() # rolling avg over 1 week
        df['rolling_avg_{}_1m'.format(feature)] = series_1m.mean() # rolling avg over 1 month
        df['rolling_max_{}_1h'.format(feature)] = series_1d.max() # rolling max over 1 hour
        df['rolling_max_{}_1d'.format(feature)] = series_1d.max() # rolling max over 1 day
        df['rolling_max_{}_1w'.format(feature)] = series_1w.max() # rolling max over 1 week
        df['rolling_max_{}_1m'.format(feature)] = series_1m.max()  # rolling max over 1 month
        #df['rolling_min_{}_1h'.format(feature)] = series_1d.min() # rolling min over 1 hour
        #df['rolling_min_{}_1d'.format(feature)] = series_1d.min() # rolling min over 1 day
        #df['rolling_min_{}_1w'.format(feature)] = series_1w.min() # rolling min over 1 week
        #df['rolling_min_{}_1m'.format(feature)] = series_1m.min() # rolling min over 1 month
    df = df.dropna()
    print('Rolling preprocessing done')
    return df

def log_transformation(df, log_transformation_features=log_transformation_feat):
    for feature in log_transformation_features:
        df['log_{}'.format(feature)] = np.log(1+df[feature].values)
        df = df.drop(feature, axis=1)
    print('Logs preprocessing done')
    return df

def _split(df, train_val_date=val_split, val_test_date=test_split, exclude_x=exclude_x):
    # Standardize the dataframe
    df = df.reset_index(drop=True)
    features_list = [col for col in df.columns if col not in exclude_x]
    X = df[features_list]
    # y_train, y_val, y_test = df_train[['y']], df_val[['y']], df_test[['y']]
    print('Splits done')
    return X, features_list

def standardize(X, x_mean, x_std):
    # x_mean = X.mean()
    # x_std = X.std()
    X = (X - x_mean) / x_std
    print('Standardization done')
    return X

def generate_input_sequences(data, feats, seq_len = 32):
    empty_np_array = np.zeros((len(data)-seq_len+1, seq_len, len(feats)))
    dfvs = data[feats].values
    for i in range(len(data)-seq_len+1):
      x = np.expand_dims(dfvs[i:i+seq_len, :], axis = 0)
      empty_np_array[i] = x
    return empty_np_array

def generate_tf_data(X, batch_size):
    X = X[-128:,:,:]
    print(X.shape)
    X = tf.convert_to_tensor(X, dtype=tf.float32)
    input_data = tf.data.Dataset.from_tensor_slices(X)
    input_data = input_data.batch(batch_size)
    input_data = input_data.prefetch(AUTOTUNE)

    return input_data

def preprocess_df(df, batch_size, location_data=location_data):
    df = select_last_year(df)
    df = time_features_func(df)
    df = create_target_variables(df)
    df = create_last_reporting(df)
    df = create_deviations(df)
    df = create_stats(df)
    # display(df.head())
    df = log_transformation(df)
    # display(df.head())
    X, features_list = _split(df)
    #return X, features_list
    X = standardize(X, x_mean, x_std)
    X_seq = generate_input_sequences(X, X.columns.values.tolist(), 32)
    print(X_seq.shape)
    print('Inputs done')
    input_data = generate_tf_data(X_seq, batch_size)
    return input_data, features_list

input_data, features_list = preprocess_df(trial_df, batch_size)

Only Selecting the last 2 years of data
Time preprocessing done
Target preprocessing done
Last preprocessing done
Deviations preprocessing done
Rolling preprocessing done
Logs preprocessing done
Splits done
Standardization done


In [ ]:
prediction = model.predict(input_data)

In [ ]:
prediction[-1]

array([-9.8405553e-07, -5.4230331e-04, -4.8589339e-03, -4.5083687e-03,
       -3.2698584e-03, -3.5638101e-03, -4.9586757e-03, -6.6472623e-03,
       -8.5288947e-03, -1.0736486e-02, -1.2732004e-02, -1.4015911e-02,
       -1.5037212e-02, -1.6451491e-02, -1.7964197e-02, -1.9437551e-02,
       -2.0654945e-02, -2.1966469e-02, -2.3575917e-02, -2.5259724e-02,
       -2.6925469e-02, -2.8193306e-02, -2.9496372e-02, -3.0684480e-02,
       -3.1617083e-02, -3.2167535e-02, -3.2662116e-02, -3.3443663e-02,
       -3.4581617e-02, -3.5566438e-02, -3.6236998e-02, -3.7210621e-02],
      dtype=float32)

# END OF NOTEBOOK